### Código de Exploración de Datos Multivariados de los estudiantes Milessi Ayrton y Torres Santiago

In [ ]:
# -*- coding: utf-8 -*-

***Cargamos las librerias a utilizar, la base de datos y funciones a utilizar***

In [ ]:
library(dplyr)
library(readxl)
library(stringr)
library(writexl)
library(ggplot2)
library('pwr')
library(reshape2)
library(car)
library(ppcor)
library(corrplot)
library(tidyr)
library(fdth)
library("moments")
library(plotly)
library("DescTools")
library("gmodels")
library(rstatix)
library('DMwR2') 
library("MASS")
library("lattice")
library("grid")
library(scatterplot3d)
library(Hotelling)
library(rstatix)
library(GGally)

***Funciones que utilizaremos a lo largo del análisis***

In [ ]:
# Función para calcular la moda
Mode <- function(x) {
  ux <- unique(x)
  ux[which.max(tabulate(match(x, ux)))]
}

In [ ]:
# Función para evaluar correlación
evaluar_correlacion <- function(spe) {
  if (spe >= 0.8) {
    return("Correlación muy alta")
  } else if (spe >= 0.6) {
    return("Correlación alta")
  } else if (spe >= 0.4) {
    return("Correlación moderada")
  } else if (spe >= 0.2) {
    return("Correlación baja")
  } else {
    return("No hay correlación")
  }
}

In [ ]:
# Función para calcular la tendencia de las notas a lo largo del año para cada alumno
calcular_tendencia <- function(data, primer_tri, segundo_tri, tercer_tri) {
  data$Tendencia <- apply(data[, c(primer_tri, segundo_tri, tercer_tri)], 1, function(x) {
    if (is.na(x[1]) | is.na(x[2]) | is.na(x[3])) return(NA)
    if (x[3] > x[2] & x[2] > x[1]) {
      return("Creciente")
    } else if (x[3] > x[2] & x[2] == x[1]) {
      return("Creciente")
    } else if (x[3] == x[2] & x[2] > x[1]) {
      return("Creciente")
    } else if (x[3] < x[2] & x[2] < x[1]) {
      return("Decreciente")
    } else if (x[3] < x[2] & x[2] == x[1]) {
      return("Decreciente")
    } else if (x[3] == x[2] & x[2] < x[1]) {
      return("Decreciente")
    } else {
      return("Estable")
    }
  })
  return(data)
}

In [ ]:
# Función para estimar la edad basada en el modelo de predicción lineal
estimar_edad <- function(dni) {
  edad_estimada <- intercept + pendiente * dni
  edad_estimada <- round(edad_estimada)
  return(edad_estimada)
}

In [ ]:
#Función de Mardia para la dsitribución de normalidad multivariada
mardia2<-function(datos){
  n<-nrow(datos)
  k<-ncol(datos)

  x<-as.matrix(datos)
  media<-colMeans(datos)
  matriz.media<-matrix(media,nrow=n,ncol=k,byrow=TRUE)

  S<-cov(datos)*(n-1)/n
  SI<-solve(S)   # Inversa de la matriz de covarianzas

  distm<-((x-matriz.media)%*%SI)%*%t((x-matriz.media))

  gl<-k*(k+1)*(k+2)/6

  # Medida de Asimetría Multivariada
  AM<-sum(distm^3)/n^2

  if (n<30){
  # Test Multivariado de Asimetría de Mardia - Muestras Pequeñas
  c <- (n+1)*(n+3)*(k+1)/(n*((n+1)*(k+1)-6))
  TAM <- n*c/6*AM
  valor_p1 <- pchisq(TAM,gl,lower.tail = FALSE)
  } else {
  # Test Multivariado de Asimetríıa de Mardia - Muestras grandes
  TAM<-n/6*AM
  valor_p1<-pchisq(TAM,gl,lower.tail = FALSE)
  }
  # Medida de Kurtosis Multivariado
  KM<-sum((diag(distm))^2)/n

  # Test Multivariado de Kurtosis de Mardia
  TKM<-(KM-k*(k+2))*sqrt(n/(8*k*(k+2)))
  TKM2<-TKM^2
  valor_p2<-pchisq(TKM2,1,lower.tail = FALSE)

  cat("Test de Normalidad Multivariada", "\n" ,
      "Ho: Los datos siguen una Distribución Normal Multivariada","\n")
  cat("----------------------------------------------------------------","\n")
  cat("Prueba de Mardia - Asimetría","\n",
      "Asimetría Multivariada =", AM, "\n",
      "Valor de TAM =",TAM, "\n",
      "Valor p=", valor_p1, "\n")
  cat("----------------------------------------------------------------","\n")
  cat("Prueba de Mardia - Curtosis","\n",
      "Curtosis Multivariado =", KM, "\n",
      "Valor de TKM =",TKM2, "\n",
      "Valor p=", valor_p2, "\n")
     }

***Abrimos nuestra base de datos***

In [ ]:
Victoria <- read.csv("C:/Users/ayrto/OneDrive/Escritorio/Exploración de Datos Multivariados/1 - Trabajo Practico/VICTORIA.csv") 

***Exploramos la base de datos***

In [ ]:
dim(Victoria) # Mostramos las dimensiones de la base de datos original
dim(filter(Victoria,NivelEnsenanza == "PRIMARIO")) # Mostramos la cantidad de datos que hay de Primaria
dim(filter(Victoria,NivelEnsenanza == "SECUNDARIO")) # Mostramos la cantidad de datos que hay de Secundaria

In [ ]:
names(Victoria) # Vemos las variables

In [ ]:
# Cambiamos los nombres de las variables para un mejor entendimiento
Victoria <- rename(Victoria, c("CodigoUnicoEscolar" = "CUE",
                               "ModEnseñanza" = "ModEnsenaza",
                               "año" = "anioLectivo",
                               "esMultiaño" ="esMultianio",
                               "IdModEnseñanza" = "IdModEnsenanza",
                               "NivelEnseñanza" = "NivelEnsenanza",
                               "Gestión"= "EsPrivada"))

In [ ]:
# Descubrimos que al intentar modificar el nombre de una varible, esta se encuentra dos veces
Victoria$ddivision <- NULL # Procedemos a eliminar una de las variables duplicada
names(Victoria) # Verificamos si se elimino correctamente

In [ ]:
# Mostramos las variables del dataset, nos ayuda a identificar el tipo de cada variable
str(Victoria)

In [ ]:
# Ahora dividiremos el dataset en dos: Primaria y Secundaria
# Nueva base de datos para primaria
Victoria_primaria <- filter(Victoria, NivelEnseñanza == "PRIMARIO" & 
                            ( ModEnseñanza == "Comun"| ModEnseñanza == "Escuelas NINA")
                            & Modalidad == "Primaria" & esMultiaño == "N")

In [ ]:
dim(Victoria_primaria) # Tamaño de las nueva base de datos 

### Ahora nos enfocaremos en la limpieza de la base de datos de primaria

In [ ]:
# Reemplazamos las notas "EPAC", "UP" y "PA" para tener un mejor análisis en los estudiantes
Victoria_primaria$nota[Victoria_primaria$nota %in% c("EPAC")] <- 5
Victoria_primaria$nota[Victoria_primaria$nota %in% c("PA")] <- 6
Victoria_primaria$nota[Victoria_primaria$nota %in% c("UP")] <- 7

#Convertimos la variable nota a tipo numérico
Victoria_primaria$nota <- suppressWarnings(as.numeric(Victoria_primaria$nota)) 

In [ ]:
# Reemplazamos los datos del tipo de gestión para un mejor entendimiento a la hora de hacer graficos o calculos
Victoria_primaria <- Victoria_primaria %>%
    mutate(Gestión = case_when(
    Gestión == "N" ~ "Pública",
    Gestión == "S" ~ "Privada"
))

In [ ]:
# Creamos nuevos data frames para almacenar las notas de cada trimestre
PrimerT_primaria <- filter(Victoria_primaria, periodoEvaluatorio == "Primer Trimestre")
PrimerT_primaria <- rename(PrimerT_primaria,NotasPrimerT = nota)
SegundoT_primaria <- filter(Victoria_primaria, periodoEvaluatorio == "Segundo Trimestre")
SegundoT_primaria <- rename(SegundoT_primaria,NotasSegundoT = nota)
TercerT_primaria <- filter(Victoria_primaria, periodoEvaluatorio == "Tercer Trimestre")
TercerT_primaria <- rename(TercerT_primaria,NotasTercerT = nota)

In [ ]:
# A traves del merge unimos las notas de cada trimestre segun el Documento de cada estudiante y la asignatura
aux <- merge(x = PrimerT_primaria, y = SegundoT_primaria, by = c("Documento", "asignatura"))
auxiliar <- merge(x = aux, y = TercerT_primaria, by = c("Documento", "asignatura"))

In [ ]:
# Organizamos las variables
Notas_Victoria_primaria <- dplyr::select(auxiliar, 'CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno' , 'curso' , 'Documento', 'asignatura','idalumno', 'NivelEnseñanza', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT','NotasSegundoT', 'NotasTercerT')

In [ ]:
# Calculamos el promedio para cada estudiante en las asignaturas que cursa
Notas_Victoria_primaria$Promedio <-round ((as.numeric(Notas_Victoria_primaria$NotasPrimerT)+as.numeric(Notas_Victoria_primaria$NotasSegundoT)+as.numeric(Notas_Victoria_primaria$NotasTercerT))/3,2)

In [ ]:
# Según el promedio conseguido tendra aprobado el año lectivo o tendrá que rendir. 
# Los alumnos que tengas una nota menor a 6 en el tercer trimestre tendrán que rendir.
# Los estudiantes que no tengan las 3 notas correspondientes a cada trimestre tendrán un registro incompleto
Notas_Victoria_primaria <- Notas_Victoria_primaria %>% 
  mutate(Condición = case_when(
    Promedio >= 6 & NotasTercerT >= 6 ~ "Aprobado", 
    Promedio < 6 | NotasTercerT < 6 ~ "Rinde",      
    TRUE ~ "sin registro completo"                    
  ))

In [ ]:
# Llamamos a la función para que calcule la tendecia de las notas a que cada estudiante segun sus notas en los diferentes trimestres
Notas_Victoria_primaria <- calcular_tendencia(Notas_Victoria_primaria, "NotasPrimerT", "NotasSegundoT", "NotasTercerT")

In [ ]:
# Nos planteamos un desafío interesante, calcular la edad según el documento de cada estudiante
# A traves de documentos de personas conocidas por nosotros y sabiendo las edades, logramos establecer un
#rango para las edades.
Notas_Victoria_primaria <- Notas_Victoria_primaria %>%
    mutate(Rango_edad = case_when(
    Documento > 56361672 & Documento <= 60000000 ~ "(5 - 6) años",
    Documento > 57000000 & Documento <= 56361672 ~ "(6 - 7) años",
    Documento > 55500000 & Documento <= 57000000 ~ "(7 - 8) años",
    Documento > 54000000 & Documento <= 55500000 ~ "(8 - 9) años",
    Documento > 52500000 & Documento <= 54000000 ~ "(9 - 10) años",
    Documento > 51000000 & Documento <= 52500000 ~ "(10 - 11) años",
    Documento > 49500000 & Documento <= 51000000 ~ "(11 - 12) años",
    Documento > 48000000 & Documento <= 49500000 ~ "(12 - 13) años",
    TRUE ~ "Fuera de rango. Este DNI puede pertenercer a una persona extranjera o esta mal cargado"
))
# Hay que aclarar que este tipo de "asignación" de edad según el documento puede tener muchos errores.
# Por eso se decidió por...

In [ ]:
# Hacer un modelo de predicción lineal, por el cual según el documento de cada estudiante podremos hacer
#una predicción de la edad de cada estudiante
# Aclaramos que se pidió la autorización correspondiente para poder utilizar estos datos sensibles como lo
#es el documento y la edad de cada persona, ya que se tratan de personas reales.
# Datos que serviran para predecir las edades segun el DNI de cada estudiante
# Los números de DNI
dnies <- c(49438652, 50732773, 51191163, 52250871, 52820485, 53111900, 53740848, 
           53702417, 53850752, 54009998, 54545454, 55191378, 55473434, 55773434, 56427707, 57010207, 
           58500000, 59500000)
# Las edades
edades <- c(14, 13, 13, 12, 12, 10, 10, 10, 9, 9, 9, 8, 8, 8, 7, 6, 6, 5)

# Armamos el modelo lineal simple, donde las edades es la variable de análisis
modelo <- lm(edades ~ dnies) # y los números de DNI es la variable explicativa

# Vemos sus salidas
modelo$coefficients # Ecuación de la recta edad =  62.5014401884189 -9.78260339940917e-07 * dnies
summary(modelo) # el modelo es significante ---> R-squared: 0.9525

# Coeficientes del modelo
intercept <- coef(modelo)[1]
pendiente <- coef(modelo)[2]

In [ ]:
# Pasamos todos los nombres de las asignaturas a mayusculas para una mayor facilidad a la hora de cambiarlos
Notas_Victoria_primaria$asignatura <- toupper(Notas_Victoria_primaria$asignatura)

In [ ]:
# Ponemos un nombre común a las asignaturas que dictan los mismos contenidos pero tienen un nombre diferente
Notas_Victoria_primaria$asignatura <- gsub("MATEMÁTICA|ACOMPAÑAMIENTO AL ESTUDIO MATEMÁTICA", "MATEMATICA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("LITERATURA|ACOMPAÑAMIENTO AL ESTUDIO LENGUA", "LENGUA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("ACOMPAÑAMIENTO AL ESTUDIO SOCIALES", "CIENCIAS SOCIALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("ACOMPAÑAMIENTO AL ESTUDIO NATURALES|TALLER VIDA EN LA NATURALEZA", "CIENCIAS NATURALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE ARTES VISUALES|TALLER DE LENGUAJE DE LAS ARTES VISUALES", "ARTES VISUALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE DEPORTE|DEPORTE", "EDUCACION FISICA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("E.D.I.", "EDI", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE MUSICA|TALLER DE FOLKLORE", "EDUCACION MUSICAL", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE TIC|TALLER DE TIC Y RECREACIÓN DE LA IMAGEN|TALLER TIC|TICS", "TIC", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("IDIOMA EXTRANJERO|INGLÉS|TALLER DE INGLES|TALLER DE INGLÉS|TALLER DE LENGUA EXTRANJERA|INGLEs|TALLER INGLES|TALLER LENGUA EXTRANJERA (INGLES)", "INGLES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE TEATRO|TALLER TEATRO", "TALLER  DE TEATRO", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("INFORMÁTICA", "INFORMATICA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER LABORATORIO", "TALLER DE LABORATORIO", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("AC.EN LENGUA Y CS. SOCIALES|T/A ESTUDIO (LENGUA-CS SOCIALES)|TALLER ACOMPAÑAMIENTO LENGUA Y CIENCIAS SOCIALES", "TALLER DE CS. SOCIALES Y LENGUA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("AC.EN MATEMÁTICA Y CS. NATURALES|T/A ESTUDIO (MATEMATICA-CS NATURALES)|TALLER ACOMPAÑAMIENTO  MATEMÁTICA Y CS. NATURALES", "TALLER DE CS. NATURALES Y MATEMATICA", Notas_Victoria_primaria$asignatura)

In [ ]:
# Verificamos que no se hayan modificado bien los nombres de las asignaturas
table(Notas_Victoria_primaria$asignatura)

In [ ]:
# No todas las asignaturas fueron modificadas correctamente, se procede a realizar denuevo el procedimiento con aquellas que falta cambiar
Notas_Victoria_primaria$asignatura <- str_replace_all(Notas_Victoria_primaria$asignatura, ".*INGLES*.", "INGLES")
Notas_Victoria_primaria$asignatura <- gsub("TALLER ACOMPAÑAMIENTO  MATEMATICA Y CS. NATURALES|T/A ESTUDIO (MATEMATICA-CS NATURALES)|AC.EN MATEMATICA Y CS. NATURALES", "TALLER DE MATEMATICA Y CS. NATURALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- str_replace_all(Notas_Victoria_primaria$asignatura, "T/A ESTUDIO \\(LENGUA-CS SOCIALES\\)", "TALLER DE CS. SOCIALES Y LENGUA")
Notas_Victoria_primaria$asignatura <- str_replace_all(Notas_Victoria_primaria$asignatura, "T/A ESTUDIO \\(MATEMATICA-CS NATURALES\\)", "TALLER DE MATEMATICA Y CS. NATURALES")

# Verificamos nuevamente
table(Notas_Victoria_primaria$asignatura)
# Ya tenemos todos los nombres de las materias correctamente

In [ ]:
# Se crearon nuevas variables según las materias de cada alumno. Como por ejemplo:
# Calcula el total de asignaturas por estudiante
Notas_Victoria_primaria <- Notas_Victoria_primaria %>%
  group_by(idalumno) %>%
  mutate(total_asig = n()) %>%
  ungroup()

# Calcula el número de asignaturas aprobadas por estudiante
Notas_Victoria_primaria <- Notas_Victoria_primaria %>%
  group_by(idalumno) %>%
  mutate(asig_aprob = sum(Promedio >= 6, na.rm = TRUE)) %>%
  ungroup()

# Calcula el número de asignaturas desaprobadas por estudiante
Notas_Victoria_primaria <- Notas_Victoria_primaria %>%
  group_by(idalumno) %>%
  mutate(asig_desaprob = sum(Promedio < 6, na.rm = TRUE)) %>%
  ungroup()

# Calcula el porcentaje de asignaturas aprobadas por estudiante
Notas_Victoria_primaria <- Notas_Victoria_primaria %>%
  mutate(porc_aprob = round((asig_aprob / total_asig) * 100,1))

# Calcula el porcentaje de asignaturas desaprobadas por estudiante
Notas_Victoria_primaria <- Notas_Victoria_primaria %>%
  mutate(porc_desaprob = round((asig_desaprob / total_asig) * 100,1))

# Mostramos el dataset con las nuevas variables
Notas_Victoria_primaria

In [ ]:
# Creamos la variable desempeño segun el promedio conseguido
Notas_Victoria_primaria <- Notas_Victoria_primaria %>%
    mutate(Desempeño = case_when(
    Promedio < 4 ~ "Insuficiente",
    Promedio >= 4 & Promedio < 6 ~ "Regular",
    Promedio >= 6 & Promedio < 7 ~ "Aprobado",
    Promedio >= 7 & Promedio < 8 ~ "Bueno",
    Promedio >= 8 & Promedio < 9 ~ "Muy Bueno",
    Promedio >= 9 & Promedio <= 10 ~ "Distinguido"))

# Creamos la variable eximido segun la cantidad de materias que tiene que rendir cada alumno
Notas_Victoria_primaria <- Notas_Victoria_primaria %>%
    mutate(Eximido = case_when(
    asig_desaprob == 0 ~ "No se llevó ninguna materia",
    asig_desaprob >= 1 & asig_desaprob <= 3 ~ "Se llevó entre 1 y 3 materias",
    asig_desaprob > 3 ~ "Se llevó más de tres materias"))

In [ ]:
# Eliminamos aquellos estudiantes que no tienen el promedio completo
Notas_Victoria_primaria <- Notas_Victoria_primaria[Notas_Victoria_primaria$Condición != "sin registro completo", ]

In [ ]:
# Eliminamos un curso multiaño que supero el filtro implementado
Notas_Victoria_primaria <- Notas_Victoria_primaria[Notas_Victoria_primaria$curso != "1-6 - A Prim", ]
table(Notas_Victoria_primaria$curso)
# Ya nos quedamos con los cursos que no son multiaños 

In [ ]:
# Cambiamos a tipo caracter los nombres de los cursos para luego quedarnos con el primer caracter que aparece
# (en cada dato, el primer caracter es el numero del grado)
Notas_Victoria_primaria$curso <- as.character(Notas_Victoria_primaria$curso)
Notas_Victoria_primaria$Año_curso <- str_sub(Notas_Victoria_primaria$curso,1, 1)

# Mostramos la cantidad de materias en cada grado
table(Notas_Victoria_primaria$Año_curso)

In [ ]:
# Modificamos la variable 'Año_curso' del número al nombre del grado
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "1" , "PRIMERO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "2" , "SEGUNDO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "3" , "TERCERO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "4" , "CUARTO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "5" , "QUINTO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "6" , "SEXTO")

In [ ]:
# Seleccionamos las variables que nos interesan para nuestro analisis
Notas_Victoria_primaria <- dplyr::select(Notas_Victoria_primaria,'CodigoUnicoEscolar','departamento','Gestión','turno','Año_curso','Documento','asignatura','idalumno','Modalidad','ModEnseñanza','NotasPrimerT','NotasSegundoT','NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', "total_asig", "porc_aprob", "porc_desaprob")

In [ ]:
# Vemos la nueva dimensión del dataset
dim(Notas_Victoria_primaria)

### Segmentamos la base de datos primaria en seis nuevas bases de datos nuevas. Cada una le pertencerá a cada grado

In [ ]:
Notas_Victoria_1Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "PRIMERO"))
Notas_Victoria_2Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "SEGUNDO"))
Notas_Victoria_3Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "TERCERO"))
Notas_Victoria_4Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "CUARTO"))
Notas_Victoria_5Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "QUINTO"))
Notas_Victoria_6Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "SEXTO"))

## Empezaremos con el analisis de la bases de datos de los grados de primaria

*Primero vamos a ir haciendo distintos analisis que describiran el estado de la base de datos en primaria*

In [ ]:
# Cantidad de escuelas primarias en Victoria
print("Cantidad de escuelas primarias en Victoria:")
length(unique(Notas_Victoria_primaria$CodigoUnicoEscolar))

# Cantidad de escuelas primarias públicas
print("Cantidad de escuelas primarias públicas en Victoria:")
length(unique(filter(Notas_Victoria_primaria, Gestión == "Pública")$CodigoUnicoEscolar))

# Cantidad de escuelas primarias privadas
print("Cantidad de escuelas primarias privadas en Victoria:")
length(unique(filter(Notas_Victoria_primaria, Gestión == "Privada")$CodigoUnicoEscolar))

# Cantidad de escuelas primarias que tienen turno mañana
print("Cantidad de escuelas primarias que tienen turno mañana en Victoria:")
length(unique(filter(Notas_Victoria_primaria, turno == "MAÑANA")$CodigoUnicoEscolar))

# Cantidad de escuelas primarias que tienen turno tarde
print("Cantidad de escuelas primarias que tienen turno tarde en Victoria:")
length(unique(filter(Notas_Victoria_primaria, turno == "TARDE")$CodigoUnicoEscolar))

# Cantidad de escuelas primarias que tienen turno completo
print("Cantidad de escuelas primarias que tienen turno completo en Victoria:")
length(unique(filter(Notas_Victoria_primaria, turno == "COMPLETO")$CodigoUnicoEscolar))

# Cantidad de escuelas primarias comun
print("Cantidad de escuelas primarias comunes en Victoria:")
length(unique(filter(Notas_Victoria_primaria, ModEnseñanza == "Comun")$CodigoUnicoEscolar))

# Cantidad de escuelas primarias que tienen turno completo
print("Cantidad de escuelas primarias NINA en Victoria:")
length(unique(filter(Notas_Victoria_primaria, ModEnseñanza == "Escuelas NINA")$CodigoUnicoEscolar))

In [ ]:
# Tabla de frecuencia de los alumnos por turnos
Notas_Victoria_primaria_unico <- distinct(Notas_Victoria_primaria, Documento, .keep_all = TRUE)
print("Tabla de frecuencias de turnos en el nivel primario")
Fa <- table(Notas_Victoria_primaria_unico$turno)
Fr <- prop.table(Fa)
fip <- prop.table(Fa)*100
FA <- cumsum(Fa)
FR <- cumsum(Fr)
tabla_frecuencias_turno <- round(cbind(Fa, Fr, fip,FA, FR),2)
tabla_frecuencias_turno

# Gráfico de sectores de la cantidad de alumnos por turnos
pie_commute <- round(100* table(Notas_Victoria_primaria_unico$turno) / length(Notas_Victoria_primaria_unico$turno))
colors = c('#ffeb3b', '#ff6e40','#ed2f52')
pie(pie_commute, labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
    col = colors,
    radius = .90,
    main = "Distribucion Porcentual de la Matricula de 2023
    en el departamento Victoria en cada 
    turno en el nivel primario",
    col.main = "black",
    border = 'black')

In [ ]:
# Tabla de frecuencia de los alumnos por tipo de gestion de las escuelas
print("Tabla de frecuencias de tipo de gestión en el nivel primario")
Fa <- table(Notas_Victoria_primaria_unico$Gestión)
Fr <- prop.table(Fa)
fip <- prop.table(Fa)*100
FA <- cumsum(Fa)
FR <- cumsum(Fr)
tabla_frecuencias_sector <- round(cbind(Fa, Fr, fip,FA, FR),2)
tabla_frecuencias_sector

# Gráfico de sectores de la cantidad de alumnos por los tipos de gestion de las escuelas
pie_commute <- round(100* table(Notas_Victoria_primaria_unico$Gestión) / length(Notas_Victoria_primaria_unico$Gestión))
colors = c('#2196f3', '#8bc34a')
pie(pie_commute, labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
    col = colors,
    radius = .90,
    main = "Distribucion Porcentual de la Matricula de 2023
    en el departamento Victoria por tipo 
    de gestión en el nivel primario",
    col.main = "black",
    border = 'black')

In [ ]:
# Tabla de frecuencia de las materias por condicion final
print("Tabla de frecuencias de la condición final en el nivel primario")
Fa <- table(Notas_Victoria_primaria$Condición)
Fr <- prop.table(Fa)
fip <- prop.table(Fa)*100
FA <- cumsum(Fa)
FR <- cumsum(Fr)
tabla_frecuencias_condicion <- round(cbind(Fa, Fr, fip,FA, FR),2)
tabla_frecuencias_condicion

# Gráfico de sectores de las condiciones finales
pie_commute <- round(100* table(Notas_Victoria_primaria$Condición) / length(Notas_Victoria_primaria$Condición))
colors <- c('#3333FF', '#FF2B00')
pie(pie_commute, labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
    col = colors,
    radius = .90,
    main = "Distribucion Porcentual de la Matricula de 2023 en
    el departamento Victoria segun la condición
    final en el nivel primario",
    col.main = "black",
    border = 'black')

In [ ]:
# Tabla de frecuencia de las materias por desempeño
print("Tabla de frecuencias del desempeño de los estudiantes en el nivel primario")
Fa <- table(Notas_Victoria_primaria$Desempeño)
Fr <- prop.table(Fa)
fip <- prop.table(Fa)*100
FA <- cumsum(Fa)
FR <- cumsum(Fr)
tabla_frecuencias_desempeño <- round(cbind(Fa, Fr, fip,FA, FR),2)
tabla_frecuencias_desempeño

# Gráfico de sectores del desempeño en cada materia
pie_commute <- round(100* table(Notas_Victoria_primaria$Desempeño) / length(Notas_Victoria_primaria$Desempeño))
colors <- c("#9467bd","#1f77b4", "#ff7f0e", "#2ca02c", "#d62728")
pie(pie_commute, labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
    col = colors,
    radius = .90,
    main = "Distribucion Porcentual de la Matricula de 2023 en
    el departamento Victoria segun el desempeño
    en cada asignatura",
    col.main = "black",
    border = 'black')

In [ ]:
# Extraemos los alumnos, se hace un unico conteo por alumno
aux <- distinct(Notas_Victoria_primaria, idalumno, .keep_all = TRUE)

# Tabla de frecuencias por los alumnos en los distintos grados
print("Tabla de frecuencias de los matriculados por grado en el nivel primario")
Fa <- table(aux$Año_curso)
Fr <- prop.table(Fa)
fip <- prop.table(Fa)*100
FA <- cumsum(Fa)
FR <- cumsum(Fr)
tabla_frecuencias_cursos <- round(cbind(Fa, Fr, fip,FA, FR),2)
tabla_frecuencias_cursos

# Lo modificamos a data frame para poder manipular el orden de salida del grafico
datos <- table(aux$Año_curso)
tabla_años <- as.data.frame(datos)
names(tabla_años) <- c("Año", "Freq")

# Nos permite ordenar las barras segun lo indicado
tabla_años <- tabla_años %>%
  arrange(factor(Año, levels = c('PRIMERO', 'SEGUNDO', 'TERCERO', 'CUARTO', 'QUINTO', 'SEXTO')))

# Gráfico de barras de las frecuencias absolutas por grado
barplot(tabla_años$Freq, 
        names.arg = tabla_años$Año, 
        main = "Frecuencias absolutas de estudiantes por grado en primaria",
        col = rainbow(6), 
        ylim = c(0, 600), 
        las = 1, 
        cex.names = 0.9)

# Para que aparezcan la frecuencia absoluta en la cima de cada barra
text(x = 1:length(tabla_años$Año), 
     y = tabla_años$Freq + 40,
     labels = tabla_años$Freq, 
     pos = 1,
     cex = 1,
     col = "black")

In [ ]:
# Test de dos medias independientes
datos_priv <- filter(Notas_Victoria_primaria, Gestión == "Privada")$Promedio
datos_pub <- filter(Notas_Victoria_primaria, Gestión == "Pública")$Promedio

# Distribuciones normales?
print("Test de normalidad para los promedios de escuelas privadas: ")
ks.test(datos_priv,"pnorm")
print("Test de normalidad para los promedios de escuelas públicas: ")
ks.test(datos_pub,"pnorm")
# Los promedios en los dos grupos de datos no siguen una distribucion normal

#H0: Los promedios de los alumnos son iguales en escuelas privadas y públicas.
#H1: Los promedios de los alumnos de escuelas privadas son mayores que en escuelas públicas.
wilcox.test(datos_priv,datos_pub, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de escuelas privadas: ")
median(datos_priv)
print("Mediana de los promedios de escuelas públicas: ")
median(datos_pub)

In [ ]:
# Análisis Bivariado
# Armamos una tabla con las materias comunes y el desempeño logrado por los estudiantes de Primer Grado
aux <- filter(Notas_Victoria_primaria, (asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" ))
curso_desempeño <- aux %>%
    select('Año_curso', 'Desempeño') %>%
    table()
curso_desempeño

# Reordenamos la tabla
curso_desempeño1 <- cbind(curso_desempeño[,5], curso_desempeño[,1], curso_desempeño[,2], curso_desempeño[,4], curso_desempeño[,3])
curso_desempeño1 <- rbind(curso_desempeño1[2,], curso_desempeño1[4,], curso_desempeño1[6,], curso_desempeño1[1,], curso_desempeño1[3,], curso_desempeño1[5,])
colnames(curso_desempeño1) <- c("Regular", "Aprobado", "Bueno","Muy Bueno","Distinguido")
rownames(curso_desempeño1) <- c("PRIMERO", "SEGUNDO", "TERCERO","CUARTO", "QUINTO", "SEXTO")
curso_desempeño1

print("Tabla con marginales fila y columna")
curso_desempeño2 <- cbind(curso_desempeño1, "Total"= margin.table(curso_desempeño1,1))
curso_desempeño2 <- rbind(curso_desempeño2, "Total"=margin.table(curso_desempeño2,2))
curso_desempeño2

print("Tabla contingencia ~ Porcentaje Total")
tabla_cont_total <- prop.table(curso_desempeño1)*100
tabla_cont_total <- cbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,1))
tabla_cont_total <- rbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,2))
round(tabla_cont_total,1)

print("Tabla contingencia ~ Porcentaje Fila")
tabla_cont_filas <- prop.table(rbind(curso_desempeño1, "Total"=margin.table(curso_desempeño1, 2)),1)*100
tabla_cont_filas <- cbind(tabla_cont_filas, "Total"=margin.table(tabla_cont_filas,1))
round(tabla_cont_filas,1)

print("Tabla contingencia ~ Porcentaje Columna")
tabla_cont_col <- prop.table(cbind(curso_desempeño1, "Total"=margin.table(curso_desempeño1, 1)),2)*100
tabla_cont_col <- rbind(tabla_cont_col, "Total"=margin.table(tabla_cont_col,2))
round(tabla_cont_col,1)

In [ ]:
chi <-chisq.test(Notas_Victoria_primaria$Año_curso, Notas_Victoria_primaria$Desempeño)
chi

# Vemos la fuerza de asociación
CramerV(Notas_Victoria_primaria$Año_curso, Notas_Victoria_primaria$Desempeño)

# Organizamos los datos para hacer un grafico de barras adosadas
tabla_variable_fila <- round(prop.table(table(Notas_Victoria_primaria$Año_curso, Notas_Victoria_primaria$Desempeño), 1)*100,1)
tabla_variable_fila <- t(tabla_variable_fila)
grafico <- barplot(tabla_variable_fila,
                  main= " Grafico de Contigencia por fila",
                  beside= TRUE,
                  space= c(0.1, 2.5),
                  ylim= c(0, 50),
                  cex.axis= 1.2,
                  cex.names= 0.8,
                  cex.main= 1.5,
                  col= c("green", "blue", "purple", "orange", "red"),
                  xlab = "Curso",
                  cex.lab = 1.5,
                  ylab= "Porcentaje",
                  col.lab= "black")
text(grafico, tabla_variable_fila, labels = tabla_variable_fila, pos = 3, cex = 0.7, col = "black")
legend(x = 36, y = 50, rownames(tabla_variable_fila),
       title = "Desempeño",
       title.adj = 0.5,     
       title.col = "black",      
       lty = c(1),
       col= c("green", "blue", "purple", "orange", "red"),
       lwd = 2,
       cex = 0.9) 

# Organizamos los datos para hacer un grafico de barras apiladas
tabla_variable_col <- round(prop.table(table(Notas_Victoria_primaria$Año_curso, Notas_Victoria_primaria$Desempeño), 1)*100,1)
tabla_variable_col <- t(tabla_variable_col)
grafico <- barplot(tabla_variable_col,
                  main= "Gráfico de contigencia por columna",
                  beside= FALSE,
                  space = rep(0.25, ncol(tabla_variable_col)),
                  ylim = c(0,110),
                  cex.axis = 1.2, 
                  cex.names=0.8, 
                  cex.main=1.5, 
                  col=c("green", "blue", "purple", "orange", "red"),
                  xlab = "Curso",
                  cex.lab=1.5,
                  ylab = "Porcentaje",
                  col.lab = "black")
legend(x = -0.7, y = 115, rownames(tabla_variable_col),cex = 1, fill=c("green", "blue", "purple", "orange", "red"),xpd = TRUE,horiz = TRUE,bty = "n")

In [ ]:
# Cargamos las librerías necesarias
library(aplpack)

# Filtramos las asignaturas deseadas
auxiliar <- filter(Notas_Victoria_primaria, asignatura %in% c(
  "CIENCIAS SOCIALES", "CIENCIAS NATURALES", "MATEMATICA", "LENGUA",
  "EDUCACION MUSICAL", "EDUCACION TECNOLOGICA", "EDUCACION FISICA",
  "ARTES VISUALES"))

# Seleccionamos las columnas necesarias
aux <- auxiliar %>%
        select(CodigoUnicoEscolar, asignatura, Promedio)

# Calculamos la media del promedio por cada escuela y asignatura
cue_medias <- aux %>%
    group_by(CodigoUnicoEscolar, asignatura) %>%
    dplyr::summarise(Promedio = mean(Promedio, na.rm = TRUE)) %>%
    spread(key = asignatura, value = Promedio)

# Verifica el resultado
head(cue_medias,16)

# Seleccionamos solo las columnas de promedios
data_for_faces <- cue_medias %>% select(-CodigoUnicoEscolar)

# Gráfico de caras
faces(as.matrix(data_for_faces), face.type = 1, main = "Escuelas", 
      labels = cue_medias$CodigoUnicoEscolar, print.info = TRUE, 
      nrow.plot = 5, ncol.plot = 4, plot.faces = TRUE, cex = 2)

In [ ]:
# Buscamos valores outliers
aux <- mahalanobis_distance(data= Notas_Victoria_primaria[,c("NotasPrimerT", "NotasSegundoT", "NotasTercerT", "Promedio")])$is.outlier    
ftable(aux)

## Análisis de primer grado

In [ ]:
# Calculamos con nuestro modelo lineal simple las edades de los alumnos de Primer grado
# Aplicamos la función a los estudiantes 
Notas_Victoria_1Grado <- Notas_Victoria_1Grado %>%
  mutate(EdadEstimada = estimar_edad(Documento))

# Ponemos este filtro para aquellos DNI que pertenecen a personas nacionalizado
#o estan mal cargados, ya que el modelo los etiquetaría con una edad muy erronea
Notas_Victoria_1Grado <- Notas_Victoria_1Grado %>%
  mutate(EdadEstimada = ifelse(EdadEstimada < 0 | EdadEstimada > 20, NA, EdadEstimada))

# Reordenamos las variables
Notas_Victoria_1Grado <- dplyr::select(Notas_Victoria_1Grado,'CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno', 'Año_curso', 'Documento', 'asignatura', 'idalumno', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT', 'NotasSegundoT', 'NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', 'EdadEstimada', "total_asig", "porc_aprob", "porc_desaprob")

# Vemos los resultados
Notas_Victoria_1Grado

In [ ]:
# Calculamos el promedio total de cada alumno
aux <- Notas_Victoria_1Grado %>% group_by(idalumno) %>% dplyr::summarise(Promedio_Total = round(mean(Promedio, na.rm = TRUE),2)) 

# Lo unimos con el dataframe original
Notas_Victoria_1Grado <- inner_join(aux, Notas_Victoria_1Grado, by = "idalumno")

# Asigna la materia con mejor promedio para cada estudiante
Notas_Victoria_1Grado$Mejor_asig <- Notas_Victoria_1Grado %>% group_by(idalumno) %>% mutate(Mejor_asig = asignatura[which.max(Promedio)]) %>% ungroup() %>% .$Mejor_asig

# Asigna la materia con peor promedio para cada estudiante
Notas_Victoria_1Grado$Peor_asig <- Notas_Victoria_1Grado %>% group_by(idalumno) %>% mutate(Peor_asig = asignatura[which.min(Promedio)]) %>% ungroup() %>% .$Peor_asig

In [ ]:
# Reordenamos las variables para que queden bien organizadas
Notas_Victoria_1Grado <- Notas_Victoria_1Grado %>% 
  dplyr::select('CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno', 'Año_curso', 'Documento', 'asignatura', 'idalumno', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT', 'NotasSegundoT', 'NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', 'EdadEstimada', "total_asig", "porc_aprob", "porc_desaprob", 'Mejor_asig', 'Peor_asig', 'Promedio_Total')

In [ ]:
# Calculamos las medidas de tendencia central y de dispersión para las 4 materias troncales de primaria
print("Medidas estadisticas descriptivas de primer grado")
Notas_Victoria_1Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>% 
  group_by(asignatura) %>%
  dplyr::summarise(Conteo = n(),
            Media = round(mean(Promedio, na.rm = TRUE), 2),
            Mediana = round(median(Promedio, na.rm = TRUE), 2),
            Moda = Mode(Promedio),            
            Varianza = round(var(Promedio, na.rm = TRUE), 2),
            Desvio = round(sd(Promedio, na.rm = TRUE), 2),
            Minimo = round(min(Promedio, na.rm = TRUE), 2),
            Maximo = round(max(Promedio, na.rm = TRUE), 2),
            Rango = Maximo - Minimo,
            IQR = round(IQR(Promedio, na.rm = TRUE), 2),
            Q1 = round(quantile(Promedio, 0.25, na.rm = TRUE), 2),
            Q3 = round(quantile(Promedio, 0.75, na.rm = TRUE), 2),
            Sup. = round((Q3 + 1.5 * IQR), 2),
            Inf. = round((Q1 - 1.5 * IQR), 2),
            CoefVar = round(sd(Promedio, na.rm = TRUE) / mean(Promedio, na.rm = TRUE) * 100, 2))

In [ ]:
# Filtramos los datos para que esten las filas de las 4 materias troncales
datos_filtrados <- Notas_Victoria_1Grado %>%
  filter(asignatura %in% c("CIENCIAS NATURALES", "CIENCIAS SOCIALES", "MATEMATICA", "LENGUA"))

# Creamos el gráfico
p <- ggplot(datos_filtrados, aes(x = asignatura, y = Promedio, fill = asignatura)) +
  geom_boxplot() +
  ggtitle("                      Comparación de Promedios en materias de Primer grado") +  
  labs(
    x = "Materias",
    y = "Promedios"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )

# Mostramos el gráfico
p

In [ ]:
# Cuantiles para añadir información en los boxplots
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.75)
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.25)

quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.75)
quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.50)

quantile(filter(datos_filtrados, asignatura == "CIENCIAS SOCIALES")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "CIENCIAS SOCIALES")$Promedio, probs = 0.25)

quantile(filter(datos_filtrados, asignatura == "CIENCIAS NATURALES")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "CIENCIAS NATURALES")$Promedio, probs = 0.25)

In [ ]:
# Histograma de promedios en Matemática
notas_mat_primero <- filter(Notas_Victoria_1Grado,asignatura == "MATEMATICA")$Promedio
histograma_mat <- fdt(notas_mat_primero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_mat,
     type= "fh", 
     col= "#FF9932",
     main= "Histograma de los promedios de primer grado
     en la materia Matemática")

# Medidas de distribución
coef_sim = skewness(notas_mat_primero, na.rm=TRUE)
kurtosis = kurtosis(notas_mat_primero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_mat_primero)

In [ ]:
# Histograma de promedios en Lengua
notas_len_primero <- filter(Notas_Victoria_1Grado,asignatura == "LENGUA")$Promedio
histograma_len <- fdt(notas_len_primero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_len,
     type= "fh", 
     col= "#FF86FF",
     main= "Histograma de los promedios de primer grado
     en la materia Lengua")

# Medidas de distribución
coef_sim = skewness(notas_len_primero, na.rm=TRUE)
kurtosis = kurtosis(notas_len_primero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_len_primero)

In [ ]:
# Histograma de promedios en Ciencias Naturales
notas_nat_primero <- filter(Notas_Victoria_1Grado,asignatura == "CIENCIAS NATURALES")$Promedio
histograma_nat <- fdt(notas_nat_primero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_nat,
     type= "fh", 
     col= "#00BB00",
     main= "Histograma de los promedios de primer grado
     en la materia Ciencias Naturales")

# Medidas de distribución
coef_sim = skewness(notas_nat_primero, na.rm=TRUE)
kurtosis = kurtosis(notas_nat_primero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_nat_primero)

In [ ]:
# Histograma de promedios en Ciencias Sociales
notas_soc_primero <- filter(Notas_Victoria_1Grado,asignatura == "CIENCIAS SOCIALES")$Promedio
histograma_soc <- fdt(notas_soc_primero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_soc,
     type= "fh", 
     col= "#3D87FF",
     main= "Histograma de los promedios de primer grado
     en la materia Ciencia Sociales")

# Medidas de distribución
coef_sim = skewness(notas_soc_primero, na.rm=TRUE)
kurtosis = kurtosis(notas_soc_primero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Tests de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_soc_primero)

In [ ]:
# Puntuación Z para saber que tipo de turno esta más alejado de un promedio de 8 en Matemática
# Filtramos los datos necesarios 
primero_mañana <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA" & turno == "MAÑANA")
primero_tarde <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA" & turno == "TARDE")
primero_completo <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA" & turno == "COMPLETO")

# Calculamos la media de los promedios
prom_primero_mañana <- mean(primero_mañana$Promedio, na.rm = TRUE)
prom_primero_tarde <- mean(primero_tarde$Promedio, na.rm = TRUE)
prom_primero_completo <- mean(primero_completo$Promedio, na.rm = TRUE)

# Calculamos los desvios de los promedios
desvio_primero_mañana <- sd(primero_mañana$Promedio, na.rm = TRUE)
desvio_primero_tarde <- sd(primero_tarde$Promedio, na.rm = TRUE)
desvio_primero_completo <- sd(primero_completo$Promedio, na.rm = TRUE)

# Establecemos el promedio a comparar
promedio_comparar <- 8

# Puntuación Z para el turno mañana
punt_z_mañana <- (promedio_comparar - prom_primero_mañana) / desvio_primero_mañana
print(paste(" Puntuación Z para los promedios de Matemática en primer grado del turno mañana: ", round(punt_z_mañana, 2)))

# Puntuación Z para el turno tarde
punt_z_tarde <- (promedio_comparar - prom_primero_tarde) / desvio_primero_tarde
print(paste(" Puntuación Z para los promedios de Matemática en primer grado del turno tarde: ", round(punt_z_tarde, 2)))

# Puntuación Z para el turno completo
punt_z_completo <- (promedio_comparar - prom_primero_completo) / desvio_primero_completo
print(paste(" Puntuación Z para los promedios de Matemática en primer grado del turno completo: ", round(punt_z_completo, 2)))

In [ ]:
# Puntuación Z en estudiantes de escuelas de gestión pública y gestión privada
# Filtramos los datos necesarios 
primero_publica <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA" & Gestión == "Pública")
primero_privada <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA" & Gestión == "Privada")

# Calculamos la media de los promedios
prom_primero_publica <- mean(primero_publica$Promedio, na.rm = TRUE)
prom_primero_privada <- mean(primero_privada$Promedio, na.rm = TRUE)

# Calculamos los desvios de los promedios
desvio_primero_publica <- sd(primero_publica$Promedio, na.rm = TRUE)
desvio_primero_privada <- sd(primero_privada$Promedio, na.rm = TRUE)

# Establecemos el promedio a comparar
promedio_comparar <- 8

# Puntuación Z para escuelas publicas
punt_z_publica <- (promedio_comparar - prom_primero_publica) / desvio_primero_publica
print(paste(" Puntuación Z para los promedios de Matemática en primer grado de escuelas públicas: ", round(punt_z_publica, 2)))

# Puntuación Z para escuelas privadas
punt_z_privada <- (promedio_comparar - prom_primero_privada) / desvio_primero_privada
print(paste(" Puntuación Z para los promedios de Matemática en primer grado de escuelas privadas: ", round(punt_z_privada, 2)))

In [ ]:
# Matriz de correlación
# Unimos en un data frame las materias seleccionadas para calcular correlaciones
df <- suppressWarnings(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "CIENCIAS NATURALES"),"Documento","Promedio") %>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "CIENCIAS SOCIALES"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "MATEMATICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "LENGUA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "EDUCACION FISICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "EDUCACION MUSICAL"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "EDUCACION TECNOLOGICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_1Grado,asignatura == "ARTES VISUALES"),"Documento","Promedio"),by="Documento"))
df <- rename(df,Prom_Nat=Promedio.x, Prom_Soc=Promedio.y, Prom_Mat=Promedio.x.x, Prom_Len=Promedio.y.y, Prom_EdFisica=Promedio.x.x.x, Prom_Mus=Promedio.y.y.y, Prom_Tecno=Promedio.x.x.x.x, Prom_Art=Promedio.y.y.y.y)

# Gráfico de la matriz de correlación
matriz_correlacion <- cor(df[, -1], method="spearman")
corrplot(matriz_correlacion, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Matriz de correlación", cex.main = 1.5)

# Gráfico de correlación parcial
variables <- c("Prom_Nat", "Prom_Soc", "Prom_Mat", "Prom_Len")
dataM <- df[, variables]
correlacion_parcial <- pcor(dataM, method = "pearson")$estimate
corrplot(correlacion_parcial, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Correlación parcial", cex.main = 1.5)

In [ ]:
# Test de una media
# En este caso, a los promedios de Primer grado en las materias de Lengua y Matematica
primero_mat <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA")$Promedio
primero_len <- filter(Notas_Victoria_1Grado, asignatura == "LENGUA")$Promedio

# Ya sabemos que no son datos con distribuciones normales
# Entonces...
# Al querer comparar las media de los promedios conseguidos por los alumnos de primer grado en
#Matematica y Lengua se deberían usar test parametricos, pero con el supuesto de que los datos 
#tengan una distribucion normal. Al no cumplir con esta condición, utilizaremos un test no parametrico 
#como lo es el test de Wilcox pero  en vez de comparar las medias, compararemos las medianas.

# H0 -> la mediana de los alumnos de Primer grado en Matematica es 8
# H1 -> la mediana de los alumnos de Primer grado en Matematica es mayor a 8
print("Test de Wilcox para los promedios de Matemática: ")
wilcox.test(primero_mat, mu=8, alternative = "greater", conf.level = 0.95)

# H0 -> la mediana de los alumnos de primer grado en Lengua es 8
# H1 -> la mediana de los alumnos de primer grado en Lengua es mayor a 8
print("Test de Wilcox para los promedios de Lengua")
wilcox.test(primero_len, mu=8, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de Matemática: ")
median(primero_mat)
print("Mediana de los promedios de Lengua: ")
median(primero_len)

In [ ]:
# Test de dos medias independientes
# Queremos ver si los promedios de los alumnos de Primer grado que asisten a una escuela privada son mejores que los de la escuela publica
datos_priv <- filter(Notas_Victoria_1Grado, Gestión == "Privada" & asignatura =="LENGUA")$Promedio
datos_pub <- filter(Notas_Victoria_1Grado, Gestión == "Pública" & asignatura =="LENGUA")$Promedio

# Distribuciones normales?
print("Test de shapiro para los promedios de escuelas privadas: ")
shapiro.test(datos_priv)
print("Test de shapiro para los promedios de escuelas públicas: ")
shapiro.test(datos_pub)
# Los promedios en los dos grupos de datos no siguen una distribucion normal

#H0: Los promedios de los alumnos de Primer grado en Lengua son iguales en escuelas privadas y públicas.
#H1: Los promedios de los alumnos de Primer grado en Lengua en escuelas privadas son mayores que en escuelas públicas.
wilcox.test(datos_priv,datos_pub, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de escuelas privadas: ")
median(datos_priv)
print("Mediana de los promedios de escuelas públicas: ")
median(datos_pub)

In [ ]:
# Test de más de dos medias independientes
# Queremos ver si los promedios de los alumnos de Primer grado que asisten a diferentes turnos tienen la mismda media
datos_primero_mañana <- filter(Notas_Victoria_1Grado, turno == "MAÑANA" & asignatura == "MATEMATICA")
datos_primero_tarde <- filter(Notas_Victoria_1Grado, turno == "TARDE" & asignatura == "MATEMATICA")
datos_primero_completo <- filter(Notas_Victoria_1Grado, turno == "COMPLETO" & asignatura == "MATEMATICA")

# Combinamos los datos para graficarlos
datos_comb <- bind_rows(
  datos_primero_mañana %>% mutate(turno = "MAÑANA"),
  datos_primero_tarde %>% mutate(turno = "TARDE"),
  datos_primero_completo %>% mutate(turno = "COMPLETO"))

# Boxplots comparativo interactivo
plot <- plot_ly(datos_comb, x = ~turno, y = ~Promedio, color = ~turno, type = "box") %>%
  layout(
    title = "Comparación de Promedios de Matemáticas en Primer Grado por Turno",
    xaxis = list(title = "Turno"),
    yaxis = list(title = "Promedio"),
    showlegend = FALSE)
plot

In [ ]:
# Pasamos los datos a tipo numerico
primero = as.numeric(c(datos_primero_mañana$Promedio, datos_primero_tarde$Promedio, datos_primero_completo$Promedio))

# Para que se repita la cantidad de alumno en cada turno su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_primero_mañana$Promedio)), rep("TARDE",length(datos_primero_tarde$Promedio)), rep("COMPLETO", length(datos_primero_completo$Promedio))))    

# Armamos el dataframe
datos_primero <- data.frame(turno, primero)

# Agrupamos los datos según el turno, calculamos la media de los promedios y contabilizamos las frecuencias
datos_primero %>% group_by(turno) %>%
    dplyr::summarise(Media = round(mean(primero),1), datos = n())

# Test de ANOVA
#H0 ---> Los tres turnos de Primero tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
#Nivel de significancia = 0.05
fm_primero = aov(primero ~ turno)
summary(fm_primero)

# Potencia del test
potencia_test <- pwr.anova.test(k = 3, n = c(75, 272, 217), f = 0.25, sig.level = 0.05)$power
potencia_test 

# Test HSD de Tukey
TukeyHSD(fm_primero, "turno", conf.level = .95)

# Pero para saber que todos estos resultados son contundentes, la distribución de los datos tiene que ser
#normal, por eso se calcularan los test de shapiro
shapiro.test(datos_primero$primero[datos_primero$turno == "MAÑANA"])
shapiro.test(datos_primero$primero[datos_primero$turno == "TARDE"])
shapiro.test(datos_primero$primero[datos_primero$turno == "COMPLETO"])

# Como los datos no siguen una distribucion normal, se descarta todo lo calculado. Se procede a calcularlo
#con un test no parametrico

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

# Test no parametrico para primer grado
#H0 ---> Los tres turnos de Primer grado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_primero$primero, datos_primero$turno)
pairwise.wilcox.test(datos_primero$primero, datos_primero$turno)

In [ ]:
# Test de más de dos medias independientes
# El mismo análisis pero para Lengua ahora
datos_primero_mañana <- filter(Notas_Victoria_1Grado, turno == "MAÑANA" & asignatura == "LENGUA")
datos_primero_tarde <- filter(Notas_Victoria_1Grado, turno == "TARDE" & asignatura == "LENGUA")
datos_primero_completo <- filter(Notas_Victoria_1Grado, turno == "COMPLETO" & asignatura == "LENGUA")

# Combinamos los datos para graficarlos
datos_comb <- bind_rows(
  datos_primero_mañana %>% mutate(turno = "MAÑANA"),
  datos_primero_tarde %>% mutate(turno = "TARDE"),
  datos_primero_completo %>% mutate(turno = "COMPLETO"))

# Boxplots comparativo interactivo
plot <- plot_ly(datos_comb, x = ~turno, y = ~Promedio, color = ~turno, type = "box") %>%
  layout(
    title = "Comparación de Promedios de Lengua en primero por Turno",
    xaxis = list(title = "Turno"),
    yaxis = list(title = "Promedio"),
    showlegend = FALSE)
plot

In [ ]:
# Pasamos los datos a tipo numerico
primero = as.numeric(c(datos_primero_mañana$Promedio, datos_primero_tarde$Promedio, datos_primero_completo$Promedio))

# Para que se repita la cantidad de alumno en cada turno su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_primero_mañana$Promedio)), rep("TARDE",length(datos_primero_tarde$Promedio)), rep("COMPLETO", length(datos_primero_completo$Promedio))))    

# Armamos el dataframe
datos_primero <- data.frame(turno, primero)

# Agrupamos los datos según el turno, calculamos la media de los promedios y contabilizamos las frecuencias
datos_primero %>% group_by(turno) %>%
    dplyr::summarise(Media = round(mean(primero),1), datos = n())

# Test de ANOVA
#H0 ---> Los tres turnos de Primero tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
#Nivel de significancia = 0.05
fm_primero = aov(primero ~ turno)
summary(fm_primero)

# Potencia del test
potencia_test <- pwr.anova.test(k = 3, n = c(75, 272, 219), f = 0.25, sig.level = 0.05)$power
potencia_test 

# Test HSD de Tukey
TukeyHSD(fm_primero, "turno", conf.level = .95)

# Pero para saber que todos estos resultados son contundentes, la distribución de los datos tiene que ser
#normal, por eso se calcularan los test de shapiro
shapiro.test(datos_primero$primero[datos_primero$turno == "MAÑANA"])
shapiro.test(datos_primero$primero[datos_primero$turno == "TARDE"])
shapiro.test(datos_primero$primero[datos_primero$turno == "COMPLETO"])

# Como los datos no siguen una distribucion normal, se descarta todo lo calculado. Se procede a calcularlo
#con un test no parametrico

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

# Test no parametrico para primer grado
#H0 ---> Los tres turnos de Primer grado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_primero$primero, datos_primero$turno)
pairwise.wilcox.test(datos_primero$primero, datos_primero$turno)

In [ ]:
# Test para mas de dos medias pareadas
# Vamos a averiguar si las notas de Matemática en los tres trimestres tienen medias iguales o difieren
datos_primer_tri <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA")$NotasPrimerT
datos_segundo_tri <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA")$NotasSegundoT
datos_tercer_tri <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA")$NotasTercerT

# Unimos los datos filtrados a un data frame
datos_completos <- data.frame(
  Trimestre = rep(c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"), each = length(datos_primer_tri)),
  Notas = c(datos_primer_tri, datos_segundo_tri, datos_tercer_tri))

# Convertimos la variable Trimestre a factor
datos_completos$Trimestre <- factor(datos_completos$Trimestre, levels = c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"))

# Creamos un gráfico interactivo de las notas en cada trimestre
plot <- plot_ly() %>%
  add_boxplot(data = datos_completos, x = ~Trimestre, y = ~Notas, color = ~Trimestre, colors = c("blue", "red", "green"), name = "Matemáticas", offsetgroup = 1) %>%
 layout(
    title = "Comparación de notas por trimestre de Matemáticas en Primero",
    xaxis = list(title = "Trimestre"),
    yaxis = list(title = "Notas"),
    showlegend = FALSE)
plot

In [ ]:
# Calculamos el tamaño de los alumnos de Primer grado en Matemática para utilizarlo posteriormente
tamanio = length(datos_primer_tri)

# Armamos el data frame con lo datos
notas_primero <- data.frame(
 sujeto = 1:tamanio,
 datos_primer_tri <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA")$NotasPrimerT,
 datos_segundo_tri <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA")$NotasSegundoT,
 datos_tercer_tri <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA")$NotasTercerT)

# Esto sirve para armar los datos de manera larga
datos_largo <- melt(notas_primero, id.vars= "sujeto", variable.name= "Notas", value.name= "Puntuacion")

# Realizar el ANOVA de medias pareadas
modelo_anova <- aov(Puntuacion ~ Notas + Error(sujeto/Notas), data = datos_largo)

# Cambiar el tipo de modelo a lm
modelo_lm <- lm(modelo_anova)

# Calcular el ANOVA con el modelo lm
anova_resultado <- Anova(modelo_lm, type = "III")
print(anova_resultado)

# Calculamos normalidad de los residuos y constatamos que las varianzas sean constantes
print("Test de Normalidad:")
shapiro.test(modelo_lm$residuals)
print("Test de homocedasticidad: ")
bartlett.test(modelo_lm$residuals ~ datos_largo$Notas)

# No se cumplen con lo supuestos, por lo cual se descarta este análisis y se procedera a realizarlo
# con un test no parametrico

In [ ]:
# Armamos el data frame con lo datos
notas= as.numeric(c(datos_primer_tri,datos_segundo_tri,datos_tercer_tri))
trimestres <- as.factor(c(rep("Primer Trimestre",length(datos_primer_tri)), rep("Segundo Trimestre",length(datos_segundo_tri)),rep("Tercer Trimestre",length(datos_tercer_tri))))
datos <- data.frame(Alumnos=rep(1:tamanio,3), notas, trimestres)
head(datos)

# Calculamos las medias de cada grupo
by(data = datos$notas, INDICES = datos$trimestres, FUN = median)

# Test para encontrar diferencias significativas entre los grupos
friedman.test(notas ~ trimestres | Alumnos, data=datos)

# Potencia del test
friedman_anova <- pwr.anova.test(k = 3, n = tamanio, f = 0.25, sig.level = 0.05)$power
print("Potencia del test:")
friedman_anova

# Para saber entre que conjuntos de datos estan las diferencias
pairwise.wilcox.test(datos$notas, datos$trimestres, paired = TRUE)

# Grafico de coordenadas paralelas de los primeros 10 estudiantes (no se hacen todos porque no se entenderia el grafico con tantas lineas)
# Primero modificamos el nombre de los datos para que a la hora de ponerlos en el grafico, se vea mas conciso cada clase
datos_largo$Notas <- gsub("datos_primer_tri.*", "Primer Trimestre", datos_largo$Notas)
datos_largo$Notas <- gsub("datos_segundo_tri.*", "Segundo Trimestre", datos_largo$Notas)
datos_largo$Notas <- gsub("datos_tercer_tri.*", "Tercer Trimestre", datos_largo$Notas)

# Filtramos los primeros 10 alumnos
datos_grafica <- datos_largo %>%
  filter(sujeto <= 10)

# Grafico de coordenadas paralelas de los primeros 10 alumnos con sus notas en cada trimestre
ggplot(datos_grafica, aes(x = Notas, y = Puntuacion, group = sujeto, color = factor(sujeto))) +
  geom_line() +
  geom_point() +
  labs(title = "Evolución de algunas de las notas trimestrales a lo largo del año 2023
    de los alumnos de Primer Grado en Matemática",
       x = "Trimestre",
       y = "Nota") +
  theme_minimal()

In [ ]:
# Análisis Bivariado
# Armamos una tabla con las materias comunes y el desempeño logrado por los estudiantes de Primer Grado
aux <- filter(Notas_Victoria_1Grado, (asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" ))
materias_desempeño <- aux %>%
    select('asignatura', 'Desempeño') %>%
    table()
materias_desempeño

# Reordenamos la tabla
materias_desempeño1 <- cbind(materias_desempeño[,5], materias_desempeño[,1], materias_desempeño[,2], materias_desempeño[,4], materias_desempeño[,3])
colnames(materias_desempeño1) <- c("Regular", "Aprobado", "Bueno","Muy Bueno","Distinguido")
rownames(materias_desempeño1) <- c("ARTES VISUALES", "CS. NATURALES", "CS. SOCIALES","ED. FISICA", "ED. MUSICAL", "ED. TECNOLOGICA", "LENGUA", "MATEMATICA")
materias_desempeño1

print("Tabla con marginales fila y columna")
materias_desempeño2 <- cbind(materias_desempeño1, "Total"= margin.table(materias_desempeño1,1))
materias_desempeño2 <- rbind(materias_desempeño2, "Total"=margin.table(materias_desempeño2,2))
materias_desempeño2

print("Tabla contingencia ~ Porcentaje Total")
tabla_cont_total <- prop.table(materias_desempeño1)*100
tabla_cont_total <- cbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,1))
tabla_cont_total <- rbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,2))
round(tabla_cont_total,1)

print("Tabla contingencia ~ Porcentaje Fila")
tabla_cont_filas <- prop.table(rbind(materias_desempeño1, "Total"=margin.table(materias_desempeño1, 2)),1)*100
tabla_cont_filas <- cbind(tabla_cont_filas, "Total"=margin.table(tabla_cont_filas,1))
round(tabla_cont_filas,1)

print("Tabla contingencia ~ Porcentaje Columna")
tabla_cont_col <- prop.table(cbind(materias_desempeño1, "Total"=margin.table(materias_desempeño1, 1)),2)*100
tabla_cont_col <- rbind(tabla_cont_col, "Total"=margin.table(tabla_cont_col,2))
round(tabla_cont_col,1)

In [ ]:
# Organizamos los datos para hacer un grafico de barras adosadas
tabla_variable_fila <- round(prop.table(table(aux$asignatura, aux$Desempeño), 1)*100,1)
tabla_variable_fila <- t(tabla_variable_fila)
grafico <- barplot(tabla_variable_fila,
                  main= " Grafico de Contigencia por fila",
                  beside= TRUE,
                  space= c(0.1, 1.5),
                  ylim= c(0, 50),
                  cex.axis= 1.2,
                  cex.names= 0.1,
                  cex.main= 1.5,
                  col = c("green","blue", "purple","orange","red"),
                  xlab = "Artes   Cs.Nat   Cs.Soc   Ed.Fis   Ed.Mus   Ed.Tecno  Leng  Mat",
                  cex.lab = 1.2,
                  ylab= "Porcentaje",
                  col.lab= "black")
text(grafico, tabla_variable_fila, labels = tabla_variable_fila, pos = 3, cex = 0.7, col = "black")
legend("topright", rownames(tabla_variable_fila),
       title = "Desempeño",
       title.adj = 0.4,     
       title.col = "black",      
       lty = c(1),
       col = c("green","blue", "purple","orange","red"),
       lwd = 4,
       cex = 1)

# Organizamos los datos para hacer un grafico de barras apiladas
tabla_variable_col <- round(prop.table(table(aux$asignatura, aux$Desempeño), 2) * 100, 1)
grafico <- barplot(tabla_variable_col,
                   main = "Gráfico de Contingencia por columna",
                   beside = FALSE,
                   space = rep(0.25, ncol(tabla_variable_col)),
                   ylim = c(0, 110),
                   cex.axis = 1.2, 
                   cex.names = 1, 
                   cex.main = 1.5, 
                   col = c("green", "red", "blue", "orange", "purple", "cyan", "yellow", "magenta"),
                   xlab = "Desempeños",
                   cex.lab = 1.5,
                   ylab = "Porcentaje",
                   col.lab = "black")
legend(x = 0, y = 115, rownames(tabla_variable_col),cex = 0.5, fill=c("green", "red", "blue", "orange", "purple", "cyan", "yellow", "magenta"),xpd = TRUE,horiz = FALSE,bty = "n")

In [ ]:
# Como muestran los graficos y las tablas, segun el tipo de materia "causa"
#el desempeño final de cada estudiante

# Para saber si realmente existe una relación entre las variables aplicaremos el test de chi cuadrado
# H0 ---> No hay relación entre las variables
# H1 ---> Si hay relación entre las variables
chi <-chisq.test(aux$asignatura, aux$Desempeño)
chi

# Vemos la fuerza de asociación
CramerV(aux$asignatura, aux$Desempeño)

In [ ]:
# Frecuencias observadas
chi$observed

# Frecuencias esperadas si las variables fueran independientes
round(chi$expected,0)

# Residuos estandarizados
round(chi$residuals,1)

# Residuos ajustados
round(chi$stdres,1)

In [ ]:
# Análisis Multivariado
# Vector de medias de las variables de primer grado agrupadas por las asignaturas
vector_medias <- Notas_Victoria_1Grado %>% group_by(asignatura) %>% summarise_all(mean)

# Nos quedamos con las materias que nos interesan
vector_medias <- vector_medias %>% filter(asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" | asignatura== "CATEQUESIS" | asignatura == "INFORMATICA" |asignatura =="INGLES" )
vector_medias

In [ ]:
# Buscamos valores outliers
aux <- mahalanobis_distance(data= Notas_Victoria_1Grado[,c("NotasPrimerT", "NotasSegundoT", "NotasTercerT", "Promedio")])$is.outlier    
ftable(aux)

In [ ]:
# Prueba de normalidad
mardia2(Notas_Victoria_1Grado[,11:14])
# Los datos no siguen una distribución normal multivariada

In [ ]:
# Grafico de perfiles paralelos para ver el rendimiento de los estudiantes en 11 materias
# Definimos colores manuales para que se distingan
colores <- c("ARTES VISUALES" = "purple","CATEQUESIS" = "orange",
            "CIENCIAS NATURALES" = "green","CIENCIAS SOCIALES" = "blue",
            "EDUCACION FISICA" = "red","EDUCACION MUSICAL" = "turquoise",
            "EDUCACION TECNOLOGICA" = "black","INFORMATICA" = "cyan",
            "INGLES" = "yellow","LENGUA" = "magenta",
            "MATEMATICA" = "skyblue")

# Grafico
ggparcoord(data = vector_medias,
           columns=c(11:14),
           groupColumn = "asignatura",
           showPoints = TRUE,
           scale = "globalminmax") +
  scale_color_manual(values = colores) +
  theme(axis.text.x = element_text(angle = 30, hjust = 1))

In [ ]:
# COMPARACION DE DOS POBLACIONES MULTIVARIADAS INDEPENDIENTES

# Asumimos que las medias de las variables para cada grupo se aproximan a una 
#distribución normal multivariada debido al Teorema del Límite Central Multivariado (TLCM). 
#Esto nos permite aplicar el Test de Hotelling.

# Queremos saber si los vectores entre los tipo de gestión son iguales o no
perfil_medio_tipo <- Notas_Victoria_1Grado %>% group_by(Gestión) %>% summarise_all(mean)

# Grafico de coordenadas paralelas
ggparcoord(data = perfil_medio_tipo,
           columns = 11:14,
           groupColumn = "Gestión",
           showPoints = TRUE,
           scale = "globalminmax")

# Test de hotelling 
# H0 ---> los vectores medios son iguales
# H1 ---> los vectores medios difieren en un grupo
prueba_N<-dplyr::select(Notas_Victoria_1Grado,Gestión,NotasPrimerT, NotasSegundoT, NotasTercerT, Promedio)
fit = hotelling.test(.~Gestión, data = prueba_N)
fit 
# Rechazamos H0, uno de los vectores medios difiere entre los grupos de Gestión

In [ ]:
# COMPARACION DE TRES O MÁS POBLACIONES MULTIVARIADAS INDEPENDIENTES
mod1<-manova(cbind(NotasPrimerT, NotasSegundoT, NotasTercerT, Promedio)~turno,data=Notas_Victoria_1Grado)
summary(mod1)

# El MANOVA nos indica que hay diferencias entre los vectoresmedios de múltiples variables dependientes entre dos o más grupos

#Supuestos de normalidad
Notas_Victoria_1Grado %>% group_by(turno) %>%  shapiro_test(NotasPrimerT, NotasSegundoT, NotasTercerT, Promedio)
# Ningun grupo de datos sigue una distribucion normal multivariada

In [ ]:
# Boxplot comparativo para las diferentes escuelas según el Promedio
g2 <- ggplot(Notas_Victoria_1Grado, aes(factor(CodigoUnicoEscolar), Promedio))
g2 + geom_boxplot(aes(fill = factor(CodigoUnicoEscolar))) +
  labs(title = "Distribución del Promedio por Código Único Escolar",
       x = "Código Único Escolar",
       y = "Promedio",
       fill = "Código Único Escolar") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        legend.position = "none")

In [ ]:
# Boxplot comparativo para las diferentes materias segun el promedio 
g2 <- ggplot(Notas_Victoria_1Grado, aes(factor(asignatura), Promedio))
g2 + geom_boxplot(aes(fill = factor(asignatura))) +
  labs(title = "Distribución del Promedio por Código Único Escolar",
       x = "Materias",
       y = "Promedio",
       fill = "Materias") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        legend.position = "none")

## Análisis de Segundo grado

In [ ]:
# Calculamos con nuestro  modelo lineal simple las edades de los alumnos de Segundo grado
# Aplicamos la función a los estudiantes 
Notas_Victoria_2Grado <- Notas_Victoria_2Grado %>%
  mutate(EdadEstimada = estimar_edad(Documento))

# Ponemos este filtro para aquellos DNI que pertenecen a personas nacionalizado o
#estan mal cargados, ya que el modelo los etiquetaría con una edad muy erronea
Notas_Victoria_2Grado <- Notas_Victoria_2Grado %>%
  mutate(EdadEstimada = ifelse(EdadEstimada < 0 | EdadEstimada > 20, NA, EdadEstimada))

# Reordenamos las variables
Notas_Victoria_2Grado <- dplyr::select(Notas_Victoria_2Grado,'CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno', 'Año_curso', 'Documento', 'asignatura', 'idalumno', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT', 'NotasSegundoT', 'NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', 'EdadEstimada', "total_asig", "porc_aprob", "porc_desaprob")

# Vemos los resultados
Notas_Victoria_2Grado

In [ ]:
# Calculamos el promedio total de cada estudiante
aux <- Notas_Victoria_2Grado %>% group_by(idalumno) %>% dplyr::summarise(Promedio_Total = round(mean(Promedio, na.rm = TRUE),2)) 

# Lo unimos con el dataframe original
Notas_Victoria_2Grado <- inner_join(aux, Notas_Victoria_2Grado, by = "idalumno")

# Asigna la materia con mejor promedio para cada estudiante
Notas_Victoria_2Grado$Mejor_asig <- Notas_Victoria_2Grado %>% group_by(idalumno) %>% mutate(Mejor_asig = asignatura[which.max(Promedio)]) %>% ungroup() %>% .$Mejor_asig

# Asigna la materia con peor promedio para cada estudiante
Notas_Victoria_2Grado$Peor_asig <- Notas_Victoria_2Grado %>% group_by(idalumno) %>% mutate(Peor_asig = asignatura[which.min(Promedio)]) %>% ungroup() %>% .$Peor_asig

In [ ]:
# Reordenamos las variables para que queden bien organizadas
Notas_Victoria_2Grado <- Notas_Victoria_2Grado %>% 
  dplyr::select('CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno', 'Año_curso', 'Documento', 'asignatura', 'idalumno', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT', 'NotasSegundoT', 'NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', 'EdadEstimada', "total_asig", "porc_aprob", "porc_desaprob", 'Mejor_asig', 'Peor_asig', 'Promedio_Total')

In [ ]:
# Calculamos las medidas de tendencia central y de dispersión para las 4 materias troncales de primaria
print("Medidas estadisticas descriptivas de Segundo grado")
Notas_Victoria_2Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>% 
  group_by(asignatura) %>%
  dplyr::summarise(Conteo = n(),
            Media = round(mean(Promedio, na.rm = TRUE), 2),
            Mediana = round(median(Promedio, na.rm = TRUE), 2),
            Moda = Mode(Promedio),            
            Varianza = round(var(Promedio, na.rm = TRUE), 2),
            Desvio = round(sd(Promedio, na.rm = TRUE), 2),
            Minimo = round(min(Promedio, na.rm = TRUE), 2),
            Maximo = round(max(Promedio, na.rm = TRUE), 2),
            Rango = Maximo - Minimo,
            IQR = round(IQR(Promedio, na.rm = TRUE), 2),
            Q1 = round(quantile(Promedio, 0.25, na.rm = TRUE), 2),
            Q3 = round(quantile(Promedio, 0.75, na.rm = TRUE), 2),
            Sup. = round((Q3 + 1.5 * IQR), 2),
            Inf. = round((Q1 - 1.5 * IQR), 2),
            CoefVar = round(sd(Promedio, na.rm = TRUE) / mean(Promedio, na.rm = TRUE) * 100, 2))

In [ ]:
# Filtramos los datos para que esten las filas de las 4 materias troncales
datos_filtrados <- Notas_Victoria_2Grado %>%
  filter(asignatura %in% c("CIENCIAS NATURALES", "CIENCIAS SOCIALES", "MATEMATICA", "LENGUA"))

# Gráfico de boxplot comparativo
p <- ggplot(datos_filtrados, aes(x = asignatura, y = Promedio, fill = asignatura)) +
  geom_boxplot() +
  ggtitle("                      Comparación de Promedios en materias de Segundo grado") +  
  labs(
    x = "Materias",
    y = "Promedios"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )

# Mostramos el gráfico
p

In [ ]:
# Cuantiles para añadir información en los boxplots
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.75)
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.25)

quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.75)

quantile(filter(datos_filtrados, asignatura == "CIENCIAS SOCIALES")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "CIENCIAS SOCIALES")$Promedio, probs = 0.25)

quantile(filter(datos_filtrados, asignatura == "CIENCIAS NATURALES")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "CIENCIAS NATURALES")$Promedio, probs = 0.25)

In [ ]:
# Histograma de promedios en Matemática
notas_mat_segundo <- filter(Notas_Victoria_2Grado,asignatura == "MATEMATICA")$Promedio
histograma_mat <- fdt(notas_mat_segundo, breaks= "Sturges", na.rm= TRUE)
plot(histograma_mat,
     type= "fh", 
     col= "#FF9932",
     main= "Histograma de los promedios de Segundo grado
     en la materia Matemática")

# Medidas de distribución
coef_sim = skewness(notas_mat_segundo, na.rm=TRUE)
kurtosis = kurtosis(notas_mat_segundo, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_mat_segundo)

In [ ]:
# Histograma de promedios en Lengua
notas_len_segundo <- filter(Notas_Victoria_2Grado,asignatura == "LENGUA")$Promedio
histograma_len <- fdt(notas_len_segundo, breaks= "Sturges", na.rm= TRUE)
plot(histograma_len,
     type= "fh", 
     col= "#FF86FF",
     main= "Histograma de los promedios de Segundo grado
     en la materia Lengua")

# Medidas de distribución
coef_sim = skewness(notas_len_segundo, na.rm=TRUE)
kurtosis = kurtosis(notas_len_segundo, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_len_segundo)

In [ ]:
# Histograma de promedios en Ciencias Naturales
notas_nat_segundo <- filter(Notas_Victoria_2Grado,asignatura == "CIENCIAS NATURALES")$Promedio
histograma_nat <- fdt(notas_nat_segundo, breaks= "Sturges", na.rm= TRUE)
plot(histograma_nat,
     type= "fh", 
     col= "#00BB00",
     main= "Histograma de los promedios de Segundo grado
     en la materia Ciencias Naturales")

# Medidas de distribución
coef_sim = skewness(notas_nat_segundo, na.rm=TRUE)
kurtosis = kurtosis(notas_nat_segundo, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_nat_segundo)

In [ ]:
# Histograma de promedios en Ciencias Sociales
notas_soc_segundo <- filter(Notas_Victoria_2Grado,asignatura == "CIENCIAS SOCIALES")$Promedio
histograma_soc <- fdt(notas_soc_segundo, breaks= "Sturges", na.rm= TRUE)
plot(histograma_soc,
     type= "fh", 
     col= "#3D87FF",
     main= "Histograma de los promedios de Segundo grado
     en la materia Ciencias Sociales")

# Medidas de distribución
coef_sim = skewness(notas_soc_segundo, na.rm=TRUE)
kurtosis = kurtosis(notas_soc_segundo, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_soc_segundo)

In [ ]:
# Puntuación Z para saber que tipo de turno esta más alejado de un promedio de 8 en Matemática
# Filtramos los datos
segundo_mañana <- filter(Notas_Victoria_2Grado, asignatura == "MATEMATICA" & turno == "MAÑANA")
segundo_tarde <- filter(Notas_Victoria_2Grado, asignatura == "MATEMATICA" & turno == "TARDE")
segundo_completo <- filter(Notas_Victoria_2Grado, asignatura == "MATEMATICA" & turno == "COMPLETO")

# Calculamos la media de los promedios
prom_segundo_mañana <- mean(segundo_mañana$Promedio, na.rm = TRUE)
prom_segundo_tarde <- mean(segundo_tarde$Promedio, na.rm = TRUE)
prom_segundo_completo <- mean(segundo_completo$Promedio, na.rm = TRUE)

# Calculamos los desvios de los promedios
desvio_segundo_mañana <- sd(segundo_mañana$Promedio, na.rm = TRUE)
desvio_segundo_tarde <- sd(primero_tarde$Promedio, na.rm = TRUE)
desvio_segundo_completo <- sd(segundo_completo$Promedio, na.rm = TRUE)

# Promedio a comparar
promedio_comparar <- 8

# Puntuación Z para el turno mañana
punt_z_mañana <- (promedio_comparar - prom_segundo_mañana) / desvio_segundo_mañana
print(paste(" Puntuación Z para los promedios de Matemática en Segundo grado del turno mañana: ", round(punt_z_mañana, 2)))

# Puntuación Z para el turno tarde
punt_z_tarde <- (promedio_comparar - prom_segundo_tarde) / desvio_segundo_tarde
print(paste(" Puntuación Z para los promedios de Matemática en Segundo grado del turno tarde: ", round(punt_z_tarde, 2)))

# Puntuación Z para el turno completo
punt_z_completo <- (promedio_comparar - prom_segundo_completo) / desvio_segundo_completo
print(paste(" Puntuación Z para los promedios de Matemática en Segundo grado del turno completo: ", round(punt_z_completo, 2)))

In [ ]:
# Puntuación Z en estudiantes de escuelas de gestión pública y gestión privada
# Filtramos los datos
segundo_publica <- filter(Notas_Victoria_2Grado, asignatura == "MATEMATICA" & Gestión == "Pública")
segundo_privada <- filter(Notas_Victoria_2Grado, asignatura == "MATEMATICA" & Gestión == "Privada")

# Calculamos la media de los promedios
prom_segundo_publica <- mean(segundo_publica$Promedio, na.rm = TRUE)
prom_segundo_privada <- mean(segundo_privada$Promedio, na.rm = TRUE)

# Calculamos los desvios de los promedios
desvio_segundo_publica <- sd(segundo_publica$Promedio, na.rm = TRUE)
desvio_segundo_privada <- sd(segundo_privada$Promedio, na.rm = TRUE)

# Promedio a comparar
promedio_comparar <- 8

# Puntuación Z para escuelas publicas
punt_z_publica <- (promedio_comparar - prom_segundo_publica) / desvio_segundo_publica
print(paste(" Puntuación Z para los promedios de Matemática en Segundo grado de escuelas públicas: ", round(punt_z_publica, 2)))

# Puntuación Z para escuelas privadas
punt_z_privada <- (promedio_comparar - prom_segundo_privada) / desvio_segundo_privada
print(paste(" Puntuación Z para los promedios de Matemática en Segundo grado de escuelas privadas: ", round(punt_z_privada, 2)))

In [ ]:
# Matriz de correlacion
# Unimos en un data frame las materias seleccionadas para calcular correlaciones
df <- suppressWarnings(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "CIENCIAS NATURALES"),"Documento","Promedio") %>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "CIENCIAS SOCIALES"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "MATEMATICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "LENGUA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "EDUCACION FISICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "EDUCACION MUSICAL"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "EDUCACION TECNOLOGICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_2Grado,asignatura == "ARTES VISUALES"),"Documento","Promedio"),by="Documento"))
df <- rename(df,Prom_Nat=Promedio.x, Prom_Soc=Promedio.y, Prom_Mat=Promedio.x.x, Prom_Len=Promedio.y.y, Prom_EdFisica=Promedio.x.x.x, Prom_Mus=Promedio.y.y.y, Prom_Tecno=Promedio.x.x.x.x, Prom_Art=Promedio.y.y.y.y)

# Gráfico de la matriz de correlación
matriz_correlacion <- cor(df[, -1], method="spearman")
corrplot(matriz_correlacion, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Matriz de correlación", cex.main = 1.5)

# Gráfico de correlación parcial
variables <- c("Prom_Nat", "Prom_Soc","Prom_Mat", "Prom_Len", "Prom_Mus", "Prom_Tecno")
dataM <- df[, variables]
correlacion_parcial <- pcor(dataM, method = "pearson")$estimate
corrplot(correlacion_parcial, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Correlación parcial", cex.main = 1.5)

In [ ]:
# Test de una media
# Filtramos los datos
# En este caso, a los promedios de Segundo grado en las materias de Lengua y Matematica
segundo_mat <- filter(Notas_Victoria_2Grado, asignatura == "MATEMATICA")$Promedio
segundo_len <- filter(Notas_Victoria_2Grado, asignatura == "LENGUA")$Promedio

# Ya sabemos que no son datos con distribuciones normales. Entonces...
# Al querer comparar las media de los promedios conseguidos por los alumnos de Segundo grado
#en Matematica y Lengua se deberían usar test parametricos, pero con el supuesto de que los 
#datos tengan una distribucion normal. Al no cumplir con esta condición, utilizaremos un 
#test no parametrico como lo es el test de Wilcox pero  en vez de comparar las medias, compararemos 
#las medianas.

# H0 -> la mediana de los alumnos de Segundo grado en Matematica es 8
# H1 -> la mediana de los alumnos de Segundo grado en Matematica es mayor a 8
print("Test de Wilcox para los promedios de Matemática: ")
wilcox.test(segundo_mat, mu=8, alternative = "greater", conf.level = 0.95)

# H0 -> la mediana de los alumnos de Segundo grado en Lengua es 8
# H1 -> la mediana de los alumnos de Segundo grado en Lengua es mayor a 8
print("Test de Wilcox para los promedios de Lengua")
wilcox.test(segundo_len, mu=8, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de Matemática: ")
median(segundo_mat)
print("Mediana de los promedios de Lengua: ")
median(segundo_len)

In [ ]:
# Test de dos medias independientes
# Queremos ver si los promedios de los alumnos de Segundo grado que asisten 
#a una escuela privada son mejores que los de la escuela publica
datos_priv <- filter(Notas_Victoria_2Grado, Gestión == "Privada" & asignatura =="LENGUA")$Promedio
datos_pub <- filter(Notas_Victoria_2Grado, Gestión == "Pública" & asignatura =="LENGUA")$Promedio

# Distribuciones normales?
print("Test de shapiro para los promedios de escuelas privadas: ")
shapiro.test(datos_priv)
print("Test de shapiro para los promedios de escuelas públicas: ")
shapiro.test(datos_pub)

# Los promedios en los dos grupos de datos no siguen una distribucion normal

#H0: Los promedios de los alumnos de Segundo grado en Lengua son iguales en escuelas privadas y públicas.
#H1: Los promedios de los alumnos de Segundo grado en Lengua en escuelas privadas son mayores que en escuelas públicas.
wilcox.test(datos_priv,datos_pub, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de escuelas privadas: ")
median(datos_priv)
print("Mediana de los promedios de escuelas públicas: ")
median(datos_pub)

In [ ]:
# Test de dos medias pareadas
# Queremos ver si las notas de los alumnos de Segundo grado son mejores en el tercer trimestre 
#comparado al primer trimestre
# Filtramos los datos
segundo_1t <- Notas_Victoria_2Grado$NotasPrimerT
segundo_3t <- Notas_Victoria_2Grado$NotasTercerT

# Calculamos las diferencias
diferencias_segundo <- segundo_3t - segundo_1t

# La distribucion de las diferencias es normal?
ks.test(diferencias_segundo, "pnorm") 

# Las diferencias entre los dos grupos de datos no siguen una distribucion normal

# H0 -> Las notas de los alumnos de Segundo grado son iguales en el tercer trimestre 
#que en el primer trimestre.
# H1 -> Las notas de los alumnos de Segundo grado son mayores en el tercer trimestre 
#que en el primer trimestre.
wilcox.test(x=segundo_3t, y=segundo_1t, alternative = "greater", paired = TRUE, correct = FALSE, conf.level=0.95)

print("Medidas del Primer trimestre")
print(mean(segundo_1t))
print(median(segundo_1t))
print("Medidas del Tercer trimestre")
print(mean(segundo_3t))
print(median(segundo_3t))

In [ ]:
# Test de más de dos medias independientes
datos_segundo_mañana <- filter(Notas_Victoria_2Grado, turno == "MAÑANA" & asignatura == "LENGUA")
datos_segundo_tarde <- filter(Notas_Victoria_2Grado, turno == "TARDE" & asignatura == "LENGUA")
datos_segundo_completo <- filter(Notas_Victoria_2Grado, turno == "COMPLETO" & asignatura == "LENGUA")

# Combinamos los datos para graficarlos
datos_comb <- bind_rows(
  datos_segundo_mañana %>% mutate(turno = "MAÑANA"),
  datos_segundo_tarde %>% mutate(turno = "TARDE"),
  datos_segundo_completo %>% mutate(turno = "COMPLETO"))

# Boxplots comparativos interactivos
plot <- plot_ly(datos_comb, x = ~turno, y = ~Promedio, color = ~turno, type = "box") %>%
  layout(
    title = "Comparación de Promedios de Lengua en segundo grado por Turno",
    xaxis = list(title = "Turno"),
    yaxis = list(title = "Promedio"),
    showlegend = FALSE)
plot

In [ ]:
# Pasamos los datos a tipo numerico
segundo = as.numeric(c(datos_segundo_mañana$Promedio, datos_segundo_tarde$Promedio, datos_segundo_completo$Promedio))

# Para que se repita la cantidad de alumno en cada turno su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_segundo_mañana$Promedio)), rep("TARDE",length(datos_segundo_tarde$Promedio)), rep("COMPLETO", length(datos_segundo_completo$Promedio))))    

# Armamos el dataframe
datos_segundo <- data.frame(turno, segundo)

# Agrupamos los datos según el turno, calculamos la media de los promedios y contabilizamos las frecuencias
datos_segundo %>% group_by(turno) %>%
    dplyr::summarise(Media = round(mean(segundo),1), datos = n())

# Test de ANOVA
#H0 ---> Los tres turnos de Segundo tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
#Nivel de significancia = 0.05
fm_segundo = aov(segundo ~ turno)
summary(fm_segundo)

# Potencia del test
potencia_test <- pwr.anova.test(k = 3, n = c(83, 237, 230), f = 0.25, sig.level = 0.05)$power
potencia_test 

# Test HSD de Tukey
TukeyHSD(fm_segundo, "turno", conf.level = .95)

# Pero para saber que todos estos resultados son contundentes, la distribución de los datos tiene que ser
#normal, por eso se calcularan los test de shapiro
shapiro.test(datos_segundo$segundo[datos_segundo$turno == "MAÑANA"])
shapiro.test(datos_segundo$segundo[datos_segundo$turno == "TARDE"])
shapiro.test(datos_segundo$segundo[datos_segundo$turno == "COMPLETO"])

# Como los datos no siguen una distribucion normal, se descarta todo lo calculado. Se procede a calcularlo
#con un test no parametrico

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

# Test no parametrico para primer Segundo
#H0 ---> Los tres turnos de Segundo grado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_segundo$segundo, datos_segundo$turno)
pairwise.wilcox.test(datos_segundo$segundo, datos_segundo$turno)

In [ ]:
# Test de más de dos medias independientes
datos_segundo_mañana <- filter(Notas_Victoria_2Grado, turno == "MAÑANA" & asignatura == "MATEMATICA")
datos_segundo_tarde <- filter(Notas_Victoria_2Grado, turno == "TARDE" & asignatura == "MATEMATICA")
datos_segundo_completo <- filter(Notas_Victoria_2Grado, turno == "COMPLETO" & asignatura == "MATEMATICA")

# Combinamos los datos para graficarlos
datos_comb <- bind_rows(
  datos_segundo_mañana %>% mutate(turno = "MAÑANA"),
  datos_segundo_tarde %>% mutate(turno = "TARDE"),
  datos_segundo_completo %>% mutate(turno = "COMPLETO"))

# Boxplots comparativos interactivos
plot <- plot_ly(datos_comb, x = ~turno, y = ~Promedio, color = ~turno, type = "box") %>%
  layout(
    title = "Comparación de Promedios de Matemática en Segundo por Turno",
    xaxis = list(title = "Turno"),
    yaxis = list(title = "Promedio"),
    showlegend = FALSE)
plot

In [ ]:
# Pasamos los datos a tipo numerico
segundo = as.numeric(c(datos_segundo_mañana$Promedio, datos_segundo_tarde$Promedio, datos_segundo_completo$Promedio))

# Para que se repita la cantidad de alumno en cada turno su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_segundo_mañana$Promedio)), rep("TARDE",length(datos_segundo_tarde$Promedio)), rep("COMPLETO", length(datos_segundo_completo$Promedio))))    

# Armamos el dataframe
datos_segundo <- data.frame(turno, segundo)

# Agrupamos los datos según el turno, calculamos la media de los promedios y contabilizamos las frecuencias
datos_segundo %>% group_by(turno) %>%
    dplyr::summarise(Media = round(mean(segundo),1), datos = n())

# Test de ANOVA
#H0 ---> Los tres turnos de Segundo tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
#Nivel de significancia = 0.05
fm_segundo = aov(segundo ~ turno)
summary(fm_segundo)

# Potencia del test
potencia_test <- pwr.anova.test(k = 3, n = c(83, 237, 230), f = 0.25, sig.level = 0.05)$power
potencia_test 

# Test HSD de Tukey
TukeyHSD(fm_segundo, "turno", conf.level = .95)

# Pero para saber que todos estos resultados son contundentes, la distribución de los datos tiene que ser
#normal, por eso se calcularan los test de shapiro
shapiro.test(datos_segundo$segundo[datos_segundo$turno == "MAÑANA"])
shapiro.test(datos_segundo$segundo[datos_segundo$turno == "TARDE"])
shapiro.test(datos_segundo$segundo[datos_segundo$turno == "COMPLETO"])

# Como los datos no siguen una distribucion normal, se descarta todo lo calculado. Se procede a calcularlo
#con un test no parametrico

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

# Test no parametrico para primer Segundo
#H0 ---> Los tres turnos de Segundo grado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_segundo$segundo, datos_segundo$turno)
pairwise.wilcox.test(datos_segundo$segundo, datos_segundo$turno)

In [ ]:
# Test para mas de dos medias pareadas
# Filtramos los datos 
datos_primer_tri <- filter(Notas_Victoria_2Grado, asignatura == "MATEMATICA")$NotasPrimerT
datos_segundo_tri <- filter(Notas_Victoria_2Grado, asignatura == "MATEMATICA")$NotasSegundoT
datos_tercer_tri <- filter(Notas_Victoria_2Grado, asignatura == "MATEMATICA")$NotasTercerT

# Unimos los datos filtrados a un data frame
datos_completos <- data.frame(
  Trimestre = rep(c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"), each = length(datos_primer_tri)),
  Notas = c(datos_primer_tri, datos_segundo_tri, datos_tercer_tri))

# Convertimos la variable Trimestre a factor
datos_completos$Trimestre <- factor(datos_completos$Trimestre, levels = c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"))

# Creamos un gráfico interactivo de las notas en cada trimestre
plot <- plot_ly() %>%
  add_boxplot(data = datos_completos, x = ~Trimestre, y = ~Notas, color = ~Trimestre, colors = c("blue", "red", "green"), name = "Matemáticas", offsetgroup = 1) %>%
 layout(
    title = "Comparación de notas por trimestre de Matemáticas en Segundo",
    xaxis = list(title = "Trimestre"),
    yaxis = list(title = "Notas"),
    showlegend = FALSE)
plot

In [ ]:
# Calculamos el tamaño de los alumnos de Segundo grado en Matemática para utilizar posteriormente
tamanio = length(datos_primer_tri)

# Armamos el data frame con lo datos
notas_segundo <- data.frame(
 sujeto = 1:tamanio,
 datos_primer_tri <- filter(Notas_Victoria_2Grado, asignatura == "MATEMATICA")$NotasPrimerT,
 datos_segundo_tri <- filter(Notas_Victoria_2Grado, asignatura == "MATEMATICA")$NotasSegundoT,
 datos_tercer_tri <- filter(Notas_Victoria_2Grado, asignatura == "MATEMATICA")$NotasTercerT)

# Esto sirve para armar los datos de manera larga
datos_largo <- melt(notas_segundo, id.vars= "sujeto", variable.name= "Notas", value.name= "Puntuacion")

# Realizar el ANOVA de medias pareadas
modelo_anova <- aov(Puntuacion ~ Notas + Error(sujeto/Notas), data = datos_largo)

# Cambiar el tipo de modelo a lm
modelo_lm <- lm(modelo_anova)

# Calcular el ANOVA con el modelo lm
anova_resultado <- Anova(modelo_lm, type = "III")
print(anova_resultado)

# Calculamos normalidad de los residuos y constatamos que las varianzas sean constantes
print("Test de Normalidad:")
shapiro.test(modelo_lm$residuals)
print("Test de homocedasticidad: ")
bartlett.test(modelo_lm$residuals ~ datos_largo$Notas)

In [ ]:
# Armamos el data frame con lo datos
notas= as.numeric(c(datos_primer_tri,datos_segundo_tri,datos_tercer_tri))
trimestres <- as.factor(c(rep("Primer Trimestre",length(datos_primer_tri)), rep("Segundo Trimestre",length(datos_segundo_tri)),rep("Tercer Trimestre",length(datos_tercer_tri))))
datos <- data.frame(Alumnos=rep(1:tamanio,3), notas, trimestres)
head(datos)

# Calculamos medias de cada grupo
by(data = datos$notas, INDICES = datos$trimestres, FUN = median)

# Test para encontrar diferencias significativas entre los grupos
friedman.test(notas ~ trimestres | Alumnos, data=datos)

# Potencia del test
friedman_anova <- pwr.anova.test(k = 3, n = tamanio, f = 0.25, sig.level = 0.05)$power
print("Potencia del test:")
friedman_anova

# Para saber entre que conjuntos de datos estan las diferencias
pairwise.wilcox.test(datos$notas, datos$trimestres, paired = TRUE)

# Grafico de coordenadas paralelas de los primeros 10 estudiantes (no se hacen todos porque no se entenderia el grafico con tantas lineas)
# Primero modificamos el nombre de los datos para que a la hora de ponerlos en el grafico, se vea mas conciso cada clase
datos_largo$Notas <- gsub("datos_primer_tri.*", "Primer Trimestre", datos_largo$Notas)
datos_largo$Notas <- gsub("datos_segundo_tri.*", "Segundo Trimestre", datos_largo$Notas)
datos_largo$Notas <- gsub("datos_tercer_tri.*", "Tercer Trimestre", datos_largo$Notas)

# Filtramos los primeros 10 alumnos
datos_grafica <- datos_largo %>%
  filter(sujeto <= 10)

# Grafico de coordenadas paralelas de los primeros 10 alumnos con sus notas en cada trimestre
ggplot(datos_grafica, aes(x = Notas, y = Puntuacion, group = sujeto, color = factor(sujeto))) +
  geom_line() +
  geom_point() +
  labs(title = "Evolución de algunas de las notas trimestrales a lo largo del año 2023
    de los alumnos de Segundo Grado en Matemática",
       x = "Trimestre",
       y = "Nota") +
  theme_minimal()

In [ ]:
# Analisis Bivariado
# Armamos una tabla con las materias troncales y el desempeño logrado por los estudiantes
aux <- filter(Notas_Victoria_2Grado, (asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" ))
materias_desempeño <- aux %>%
    select('asignatura', 'Desempeño') %>%
    table()
materias_desempeño

# Reordenamos la tabla
materias_desempeño1 <- cbind(materias_desempeño[,5], materias_desempeño[,1], materias_desempeño[,2], materias_desempeño[,4], materias_desempeño[,3])
colnames(materias_desempeño1) <- c("Regular", "Aprobado", "Bueno","Muy Bueno","Distinguido")
rownames(materias_desempeño1) <- c("ARTES VISUALES", "CS. NATURALES", "CS. SOCIALES","ED. FISICA", "ED. MUSICAL", "ED. TECNOLOGICA", "LENGUA", "MATEMATICA")
materias_desempeño1

print("Tabla con marginales fila y columna")
materias_desempeño2 <- cbind(materias_desempeño1, "Total"= margin.table(materias_desempeño1,1))
materias_desempeño2 <- rbind(materias_desempeño2, "Total"=margin.table(materias_desempeño2,2))
materias_desempeño2

print("Tabla contingencia ~ Porcentaje Total")
tabla_cont_total <- prop.table(materias_desempeño1)*100
tabla_cont_total <- cbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,1))
tabla_cont_total <- rbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,2))
round(tabla_cont_total,1)

print("Tabla contingencia ~ Porcentaje Fila")
tabla_cont_filas <- prop.table(rbind(materias_desempeño1, "Total"=margin.table(materias_desempeño1, 2)),1)*100
tabla_cont_filas <- cbind(tabla_cont_filas, "Total"=margin.table(tabla_cont_filas,1))
round(tabla_cont_filas,1)

print("Tabla contingencia ~ Porcentaje Columna")
tabla_cont_col <- prop.table(cbind(materias_desempeño1, "Total"=margin.table(materias_desempeño1, 1)),2)*100
tabla_cont_col <- rbind(tabla_cont_col, "Total"=margin.table(tabla_cont_col,2))
round(tabla_cont_col,1)


In [ ]:
# Organizamos los datos para hacer un grafico de barras adosadas
tabla_variable_fila <- round(prop.table(table(aux$asignatura, aux$Desempeño), 1)*100,1)
tabla_variable_fila <- t(tabla_variable_fila)
grafico <- barplot(tabla_variable_fila,
                  main= " Grafico de Contigencia por fila",
                  beside= TRUE,
                  space= c(0.1, 1.5),
                  ylim= c(0, 50),
                  cex.axis= 1.2,
                  cex.names= 0.1,
                  cex.main= 1.5,
                  col = c("green","blue", "purple","orange","red"),
                  xlab = "Artes   Cs.Nat   Cs.Soc   Ed.Fis   Ed.Mus   Ed.Tecno  Leng  Mat",
                  cex.lab = 1.2,
                  ylab= "Porcentaje",
                  col.lab= "black")
text(grafico, tabla_variable_fila, labels = tabla_variable_fila, pos = 3, cex = 0.7, col = "black")
legend("topright", rownames(tabla_variable_fila),
       title = "Desempeño",
       title.adj = 0.4,     
       title.col = "black",      
       lty = c(1),
       col = c("green","blue", "purple","orange","red"),
       lwd = 4,
       cex = 1)

# Organizamos los datos para hacer un grafico de barras apiladas
tabla_variable_col <- round(prop.table(table(aux$asignatura, aux$Desempeño), 2) * 100, 1)
grafico <- barplot(tabla_variable_col,
                   main = "Gráfico de Contingencia por columna",
                   beside = FALSE,
                   space = rep(0.25, ncol(tabla_variable_col)),
                   ylim = c(0, 110),
                   cex.axis = 1.2, 
                   cex.names = 1, 
                   cex.main = 1.5, 
                   col = c("green", "red", "blue", "orange", "purple", "cyan", "yellow", "magenta"),
                   xlab = "Desempeños",
                   cex.lab = 1.5,
                   ylab = "Porcentaje",
                   col.lab = "black")
legend(x = 0, y = 115, rownames(tabla_variable_col),cex = 0.5, fill=c("green", "red", "blue", "orange", "purple", "cyan", "yellow", "magenta"),xpd = TRUE,horiz = FALSE,bty = "n")

In [ ]:
# Como muestran los graficos y las tablas, segun el tipo de materia "causa" el desempeño final
#de cada alumno

# Para saber si realmente existe una relación entre las variables aplicaremos el test de chi cuadrado
# H0 ---> No hay relación entre las variables
# H1 ---> Si hay relación entre las variables
chi <-chisq.test(aux$asignatura, aux$Desempeño)
chi

# Vemos la fuerza de asociación
CramerV(aux$asignatura, aux$Desempeño)

In [ ]:
# Frecuencias observadas
chi$observed

# Frecuencias esperadas si las variables fueran independientes
round(chi$expected,0)

# Residuos estandarizados
round(chi$residuals,1)

# Residuos ajustados
round(chi$stdres,1)

In [ ]:
# Análisis Multivariado
# Vector de medias de las variables de Segundo grado agrupadas por las asignaturas
vector_medias <- Notas_Victoria_2Grado %>% group_by(asignatura) %>% summarise_all(mean)

# Nos quedamos con las materias que nos interesan
vector_medias <- vector_medias %>% filter(asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" | asignatura== "CATEQUESIS" | asignatura == "INFORMATICA" |asignatura =="INGLES" )
vector_medias

In [ ]:
# Buscamos valores outliers
aux <- mahalanobis_distance(data= Notas_Victoria_2Grado[,c("NotasPrimerT", "NotasSegundoT", "NotasTercerT", "Promedio")])$is.outlier    
ftable(aux)

In [ ]:
# Grafico de perfiles paralelos para ver el rendimiento de los estudiantes en 11 materias
# Definimos colores manuales para que se distingan
colores <- c("ARTES VISUALES" = "purple","CATEQUESIS" = "orange",
            "CIENCIAS NATURALES" = "green","CIENCIAS SOCIALES" = "blue",
            "EDUCACION FISICA" = "red","EDUCACION MUSICAL" = "turquoise",
            "EDUCACION TECNOLOGICA" = "black","INFORMATICA" = "cyan",
            "INGLES" = "yellow","LENGUA" = "magenta",
            "MATEMATICA" = "skyblue")

# Grafico
ggparcoord(data = vector_medias,
           columns=c(11:14),
           groupColumn = "asignatura",
           showPoints = TRUE,
           scale = "globalminmax") +
  scale_color_manual(values = colores) +
  theme(axis.text.x = element_text(angle = 30, hjust = 1))

In [ ]:
# Prueba de normalidad
mardia2(Notas_Victoria_2Grado[,11:14])
# Los datos no siguen una distribución normal multivariada

In [ ]:
# Asumimos que las medias de las variables para cada grupo se aproximan a una 
#distribución normal multivariada debido al Teorema del Límite Central Multivariado (TLCM). 
# Esto nos permite hacer la comparación de tres o más poblaciones multivariadas independientes
mod2<-manova(cbind(NotasPrimerT, NotasSegundoT, NotasTercerT, Promedio)~turno,data=Notas_Victoria_2Grado)
summary(mod2)

# El MANOVA nos indica que hay diferencias entre los vectores medios de múltiples variables
#dependientes entre dos o más grupos

#Supuestos de normalidad
Notas_Victoria_2Grado %>% group_by(turno) %>%  shapiro_test(NotasPrimerT, NotasSegundoT, NotasTercerT, Promedio)

In [ ]:
# Boxplot comparativo para las diferentes escuelas según el Promedio
g2 <- ggplot(Notas_Victoria_2Grado, aes(factor(CodigoUnicoEscolar), Promedio))
g2 + geom_boxplot(aes(fill = factor(CodigoUnicoEscolar))) +
  labs(title = "Distribución del Promedio por Código Único Escolar",
       x = "Código Único Escolar",
       y = "Promedio",
       fill = "Código Único Escolar") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        legend.position = "none")

In [ ]:
# Boxplot comparativo para las diferentes materias segun el promedio 
g2 <- ggplot(Notas_Victoria_2Grado, aes(factor(asignatura), Promedio))
g2 + geom_boxplot(aes(fill = factor(asignatura))) +
  labs(title = "Distribución del Promedio por materia",
       x = "Materias",
       y = "Promedio",
       fill = "Materias") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        legend.position = "none")

## Análisis de Tercer grado

In [ ]:
# Calculamos con nuestro modelo de predicción lineal las edades de los alumnos de tercer grado
# Aplicamos la función a los estudiantes 
Notas_Victoria_3Grado <- Notas_Victoria_3Grado %>%
  mutate(EdadEstimada = estimar_edad(Documento))

# Ponemos este filtro para aquellos DNI que pertenecen a personas nacionalizado o estan mal cargados, ya que
#el modelo los etiquetaría con una edad muy erronea
Notas_Victoria_3Grado <- Notas_Victoria_3Grado %>%
  mutate(EdadEstimada = ifelse(EdadEstimada < 0 | EdadEstimada > 20, NA, EdadEstimada))

# Reordenamos las variables
Notas_Victoria_3Grado <- dplyr::select(Notas_Victoria_3Grado,'CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno', 'Año_curso', 'Documento', 'asignatura', 'idalumno', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT', 'NotasSegundoT', 'NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', 'EdadEstimada', "total_asig", "porc_aprob", "porc_desaprob")

# Vemos los resultados
Notas_Victoria_3Grado

In [ ]:
# Calculamos el promedio total de cada alumno
aux <- Notas_Victoria_3Grado %>% group_by(idalumno) %>% dplyr::summarise(Promedio_Total = round(mean(Promedio, na.rm = TRUE),2)) 

# Lo unimos con el dataframe original
Notas_Victoria_3Grado <- inner_join(aux, Notas_Victoria_3Grado, by = "idalumno")

# Asigna la materia con mejor promedio para cada estudiante
Notas_Victoria_3Grado$Mejor_asig <- Notas_Victoria_3Grado %>% group_by(idalumno) %>% mutate(Mejor_asig = asignatura[which.max(Promedio)]) %>% ungroup() %>% .$Mejor_asig

# Asigna la materia con peor promedio para cada estudiante
Notas_Victoria_3Grado$Peor_asig <- Notas_Victoria_3Grado %>% group_by(idalumno) %>% mutate(Peor_asig = asignatura[which.min(Promedio)]) %>% ungroup() %>% .$Peor_asig

In [ ]:
# Reordenamos las variables para que queden bien organizadas
Notas_Victoria_3Grado <- Notas_Victoria_3Grado %>% 
  dplyr::select('CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno', 'Año_curso', 'Documento', 'asignatura', 'idalumno', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT', 'NotasSegundoT', 'NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', 'EdadEstimada', "total_asig", "porc_aprob", "porc_desaprob", 'Mejor_asig', 'Peor_asig', 'Promedio_Total')

In [ ]:
# Calculamos las medidas de tendencia central y de dispersión para las 4 materias troncales de primaria
print("Medidas estadisticas descriptivas de tercer grado")
Notas_Victoria_3Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>% 
  group_by(asignatura) %>%
  dplyr::summarise(Conteo = n(),
            Media = round(mean(Promedio, na.rm = TRUE), 2),
            Mediana = round(median(Promedio, na.rm = TRUE), 2),
            Moda = Mode(Promedio),            
            Varianza = round(var(Promedio, na.rm = TRUE), 2),
            Desvio = round(sd(Promedio, na.rm = TRUE), 2),
            Minimo = round(min(Promedio, na.rm = TRUE), 2),
            Maximo = round(max(Promedio, na.rm = TRUE), 2),
            Rango = Maximo - Minimo,
            IQR = round(IQR(Promedio, na.rm = TRUE), 2),
            Q1 = round(quantile(Promedio, 0.25, na.rm = TRUE), 2),
            Q3 = round(quantile(Promedio, 0.75, na.rm = TRUE), 2),
            Sup. = round((Q3 + 1.5 * IQR), 2),
            Inf. = round((Q1 - 1.5 * IQR), 2),
            CoefVar = round(sd(Promedio, na.rm = TRUE) / mean(Promedio, na.rm = TRUE) * 100, 2))

In [ ]:
# Gráfico de boxplot comparativo
datos_filtrados <- Notas_Victoria_3Grado %>%
  filter(asignatura %in% c("CIENCIAS NATURALES", "CIENCIAS SOCIALES", "MATEMATICA", "LENGUA"))

p <- ggplot(datos_filtrados, aes(x = asignatura, y = Promedio, fill = asignatura)) +
  geom_boxplot() +
  ggtitle("                      Comparación de Promedios en materias de tercer grado") +  
  labs(
    x = "Materias",
    y = "Promedios"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )

# Mostramos el gráfico
p

In [ ]:
# Cuantiles para añadir información en los boxplots
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.75)
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.25)

quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.5)

quantile(filter(datos_filtrados, asignatura == "CIENCIAS SOCIALES")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "CIENCIAS SOCIALES")$Promedio, probs = 0.25)

quantile(filter(datos_filtrados, asignatura == "CIENCIAS NATURALES")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "CIENCIAS NATURALES")$Promedio, probs = 0.25)

In [ ]:
# Histograma de promedios en matemática
notas_mat_tercero <- filter(Notas_Victoria_3Grado,asignatura == "MATEMATICA")$Promedio
histograma_mat <- fdt(notas_mat_tercero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_mat,
     type= "fh", 
     col= "#FF9932",
     main= "Histograma de los promedios de tercer grado
     en la materia Matemática")

# Medidas de distribución
coef_sim = skewness(notas_mat_tercero, na.rm=TRUE)
kurtosis = kurtosis(notas_mat_tercero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_mat_tercero)

In [ ]:
# Histograma de promedios en lengua
notas_len_tercero <- filter(Notas_Victoria_3Grado,asignatura == "LENGUA")$Promedio
histograma_len <- fdt(notas_len_tercero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_len,
     type= "fh", 
     col= "#FF86FF",
     main= "Histograma de los promedios de tercer grado
     en la materia Lengua")

# Medidas de distribución
coef_sim = skewness(notas_len_tercero, na.rm=TRUE)
kurtosis = kurtosis(notas_len_tercero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_len_tercero)

In [ ]:
# Histograma de promedios en ciencias naturales
notas_nat_tercero <- filter(Notas_Victoria_3Grado,asignatura == "CIENCIAS NATURALES")$Promedio
histograma_nat <- fdt(notas_nat_tercero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_nat,
     type= "fh", 
     col= "#00BB00",
     main= "Histograma de los promedios de tercer grado
     en la materia Ciencias Naturales")

# Medidas de distribución
coef_sim = skewness(notas_nat_tercero, na.rm=TRUE)
kurtosis = kurtosis(notas_nat_tercero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_nat_tercero)

In [ ]:
# Histograma de promedios en ciencias sociales
notas_soc_tercero <- filter(Notas_Victoria_3Grado,asignatura == "CIENCIAS SOCIALES")$Promedio
histograma_soc <- fdt(notas_soc_tercero, breaks= "Sturges", na.rm= TRUE)
plot(histograma_soc,
     type= "fh", 
     col= "#3D87FF",
     main= "Histograma de los promedios de tercer grado
     en la materia Ciencias Sociales")

# Medidas de distribución
coef_sim = skewness(notas_soc_tercero, na.rm=TRUE)
kurtosis = kurtosis(notas_soc_tercero, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_soc_tercero)

In [ ]:
# Puntuación Z para saber que tipo de turno esta más alejado de un promedio de 8 en Matemática
# Filtramos los datos necesarios 
tercero_mañana <- filter(Notas_Victoria_3Grado, asignatura == "MATEMATICA" & turno == "MAÑANA")
tercero_tarde <- filter(Notas_Victoria_3Grado, asignatura == "MATEMATICA" & turno == "TARDE")
tercero_completo <- filter(Notas_Victoria_3Grado, asignatura == "MATEMATICA" & turno == "COMPLETO")

# Calculamos la media de los promedios
prom_tercero_mañana <- mean(tercero_mañana$Promedio, na.rm = TRUE)
prom_tercero_tarde <- mean(tercero_tarde$Promedio, na.rm = TRUE)
prom_tercero_completo <- mean(tercero_completo$Promedio, na.rm = TRUE)

# Calculamos los desvios de los promedios
desvio_tercero_mañana <- sd(tercero_mañana$Promedio, na.rm = TRUE)
desvio_tercero_tarde <- sd(tercero_tarde$Promedio, na.rm = TRUE)
desvio_tercero_completo <- sd(tercero_completo$Promedio, na.rm = TRUE)

# Establecemos el promedio a comparar
promedio_comparar <- 8

# Puntuación Z para el turno mañana
punt_z_mañana <- (promedio_comparar - prom_tercero_mañana) / desvio_tercero_mañana
print(paste(" Puntuación Z para los promedios de Matemática en tercer grado del turno mañana: ", round(punt_z_mañana, 2)))

# Puntuación Z para el turno tarde
punt_z_tarde <- (promedio_comparar - prom_tercero_tarde) / desvio_tercero_tarde
print(paste(" Puntuación Z para los promedios de Matemática en tercer grado del turno tarde: ", round(punt_z_tarde, 2)))

# Puntuación Z para el turno completo
punt_z_completo <- (promedio_comparar - prom_tercero_completo) / desvio_tercero_completo
print(paste(" Puntuación Z para los promedios de Matemática en tercer grado del turno completo: ", round(punt_z_completo, 2)))

In [ ]:
# Puntuación Z en estudiantes de escuelas de gestión pública y gestión privada
# Filtramos los datos necesarios 
tercer_publica <- filter(Notas_Victoria_3Grado, asignatura == "LENGUA" & Gestión == "Pública")
tercer_privada <- filter(Notas_Victoria_3Grado, asignatura == "LENGUA" & Gestión == "Privada")

# Calculamos la media de los promedios
prom_tercer_publica <- mean(tercer_publica$Promedio, na.rm = TRUE)
prom_tercer_privada <- mean(tercer_privada$Promedio, na.rm = TRUE)

# Calculamos los desvios de los promedios
desvio_tercer_publica <- sd(tercer_publica$Promedio, na.rm = TRUE)
desvio_tercer_privada <- sd(tercer_privada$Promedio, na.rm = TRUE)

# Establecemos el promedio a comparar
promedio_comparar <- 8

# Puntuación Z para escuelas publicas
punt_z_publica <- (promedio_comparar - prom_tercer_publica) / desvio_tercer_publica
print(paste(" Puntuación Z para los promedios de Lengua en tercer grado de escuelas públicas: ", round(punt_z_publica, 2)))

# Puntuación Z para escuelas privadas
punt_z_privada <- (promedio_comparar - prom_tercer_privada) / desvio_tercer_privada
print(paste(" Puntuación Z para los promedios de Lengua en tercer grado de escuelas privadas: ", round(punt_z_privada, 2)))

In [ ]:
# Correlaciones
# Unimos en un data frame las materias seleccionadas para calcular correlaciones
df <- suppressWarnings(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "CIENCIAS NATURALES"),"Documento","Promedio") %>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "CIENCIAS SOCIALES"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "MATEMATICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "LENGUA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "EDUCACION FISICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "EDUCACION MUSICAL"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "EDUCACION TECNOLOGICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_3Grado,asignatura == "ARTES VISUALES"),"Documento","Promedio"),by="Documento"))
df <- rename(df,Prom_Nat=Promedio.x, Prom_Soc=Promedio.y, Prom_Mat=Promedio.x.x, Prom_Len=Promedio.y.y, Prom_EdFisica=Promedio.x.x.x, Prom_Mus=Promedio.y.y.y, Prom_Tecno=Promedio.x.x.x.x, Prom_Art=Promedio.y.y.y.y)

# Gráfico de la matriz de correlación
matriz_correlacion <- cor(df[, -1], method="spearman")
corrplot(matriz_correlacion, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Matriz de correlación", cex.main = 1.5)

# Gráfico de correlación parcial
variables <- c("Prom_Nat", "Prom_Soc","Prom_Mat", "Prom_Len", "Prom_Mus", "Prom_Tecno", "Prom_Art")
dataM <- df[, variables]
correlacion_parcial <- pcor(dataM, method = "pearson")$estimate
corrplot(correlacion_parcial, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Correlación parcial", cex.main = 1.5)

In [ ]:
# Test de una media
# Filtramos los datos
# En este caso, a los promedios de tercer grado en las materias de Lengua y Matematica
tercer_mat <- filter(Notas_Victoria_3Grado, asignatura == "MATEMATICA")$Promedio
tercer_len <- filter(Notas_Victoria_3Grado, asignatura == "LENGUA")$Promedio

# Ya sabemos que no son datos con distribuciones normales
# Entonces...
# Al querer comparar las media de los promedios conseguidos por los alumnos de tercer grado en Matematica y
# Lengua se deberían usar test parametricos, pero con el supuesto de que los datos tengan una distribucion 
#normal. Al no cumplir con esta condición, utilizaremos un test no parametrico como lo es el test de Wilcox
#pero  en vez de comparar las medias, compararemos las medianas.

# H0 -> la mediana de los alumnos de tercer grado en Matematica es 8
# H1 -> la mediana de los alumnos de tercer grado en Matematica es mayor a 8
print("Test de Wilcox para los promedios de Matemática: ")
wilcox.test(tercer_mat, mu=8, alternative = "greater", conf.level = 0.95)

# H0 -> la mediana de los alumnos de tercer grado en Lengua es 8
# H1 -> la mediana de los alumnos de tercer grado en Lengua es mayor a 8
print("Test de Wilcox para los promedios de Lengua")
wilcox.test(tercer_len, mu=8, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de Matemática: ")
median(tercer_mat)
print("Mediana de los promedios de Lengua: ")
median(tercer_len)

In [ ]:
# Test de dos medias independientes
# Queremos ver si los promedios de los alumnos de tercer grado que asisten a una escuela privada son mejores que los de la escuela publica
datos_priv <- filter(Notas_Victoria_3Grado, Gestión == "Privada" & asignatura =="MATEMATICA")$Promedio
datos_pub <- filter(Notas_Victoria_3Grado, Gestión == "Pública" & asignatura =="MATEMATICA")$Promedio

# Distribuciones normales?
print("Test de shapiro para los promedios de escuelas privadas: ")
shapiro.test(datos_priv)
print("Test de shapiro para los promedios de escuelas públicas: ")
shapiro.test(datos_pub)

# Los promedios en los dos grupos de datos no siguen una distribucion normal

#H0: Los promedios de los alumnos de tercer grado en Lengua son iguales en escuelas privadas y públicas.
#H1: Los promedios de los alumnos de tercer grado en Lengua en escuelas privadas son mayores que en escuelas públicas.
wilcox.test(datos_priv,datos_pub, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de escuelas privadas: ")
median(datos_priv)
print("Mediana de los promedios de escuelas públicas: ")
median(datos_pub)

In [ ]:
# Test de dos medias pareadas
# Queremos ver si las notas de los alumnos de tercer grado son mejores en el tercer trimestre comparado al primer trimestre
# Filtramos los datos
tercer_1t <- Notas_Victoria_3Grado$NotasPrimerT
tercer_3t <- Notas_Victoria_3Grado$NotasTercerT

# Calculamos las diferencias
diferencias_tercer <- tercer_3t - tercer_1t

# La distribucion de las diferencias es normal?
ks.test(diferencias_tercer, "pnorm") 

# Las diferencias entre los dos grupos de datos no siguen una distribucion normal

# H0 -> Las notas de los alumnos de tercer grado son iguales en el tercer trimestre que en el primer trimestre.
# H1 -> Las notas de los alumnos de tercer grado son mayores en el tercer trimestre que en el primer trimestre.
wilcox.test(x=tercer_3t, y=tercer_1t, alternative = "greater", paired = TRUE, correct = FALSE, conf.level=0.95)

print("Medidas del Primer trimestre")
print(mean(tercer_1t))
print(median(tercer_1t))
print("Medidas del Tercer trimestre")
print(mean(tercer_3t))
print(median(tercer_3t))

In [ ]:
# Test de más de dos medias independientes
datos_mañana <- filter(Notas_Victoria_3Grado, turno == "MAÑANA" & asignatura == "MATEMATICA")
datos_tarde <- filter(Notas_Victoria_3Grado, turno == "TARDE" & asignatura == "MATEMATICA")
datos_completo <- filter(Notas_Victoria_3Grado, turno == "COMPLETO" & asignatura == "MATEMATICA")

# Combinamos los datos para graficarlos
datos_comb <- bind_rows(
  datos_mañana %>% mutate(turno = "MAÑANA"),
  datos_tarde %>% mutate(turno = "TARDE"),
  datos_completo %>% mutate(turno = "COMPLETO"))

# Boxplots comparativos interactivos
plot <- plot_ly(datos_comb, x = ~turno, y = ~Promedio, color = ~turno, type = "box") %>%
  layout(
    title = "Comparación de Promedios de Matemáticas en tercero por Turno",
    xaxis = list(title = "Turno"),
    yaxis = list(title = "Promedio"),
    showlegend = FALSE)
plot

In [ ]:
# Pasamos los datos a tipo numerico
tercero = as.numeric(c(datos_mañana$Promedio, datos_tarde$Promedio, datos_completo$Promedio))

# Para que se repita la cantidad de alumno en cada turno su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana$Promedio)), rep("TARDE",length(datos_tarde$Promedio)), rep("COMPLETO", length(datos_completo$Promedio))))    

# Armamos el dataframe
datos_tercero <- data.frame(turno, tercero)

# Agrupamos los datos según el turno, calculamos la media de los promedios y contabilizamos las frecuencias
datos_tercero %>% group_by(turno) %>%
    dplyr::summarise(Media = round(mean(tercero),1), datos = n())

# Test de ANOVA
#H0 ---> Los tres turnos de TERCERO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
#Nivel de significancia = 0.05
fm_tercero = aov(tercero ~ turno)
summary(fm_tercero)

# Potencia del test
potencia_test <- pwr.anova.test(k = 3, n = c(83, 249, 248), f = 0.25, sig.level = 0.05)$power
potencia_test 

# Test HSD de Tukey
TukeyHSD(fm_tercero, "turno", conf.level = .95)

# Pero para saber que todos estos resultados son contundentes, la distribución de los datos tiene que ser
#normal, por eso se calcularan los test de shapiro
shapiro.test(datos_tercero$tercero[datos_tercero$turno == "MAÑANA"])
shapiro.test(datos_tercero$tercero[datos_tercero$turno == "TARDE"])
shapiro.test(datos_tercero$tercero[datos_tercero$turno == "COMPLETO"])

# Como los datos no siguen una distribucion normal, se descarta todo lo calculado. Se procede a calcularlo
#con un test no parametrico

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

# Test no parametrico para primer grado
#H0 ---> Los tres turnos de primer grado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_tercero$tercero, datos_tercero$turno)
pairwise.wilcox.test(datos_tercero$tercero, datos_tercero$turno)

In [ ]:
# Test de más de dos medias independientes
datos_mañana <- filter(Notas_Victoria_3Grado, turno == "MAÑANA" & asignatura == "LENGUA")
datos_tarde <- filter(Notas_Victoria_3Grado, turno == "TARDE" & asignatura == "LENGUA")
datos_completo <- filter(Notas_Victoria_3Grado, turno == "COMPLETO" & asignatura == "LENGUA")

# Combinamos los datos para graficarlos
datos_comb <- bind_rows(
  datos_mañana %>% mutate(turno = "MAÑANA"),
  datos_tarde %>% mutate(turno = "TARDE"),
  datos_completo %>% mutate(turno = "COMPLETO"))

# Boxplots comparativos interactivos
plot <- plot_ly(datos_comb, x = ~turno, y = ~Promedio, color = ~turno, type = "box") %>%
  layout(
    title = "Comparación de Promedios de Lengua en tercero por Turno",
    xaxis = list(title = "Turno"),
    yaxis = list(title = "Promedio"),
    showlegend = FALSE)
plot

In [ ]:
# Pasamos los datos a tipo numerico
tercero = as.numeric(c(datos_mañana$Promedio, datos_tarde$Promedio, datos_completo$Promedio))

# Para que se repita la cantidad de alumno en cada turno su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana$Promedio)), rep("TARDE",length(datos_tarde$Promedio)), rep("COMPLETO", length(datos_completo$Promedio))))    

# Armamos el dataframe
datos_tercero <- data.frame(turno, tercero)

# Agrupamos los datos según el turno, calculamos la media de los promedios y contabilizamos las frecuencias
datos_tercero %>% group_by(turno) %>%
    dplyr::summarise(Media = round(mean(tercero),1), datos = n())

# Test de ANOVA
#H0 ---> Los tres turnos de TERCERO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
#Nivel de significancia = 0.05
fm_tercero = aov(tercero ~ turno)
summary(fm_tercero)

# Potencia del test
potencia_test <- pwr.anova.test(k = 3, n = c(83, 247, 248), f = 0.25, sig.level = 0.05)$power
potencia_test 

# Test HSD de Tukey
TukeyHSD(fm_tercero, "turno", conf.level = .95)

# Pero para saber que todos estos resultados son contundentes, la distribución de los datos tiene que ser
#normal, por eso se calcularan los test de shapiro
shapiro.test(datos_tercero$tercero[datos_tercero$turno == "MAÑANA"])
shapiro.test(datos_tercero$tercero[datos_tercero$turno == "TARDE"])
shapiro.test(datos_tercero$tercero[datos_tercero$turno == "COMPLETO"])

# Como los datos no siguen una distribucion normal, se descarta todo lo calculado. Se procede a calcularlo
#con un test no parametrico

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

# Test no parametrico para primer grado
#H0 ---> Los tres turnos de primer grado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_tercero$tercero, datos_tercero$turno)
pairwise.wilcox.test(datos_tercero$tercero, datos_tercero$turno)

In [ ]:
# Test para mas de dos medias pareadas
# Filtramos los datos 
datos_primer_tri <- filter(Notas_Victoria_3Grado, asignatura == "MATEMATICA")$NotasPrimerT
datos_segundo_tri <- filter(Notas_Victoria_3Grado, asignatura == "MATEMATICA")$NotasSegundoT
datos_tercer_tri <- filter(Notas_Victoria_3Grado, asignatura == "MATEMATICA")$NotasTercerT

# Unimos los datos filtrados a un data frame
datos_completos <- data.frame(
  Trimestre = rep(c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"), each = length(datos_primer_tri)),
  Notas = c(datos_primer_tri, datos_segundo_tri, datos_tercer_tri))

# Convertimos la variable Trimestre a factor
datos_completos$Trimestre <- factor(datos_completos$Trimestre, levels = c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"))

# Creamos un gráfico interactivo de las notas en cada trimestre
plot <- plot_ly() %>%
  add_boxplot(data = datos_completos, x = ~Trimestre, y = ~Notas, color = ~Trimestre, colors = c("blue", "red", "green"), name = "Matemáticas", offsetgroup = 1) %>%
 layout(
    title = "Comparación de notas por trimestre de Matemáticas en Tercero",
    xaxis = list(title = "Trimestre"),
    yaxis = list(title = "Notas"),
    showlegend = FALSE)
plot

In [ ]:
# Calculamos el tamaño de los alumnos de tercer grado en Matemática para utilizar posteriormente
tamanio = length(datos_primer_tri)

# Armamos el data frame con lo datos
notas <- data.frame(
 sujeto = 1:tamanio,
 datos_primer_tri <- filter(Notas_Victoria_3Grado, asignatura == "MATEMATICA")$NotasPrimerT,
 datos_segundo_tri <- filter(Notas_Victoria_3Grado, asignatura == "MATEMATICA")$NotasSegundoT,
 datos_tercer_tri <- filter(Notas_Victoria_3Grado, asignatura == "MATEMATICA")$NotasTercerT)

# Esto sirve para armar los datos de manera larga
datos_largo <- melt(notas, id.vars= "sujeto", variable.name= "Notas", value.name= "Puntuacion")

# Realizar el ANOVA de medias pareadas
modelo_anova <- aov(Puntuacion ~ Notas + Error(sujeto/Notas), data = datos_largo)

# Cambiar el tipo de modelo a lm
modelo_lm <- lm(modelo_anova)

# Calcular el ANOVA con el modelo lm
anova_resultado <- Anova(modelo_lm, type = "III")
print(anova_resultado)

# Calculamos normalidad de los residuos y constatamos que las varianzas sean constantes
print("Test de Normalidad:")
shapiro.test(modelo_lm$residuals)
print("Test de homocedasticidad: ")
bartlett.test(modelo_lm$residuals ~ datos_largo$Notas)

In [ ]:
# Armamos el data frame con lo datos
notas= as.numeric(c(datos_primer_tri,datos_segundo_tri,datos_tercer_tri))
trimestres <- as.factor(c(rep("Primer Trimestre",length(datos_primer_tri)), rep("Segundo Trimestre",length(datos_segundo_tri)),rep("Tercer Trimestre",length(datos_tercer_tri))))
datos <- data.frame(Alumnos=rep(1:tamanio,3), notas, trimestres)
head(datos)

# Calculamos medias de cada grupo
by(data = datos$notas, INDICES = datos$trimestres, FUN = median)

# Test para encontrar diferencias significativas entre los grupos
friedman.test(notas ~ trimestres | Alumnos, data=datos)

# Potencia del test
friedman_anova <- pwr.anova.test(k = 3, n = tamanio, f = 0.25, sig.level = 0.05)$power
print("Potencia del test:")
friedman_anova

# Para saber entre que conjuntos de datos estan las diferencias
pairwise.wilcox.test(datos$notas, datos$trimestres, paired = TRUE)

# Grafico de coordenadas paralelas de los primeros 10 estudiantes (no se hacen todos porque no se entenderia el grafico con tantas lineas)
# Primero modificamos el nombre de los datos para que a la hora de ponerlos en el grafico, se vea mas conciso cada clase
datos_largo$Notas <- gsub("datos_primer_tri.*", "Primer Trimestre", datos_largo$Notas)
datos_largo$Notas <- gsub("datos_segundo_tri.*", "Segundo Trimestre", datos_largo$Notas)
datos_largo$Notas <- gsub("datos_tercer_tri.*", "Tercer Trimestre", datos_largo$Notas)

# Filtramos los primeros 10 alumnos
datos_grafica <- datos_largo %>%
  filter(sujeto <= 10)

# Grafico de coordenadas paralelas de los primeros 10 alumnos con sus notas en cada trimestre
ggplot(datos_grafica, aes(x = Notas, y = Puntuacion, group = sujeto, color = factor(sujeto))) +
  geom_line() +
  geom_point() +
  labs(title = "Evolución de algunas de las notas trimestrales a lo largo del año 2023
    de los alumnos de Tercer Grado en Matemática",
       x = "Trimestre",
       y = "Nota") +
  theme_minimal()

In [ ]:
# Armamos una tabla con las materias troncales y el desempeño logrado por los estudiantes
aux <- filter(Notas_Victoria_3Grado, (asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" ))
materias_desempeño <- aux %>%
    select('asignatura', 'Desempeño') %>%
    table()
materias_desempeño

# Reordenamos la tabla
materias_desempeño1 <- cbind(materias_desempeño[,5], materias_desempeño[,1], materias_desempeño[,2], materias_desempeño[,4], materias_desempeño[,3])
colnames(materias_desempeño1) <- c("Regular", "Aprobado", "Bueno","Muy Bueno","Distinguido")
rownames(materias_desempeño1) <- c("ARTES VISUALES", "CS. NATURALES", "CS. SOCIALES","ED. FISICA", "ED. MUSICAL", "ED. TECNOLOGICA", "LENGUA", "MATEMATICA")
materias_desempeño1

print("Tabla con marginales fila y columna")
materias_desempeño2 <- cbind(materias_desempeño1, "Total"= margin.table(materias_desempeño1,1))
materias_desempeño2 <- rbind(materias_desempeño2, "Total"=margin.table(materias_desempeño2,2))
materias_desempeño2

print("Tabla contingencia ~ Porcentaje Total")
tabla_cont_total <- prop.table(materias_desempeño1)*100
tabla_cont_total <- cbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,1))
tabla_cont_total <- rbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,2))
round(tabla_cont_total,1)

print("Tabla contingencia ~ Porcentaje Fila")
tabla_cont_filas <- prop.table(rbind(materias_desempeño1, "Total"=margin.table(materias_desempeño1, 2)),1)*100
tabla_cont_filas <- cbind(tabla_cont_filas, "Total"=margin.table(tabla_cont_filas,1))
round(tabla_cont_filas,1)

print("Tabla contingencia ~ Porcentaje Columna")
tabla_cont_col <- prop.table(cbind(materias_desempeño1, "Total"=margin.table(materias_desempeño1, 1)),2)*100
tabla_cont_col <- rbind(tabla_cont_col, "Total"=margin.table(tabla_cont_col,2))
round(tabla_cont_col,1)


In [ ]:
# Organizamos los datos para hacer un grafico de barras adosadas
tabla_variable_fila <- round(prop.table(table(aux$asignatura, aux$Desempeño), 1)*100,1)
tabla_variable_fila <- t(tabla_variable_fila)
grafico <- barplot(tabla_variable_fila,
                  main= " Grafico de Contigencia por fila",
                  beside= TRUE,
                  space= c(0.1, 1.5),
                  ylim= c(0, 50),
                  cex.axis= 1.2,
                  cex.names= 0.1,
                  cex.main= 1.5,
                  col = c("green","blue", "purple","orange","red"),
                  xlab = "Artes   Cs.Nat   Cs.Soc   Ed.Fis   Ed.Mus   Ed.Tecno  Leng  Mat",
                  cex.lab = 1.2,
                  ylab= "Porcentaje",
                  col.lab= "black")
text(grafico, tabla_variable_fila, labels = tabla_variable_fila, pos = 3, cex = 0.7, col = "black")
legend("topright", rownames(tabla_variable_fila),
       title = "Desempeño",
       title.adj = 0.4,     
       title.col = "black",      
       lty = c(1),
       col = c("green","blue", "purple","orange","red"),
       lwd = 4,
       cex = 1)

# Organizamos los datos para hacer un grafico de barras apiladas
tabla_variable_col <- round(prop.table(table(aux$asignatura, aux$Desempeño), 2) * 100, 1)
grafico <- barplot(tabla_variable_col,
                   main = "Gráfico de Contingencia por columna",
                   beside = FALSE,
                   space = rep(0.25, ncol(tabla_variable_col)),
                   ylim = c(0, 110),
                   cex.axis = 1.2, 
                   cex.names = 1, 
                   cex.main = 1.5, 
                   col = c("green", "red", "blue", "orange", "purple", "cyan", "yellow", "magenta"),
                   xlab = "Desempeños",
                   cex.lab = 1.5,
                   ylab = "Porcentaje",
                   col.lab = "black")
legend(x = 0, y = 115, rownames(tabla_variable_col),cex = 0.5, fill=c("green", "red", "blue", "orange", "purple", "cyan", "yellow", "magenta"),xpd = TRUE,horiz = FALSE,bty = "n")

In [ ]:
# Como muestran los graficos y las tablas, segun el tipo de materia "causa" el desempeño final
#de cada alumno
# Para saber si realmente existe una relación entre las variables aplicaremos el test de chi cuadrado
# H0 ---> No hay relación entre las variables
# H1 ---> Si hay relación entre las variables
chi <-chisq.test(aux$asignatura, aux$Desempeño)
chi

# Vemos la fuerza de asociación
CramerV(aux$asignatura, aux$Desempeño)

In [ ]:
# Frecuencias observadas
chi$observed

# Frecuencias esperadas si las variables fueran independientes
round(chi$expected,0)

# Residuos estandarizados
round(chi$residuals,1)

# Residuos ajustados
round(chi$stdres,1)

In [ ]:
# Análisis Multivariado
# Vector de medias de las variables de primer grado agrupadas por las asignaturas
vector_medias <- Notas_Victoria_3Grado %>% group_by(asignatura) %>% summarise_all(mean)
# Nos quedamos con las materias que nos interesan
vector_medias <- vector_medias %>% filter(asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" | asignatura== "CATEQUESIS" | asignatura == "INFORMATICA" |asignatura =="INGLES" )
vector_medias

In [ ]:
# Buscamos valores outliers
aux <- mahalanobis_distance(data= Notas_Victoria_3Grado[,c("NotasPrimerT", "NotasSegundoT", "NotasTercerT", "Promedio")])$is.outlier    
ftable(aux)

In [ ]:
# Grafico de perfiles paralelos para ver el rendimiento de los estudiantes en 11 materias
# Definimos colores manuales para que se distingan
colores <- c("ARTES VISUALES" = "purple","CATEQUESIS" = "orange",
            "CIENCIAS NATURALES" = "green","CIENCIAS SOCIALES" = "blue",
            "EDUCACION FISICA" = "red","EDUCACION MUSICAL" = "turquoise",
            "EDUCACION TECNOLOGICA" = "black","INFORMATICA" = "cyan",
            "INGLES" = "yellow","LENGUA" = "magenta",
            "MATEMATICA" = "skyblue")

# Grafico
ggparcoord(data = vector_medias,
           columns=c(11:14),
           groupColumn = "asignatura",
           showPoints = TRUE,
           scale = "globalminmax") +
  scale_color_manual(values = colores) +
  theme(axis.text.x = element_text(angle = 30, hjust = 1))

In [ ]:
# Prueba de normalidad
mardia2(Notas_Victoria_3Grado[,11:14])
# Los datos no siguen una distribución normal multivariada

In [ ]:
# COMPARACION DE TRES O MÁS POBLACIONES MULTIVARIADAS INDEPENDIENTES
mod3<-manova(cbind(NotasPrimerT, NotasSegundoT, NotasTercerT, Promedio)~turno,data=Notas_Victoria_3Grado)
summary(mod3)

# El MANOVA nos indica que hay diferencias entre los vectores medios

#Supuestos de normalidad
Notas_Victoria_3Grado %>% group_by(turno) %>%  shapiro_test(NotasPrimerT, NotasSegundoT, NotasTercerT, Promedio)
# Ningun grupo de datos sigue una distribucion normal multivariada

## Análisis de Cuarto grado

In [ ]:
# Calculamos con nuestro modelo de predicción lineal las edades de los alumnos de cuarto grado
# Aplicamos la función a los estudiantes 
Notas_Victoria_4Grado <- Notas_Victoria_4Grado %>%
  mutate(EdadEstimada = estimar_edad(Documento))

# Ponemos este filtro para aquellos DNI que pertenecen a personas nacionalizado o estan mal cargados, ya que
#el modelo los etiquetaría con una edad muy erronea
Notas_Victoria_4Grado <- Notas_Victoria_4Grado %>%
  mutate(EdadEstimada = ifelse(EdadEstimada < 0 | EdadEstimada > 20, NA, EdadEstimada))

# Reordenamos las variables
Notas_Victoria_4Grado <- dplyr::select(Notas_Victoria_4Grado,'CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno', 'Año_curso', 'Documento', 'asignatura', 'idalumno', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT', 'NotasSegundoT', 'NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', 'EdadEstimada', "total_asig", "porc_aprob", "porc_desaprob")

# Vemos los resultados
Notas_Victoria_4Grado

In [ ]:
# Calculamos el promedio total de cada alumno
aux <- Notas_Victoria_4Grado %>% group_by(idalumno) %>% dplyr::summarise(Promedio_Total = round(mean(Promedio, na.rm = TRUE),2)) 

# Lo unimos con el dataframe original
Notas_Victoria_4Grado <- inner_join(aux, Notas_Victoria_4Grado, by = "idalumno")

# Asigna la materia con mejor promedio para cada estudiante
Notas_Victoria_4Grado$Mejor_asig <- Notas_Victoria_4Grado %>% group_by(idalumno) %>% mutate(Mejor_asig = asignatura[which.max(Promedio)]) %>% ungroup() %>% .$Mejor_asig

# Asigna la materia con peor promedio para cada estudiante
Notas_Victoria_4Grado$Peor_asig <- Notas_Victoria_4Grado %>% group_by(idalumno) %>% mutate(Peor_asig = asignatura[which.min(Promedio)]) %>% ungroup() %>% .$Peor_asig

In [ ]:
# Reordenamos las variables para que queden bien organizadas
Notas_Victoria_4Grado <- Notas_Victoria_4Grado %>% 
  dplyr::select('CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno', 'Año_curso', 'Documento', 'asignatura', 'idalumno', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT', 'NotasSegundoT', 'NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', 'EdadEstimada', "total_asig", "porc_aprob", "porc_desaprob", 'Mejor_asig', 'Peor_asig', 'Promedio_Total')

In [ ]:
# Calculamos las medidas de tendencia central y de dispersión para las 4 materias troncales de primaria
print("Medidas estadisticas descriptivas de cuarto grado")
Notas_Victoria_4Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>% 
  group_by(asignatura) %>%
  dplyr::summarise(Conteo = n(),
            Media = round(mean(Promedio, na.rm = TRUE), 2),
            Mediana = round(median(Promedio, na.rm = TRUE), 2),
            Moda = Mode(Promedio),            
            Varianza = round(var(Promedio, na.rm = TRUE), 2),
            Desvio = round(sd(Promedio, na.rm = TRUE), 2),
            Minimo = round(min(Promedio, na.rm = TRUE), 2),
            Maximo = round(max(Promedio, na.rm = TRUE), 2),
            Rango = Maximo - Minimo,
            IQR = round(IQR(Promedio, na.rm = TRUE), 2),
            Q1 = round(quantile(Promedio, 0.25, na.rm = TRUE), 2),
            Q3 = round(quantile(Promedio, 0.75, na.rm = TRUE), 2),
            Sup. = round((Q3 + 1.5 * IQR), 2),
            Inf. = round((Q1 - 1.5 * IQR), 2),
            CoefVar = round(sd(Promedio, na.rm = TRUE) / mean(Promedio, na.rm = TRUE) * 100, 2))

In [ ]:
# Gráfico de boxplot comparativo
datos_filtrados <- Notas_Victoria_4Grado %>%
  filter(asignatura %in% c("CIENCIAS NATURALES", "CIENCIAS SOCIALES", "MATEMATICA", "LENGUA"))

p <- ggplot(datos_filtrados, aes(x = asignatura, y = Promedio, fill = asignatura)) +
  geom_boxplot() +
  ggtitle("                      Comparación de Promedios en materias de cuarto grado") +  
  labs(
    x = "Materias",
    y = "Promedios"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )

# Mostramos el gráfico
p

In [ ]:
# Cuantiles para añadir información en los boxplots
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.75)
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.25)

quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.15)

quantile(filter(datos_filtrados, asignatura == "CIENCIAS SOCIALES")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "CIENCIAS SOCIALES")$Promedio, probs = 0.25)

quantile(filter(datos_filtrados, asignatura == "CIENCIAS NATURALES")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "CIENCIAS NATURALES")$Promedio, probs = 0.25)

In [ ]:
# Histograma de promedios en matemática
notas_mat_cuarto <- filter(Notas_Victoria_4Grado,asignatura == "MATEMATICA")$Promedio
histograma_mat <- fdt(notas_mat_cuarto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_mat,
     type= "fh", 
     col= "#FF9932",
     main= "Histograma de los promedios de cuarto grado
     en la materia Matemática")

# Medidas de distribución
coef_sim = skewness(notas_mat_cuarto, na.rm=TRUE)
kurtosis = kurtosis(notas_mat_cuarto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_mat_cuarto)

In [ ]:
# Histograma de promedios en lengua
notas_len_cuarto <- filter(Notas_Victoria_4Grado,asignatura == "LENGUA")$Promedio
histograma_len <- fdt(notas_len_cuarto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_len,
     type= "fh", 
     col= "#FF86FF",
     main= "Histograma de los promedios de cuarto grado
     en la materia Lengua")

# Medidas de distribución
coef_sim = skewness(notas_len_cuarto, na.rm=TRUE)
kurtosis = kurtosis(notas_len_cuarto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_len_cuarto)

In [ ]:
# Histograma de promedios en ciencias naturales
notas_nat_cuarto <- filter(Notas_Victoria_4Grado,asignatura == "CIENCIAS NATURALES")$Promedio
histograma_nat <- fdt(notas_nat_cuarto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_nat,
     type= "fh", 
     col= "#00BB00",
     main= "Histograma de los promedios de cuarto grado
     en la materia Ciencias Naturales")

# Medidas de distribución
coef_sim = skewness(notas_nat_cuarto, na.rm=TRUE)
kurtosis = kurtosis(notas_nat_cuarto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_nat_cuarto)

In [ ]:
# Histograma de promedios en ciencias sociales
notas_soc_cuarto <- filter(Notas_Victoria_4Grado,asignatura == "CIENCIAS SOCIALES")$Promedio
histograma_soc <- fdt(notas_soc_cuarto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_soc,
     type= "fh", 
     col= "#3D87FF",
     main= "Histograma de los promedios de cuarto grado
     en la materia Ciencias Sociales")

# Medidas de distribución
coef_sim = skewness(notas_soc_cuarto, na.rm=TRUE)
kurtosis = kurtosis(notas_soc_cuarto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_soc_cuarto)

In [ ]:
# Puntuación Z para saber que tipo de turno esta más alejado de un promedio de 8 en Matemática
# Filtramos los datos necesarios 
cuarto_mañana <- filter(Notas_Victoria_4Grado, asignatura == "MATEMATICA" & turno == "MAÑANA")
cuarto_tarde <- filter(Notas_Victoria_4Grado, asignatura == "MATEMATICA" & turno == "TARDE")
cuarto_completo <- filter(Notas_Victoria_4Grado, asignatura == "MATEMATICA" & turno == "COMPLETO")

# Calculamos la media de los promedios
prom_cuarto_mañana <- mean(cuarto_mañana$Promedio, na.rm = TRUE)
prom_cuarto_tarde <- mean(cuarto_tarde$Promedio, na.rm = TRUE)
prom_cuarto_completo <- mean(cuarto_completo$Promedio, na.rm = TRUE)

# Calculamos los desvios de los promedios
desvio_cuarto_mañana <- sd(cuarto_mañana$Promedio, na.rm = TRUE)
desvio_cuarto_tarde <- sd(cuarto_tarde$Promedio, na.rm = TRUE)
desvio_cuarto_completo <- sd(cuarto_completo$Promedio, na.rm = TRUE)

# Establecemos el promedio a comparar
promedio_comparar <- 8

# Puntuación Z para el turno mañana
punt_z_mañana <- (promedio_comparar - prom_cuarto_mañana) / desvio_cuarto_mañana
print(paste(" Puntuación Z para los promedios de Matemática en cuarto grado del turno mañana: ", round(punt_z_mañana, 2)))

# Puntuación Z para el turno tarde
punt_z_tarde <- (promedio_comparar - prom_cuarto_tarde) / desvio_cuarto_tarde
print(paste(" Puntuación Z para los promedios de Matemática en cuarto grado del turno tarde: ", round(punt_z_tarde, 2)))

# Puntuación Z para el turno completo
punt_z_completo <- (promedio_comparar - prom_cuarto_completo) / desvio_cuarto_completo
print(paste(" Puntuación Z para los promedios de Matemática en cuarto grado del turno completo: ", round(punt_z_completo, 2)))

In [ ]:
# Puntuación Z en estudiantes de escuelas de gestión pública y gestión privada
# Filtramos los datos necesarios 
cuarto_publica <- filter(Notas_Victoria_4Grado, asignatura == "MATEMATICA" & Gestión == "Pública")
cuarto_privada <- filter(Notas_Victoria_4Grado, asignatura == "MATEMATICA" & Gestión == "Privada")

# Calculamos la media de los promedios
prom_cuarto_publica <- mean(cuarto_publica$Promedio, na.rm = TRUE)
prom_cuarto_privada <- mean(cuarto_privada$Promedio, na.rm = TRUE)

# Calculamos los desvios de los promedios
desvio_cuarto_publica <- sd(cuarto_publica$Promedio, na.rm = TRUE)
desvio_cuarto_privada <- sd(cuarto_privada$Promedio, na.rm = TRUE)

# Establecemos el promedio a comparar
promedio_comparar <- 8

# Puntuación Z para escuelas publicas
punt_z_publica <- (promedio_comparar - prom_cuarto_publica) / desvio_cuarto_publica
print(paste(" Puntuación Z para los promedios de Matemática en cuarto grado de escuelas públicas: ", round(punt_z_publica, 2)))

# Puntuación Z para escuelas privadas
punt_z_privada <- (promedio_comparar - prom_cuarto_privada) / desvio_cuarto_privada
print(paste(" Puntuación Z para los promedios de Matemática en cuarto grado de escuelas privadas: ", round(punt_z_privada, 2)))

In [ ]:
# Unimos en un data frame las materias seleccionadas para calcular correlaciones
df <- suppressWarnings(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "CIENCIAS NATURALES"),"Documento","Promedio") %>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "CIENCIAS SOCIALES"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "MATEMATICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "LENGUA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "EDUCACION FISICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "EDUCACION MUSICAL"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "EDUCACION TECNOLOGICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_4Grado,asignatura == "ARTES VISUALES"),"Documento","Promedio"),by="Documento"))
df <- rename(df,Prom_Nat=Promedio.x, Prom_Soc=Promedio.y, Prom_Mat=Promedio.x.x, Prom_Len=Promedio.y.y, Prom_EdFisica=Promedio.x.x.x, Prom_Mus=Promedio.y.y.y, Prom_Tecno=Promedio.x.x.x.x, Prom_Art=Promedio.y.y.y.y)

# Gráfico de la matriz de correlación
matriz_correlacion <- cor(df[, -1], method="spearman")
corrplot(matriz_correlacion, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Matriz de correlación", cex.main = 1.5)

# Gráfico de correlación parcial
correlacion_parcial <- pcor(df[, -1], method = "pearson")$estimate
corrplot(correlacion_parcial, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Correlación parcial", cex.main = 1.5)

In [ ]:
# Test de una media
# Filtramos los datos
# En este caso, a los promedios de cuarto grado en las materias de Lengua y Matematica
cuarto_mat <- filter(Notas_Victoria_4Grado, asignatura == "MATEMATICA")$Promedio
cuarto_len <- filter(Notas_Victoria_4Grado, asignatura == "LENGUA")$Promedio

# Ya sabemos que no son datos con distribuciones normales
# Entonces...
# Al querer comparar las media de los promedios conseguidos por los alumnos de cuarto grado en Matematica y
# Lengua se deberían usar test parametricos, pero con el supuesto de que los datos tengan una distribucion 
#normal. Al no cumplir con esta condición, utilizaremos un test no parametrico como lo es el test de Wilcox
#pero  en vez de comparar las medias, compararemos las medianas.

# H0 -> la mediana de los alumnos de cuarto grado en Matematica es 8
# H1 -> la mediana de los alumnos de cuarto grado en Matematica es mayor a 8
print("Test de Wilcox para los promedios de Matemática: ")
wilcox.test(cuarto_mat, mu=8, alternative = "greater", conf.level = 0.95)

# H0 -> la mediana de los alumnos de cuarto grado en Lengua es 8
# H1 -> la mediana de los alumnos de cuarto grado en Lengua es mayor a 8
print("Test de Wilcox para los promedios de Lengua")
wilcox.test(cuarto_len, mu=8, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de Matemática: ")
median(cuarto_mat)
print("Mediana de los promedios de Lengua: ")
median(cuarto_len)

In [ ]:
# Test de dos medias independientes
# Queremos ver si los promedios de los alumnos de tercer grado que asisten a una escuela privada son mejores que los de la escuela publica
datos_priv <- filter(Notas_Victoria_4Grado, Gestión == "Privada" & asignatura =="LENGUA")$Promedio
datos_pub <- filter(Notas_Victoria_4Grado, Gestión == "Pública" & asignatura =="LENGUA")$Promedio

# Distribuciones normales?
print("Test de shapiro para los promedios de escuelas privadas: ")
shapiro.test(datos_priv)
print("Test de shapiro para los promedios de escuelas públicas: ")
shapiro.test(datos_pub)

# Los promedios en los dos grupos de datos no siguen una distribucion normal

#H0: Los promedios de los alumnos de cuarto grado en Lengua son iguales en escuelas privadas y públicas.
#H1: Los promedios de los alumnos de cuarto grado en Lengua en escuelas privadas son mayores que en escuelas públicas.
wilcox.test(datos_priv,datos_pub, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de escuelas privadas: ")
median(datos_priv)
print("Mediana de los promedios de escuelas públicas: ")
median(datos_pub)

In [ ]:
# Test de más de dos medias independientes
datos_mañana <- filter(Notas_Victoria_4Grado, turno == "MAÑANA" & asignatura == "MATEMATICA")
datos_tarde <- filter(Notas_Victoria_4Grado, turno == "TARDE" & asignatura == "MATEMATICA")
datos_completo <- filter(Notas_Victoria_4Grado, turno == "COMPLETO" & asignatura == "MATEMATICA")

# Combinamos los datos para graficarlos
datos_comb <- bind_rows(
  datos_mañana %>% mutate(turno = "MAÑANA"),
  datos_tarde %>% mutate(turno = "TARDE"),
  datos_completo %>% mutate(turno = "COMPLETO"))

# Boxplots comparativos interactivos
plot <- plot_ly(datos_comb, x = ~turno, y = ~Promedio, color = ~turno, type = "box") %>%
  layout(
    title = "Comparación de Promedios de Matemáticas en cuarto por Turno",
    xaxis = list(title = "Turno"),
    yaxis = list(title = "Promedio"),
    showlegend = FALSE)
plot

In [ ]:
# Pasamos los datos a tipo numerico
cuarto = as.numeric(c(datos_mañana$Promedio, datos_tarde$Promedio, datos_completo$Promedio))

# Para que se repita la cantidad de alumno en cada turno su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana$Promedio)), rep("TARDE",length(datos_tarde$Promedio)), rep("COMPLETO", length(datos_completo$Promedio))))    

# Armamos el dataframe
datos_cuarto <- data.frame(turno, cuarto)

# Agrupamos los datos según el turno, calculamos la media de los promedios y contabilizamos las frecuencias
datos_cuarto %>% group_by(turno) %>%
    dplyr::summarise(Media = round(mean(cuarto),1), datos = n())

# Test de ANOVA
#H0 ---> Los tres turnos de CUARTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
#Nivel de significancia = 0.05
fm_cuarto = aov(cuarto ~ turno)
summary(fm_cuarto)

# Potencia del test (CAMBIAR)
potencia_test <- pwr.anova.test(k = 3, n = c(66, 232, 186), f = 0.25, sig.level = 0.05)$power
potencia_test 

# Test HSD de Tukey
TukeyHSD(fm_cuarto, "turno", conf.level = .95)

# Pero para saber que todos estos resultados son contundentes, la distribución de los datos tiene que ser
#normal, por eso se calcularan los test de shapiro
shapiro.test(datos_cuarto$cuarto[datos_cuarto$turno == "MAÑANA"])
shapiro.test(datos_cuarto$cuarto[datos_cuarto$turno == "TARDE"])
shapiro.test(datos_cuarto$cuarto[datos_cuarto$turno == "COMPLETO"])

# Como los datos no siguen una distribucion normal, se descarta todo lo calculado. Se procede a calcularlo
#con un test no parametrico

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

# Test no parametrico para primer grado
#H0 ---> Los tres turnos de primer grado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_cuarto$cuarto, datos_cuarto$turno)
pairwise.wilcox.test(datos_cuarto$cuarto, datos_cuarto$turno)

In [ ]:
# Test de más de dos medias independientes
datos_mañana <- filter(Notas_Victoria_4Grado, turno == "MAÑANA" & asignatura == "LENGUA")
datos_tarde <- filter(Notas_Victoria_4Grado, turno == "TARDE" & asignatura == "LENGUA")
datos_completo <- filter(Notas_Victoria_4Grado, turno == "COMPLETO" & asignatura == "LENGUA")

# Combinamos los datos para graficarlos
datos_comb <- bind_rows(
  datos_mañana %>% mutate(turno = "MAÑANA"),
  datos_tarde %>% mutate(turno = "TARDE"),
  datos_completo %>% mutate(turno = "COMPLETO"))

# Boxplots comparativos interactivos
plot <- plot_ly(datos_comb, x = ~turno, y = ~Promedio, color = ~turno, type = "box") %>%
  layout(
    title = "Comparación de Promedios de Lengua en cuarto por Turno",
    xaxis = list(title = "Turno"),
    yaxis = list(title = "Promedio"),
    showlegend = FALSE)
plot

In [ ]:
# Pasamos los datos a tipo numerico
cuarto = as.numeric(c(datos_mañana$Promedio, datos_tarde$Promedio, datos_completo$Promedio))

# Para que se repita la cantidad de alumno en cada turno su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana$Promedio)), rep("TARDE",length(datos_tarde$Promedio)), rep("COMPLETO", length(datos_completo$Promedio))))    

# Armamos el dataframe
datos_cuarto <- data.frame(turno, cuarto)

# Agrupamos los datos según el turno, calculamos la media de los promedios y contabilizamos las frecuencias
datos_cuarto %>% group_by(turno) %>%
    dplyr::summarise(Media = round(mean(cuarto),1), datos = n())

# Test de ANOVA
#H0 ---> Los tres turnos de CUARTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
#Nivel de significancia = 0.05
fm_cuarto = aov(cuarto ~ turno)
summary(fm_cuarto)

# Potencia del test (CAMBIAR)
potencia_test <- pwr.anova.test(k = 3, n = c(66, 218, 185), f = 0.25, sig.level = 0.05)$power
potencia_test 

# Test HSD de Tukey
TukeyHSD(fm_cuarto, "turno", conf.level = .95)

# Pero para saber que todos estos resultados son contundentes, la distribución de los datos tiene que ser
#normal, por eso se calcularan los test de shapiro
shapiro.test(datos_cuarto$cuarto[datos_cuarto$turno == "MAÑANA"])
shapiro.test(datos_cuarto$cuarto[datos_cuarto$turno == "TARDE"])
shapiro.test(datos_cuarto$cuarto[datos_cuarto$turno == "COMPLETO"])

# Como los datos no siguen una distribucion normal, se descarta todo lo calculado. Se procede a calcularlo
#con un test no parametrico

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

# Test no parametrico para primer grado
#H0 ---> Los tres turnos de primer grado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_cuarto$cuarto, datos_cuarto$turno)
pairwise.wilcox.test(datos_cuarto$cuarto, datos_cuarto$turno)

In [ ]:
# Test para mas de dos medias pareadas
# Filtramos los datos 
datos_primer_tri <- filter(Notas_Victoria_4Grado, asignatura == "MATEMATICA")$NotasPrimerT
datos_segundo_tri <- filter(Notas_Victoria_4Grado, asignatura == "MATEMATICA")$NotasSegundoT
datos_tercer_tri <- filter(Notas_Victoria_4Grado, asignatura == "MATEMATICA")$NotasTercerT

# Unimos los datos filtrados a un data frame
datos_completos <- data.frame(
  Trimestre = rep(c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"), each = length(datos_primer_tri)),
  Notas = c(datos_primer_tri, datos_segundo_tri, datos_tercer_tri))

# Convertimos la variable Trimestre a factor
datos_completos$Trimestre <- factor(datos_completos$Trimestre, levels = c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"))

# Creamos un gráfico interactivo de las notas en cada trimestre
plot <- plot_ly() %>%
  add_boxplot(data = datos_completos, x = ~Trimestre, y = ~Notas, color = ~Trimestre, colors = c("blue", "red", "green"), name = "Matemáticas", offsetgroup = 1) %>%
 layout(
    title = "Comparación de notas por trimestre de Matemáticas en Cuarto",
    xaxis = list(title = "Trimestre"),
    yaxis = list(title = "Notas"),
    showlegend = FALSE)
plot

In [ ]:
# Calculamos el tamaño de los alumnos de tercer grado en Matemática para utilizar posteriormente
tamanio = length(datos_primer_tri)

# Armamos el data frame con lo datos
notas <- data.frame(
 sujeto = 1:tamanio,
 datos_primer_tri <- filter(Notas_Victoria_4Grado, asignatura == "MATEMATICA")$NotasPrimerT,
 datos_segundo_tri <- filter(Notas_Victoria_4Grado, asignatura == "MATEMATICA")$NotasSegundoT,
 datos_tercer_tri <- filter(Notas_Victoria_4Grado, asignatura == "MATEMATICA")$NotasTercerT)

# Esto sirve para armar los datos de manera larga
datos_largo <- melt(notas, id.vars= "sujeto", variable.name= "Notas", value.name= "Puntuacion")

# Realizar el ANOVA de medias pareadas
modelo_anova <- aov(Puntuacion ~ Notas + Error(sujeto/Notas), data = datos_largo)

# Cambiar el tipo de modelo a lm
modelo_lm <- lm(modelo_anova)

# Calcular el ANOVA con el modelo lm
anova_resultado <- Anova(modelo_lm, type = "III")
print(anova_resultado)

# Calculamos normalidad de los residuos y constatamos que las varianzas sean constantes
print("Test de Normalidad:")
shapiro.test(modelo_lm$residuals)
print("Test de homocedasticidad: ")
bartlett.test(modelo_lm$residuals ~ datos_largo$Notas)

In [ ]:
# Armamos el data frame con lo datos
notas= as.numeric(c(datos_primer_tri,datos_segundo_tri,datos_tercer_tri))
trimestres <- as.factor(c(rep("Primer Trimestre",length(datos_primer_tri)), rep("Segundo Trimestre",length(datos_segundo_tri)),rep("Tercer Trimestre",length(datos_tercer_tri))))
datos <- data.frame(Alumnos=rep(1:tamanio,3), notas, trimestres)
head(datos)

# Calculamos medias de cada grupo
by(data = datos$notas, INDICES = datos$trimestres, FUN = median)

# Test para encontrar diferencias significativas entre los grupos
friedman.test(notas ~ trimestres | Alumnos, data=datos)

# Potencia del test
friedman_anova <- pwr.anova.test(k = 3, n = tamanio, f = 0.25, sig.level = 0.05)$power
print("Potencia del test:")
friedman_anova

# Para saber entre que conjuntos de datos estan las diferencias
pairwise.wilcox.test(datos$notas, datos$trimestres, paired = TRUE)

# Grafico de coordenadas paralelas de los primeros 10 estudiantes (no se hacen todos porque no se entenderia el grafico con tantas lineas)
# Primero modificamos el nombre de los datos para que a la hora de ponerlos en el grafico, se vea mas conciso cada clase
datos_largo$Notas <- gsub("datos_primer_tri.*", "Primer Trimestre", datos_largo$Notas)
datos_largo$Notas <- gsub("datos_segundo_tri.*", "Segundo Trimestre", datos_largo$Notas)
datos_largo$Notas <- gsub("datos_tercer_tri.*", "Tercer Trimestre", datos_largo$Notas)

# Filtramos los primeros 10 alumnos
datos_grafica <- datos_largo %>%
  filter(sujeto <= 10)

# Grafico de coordenadas paralelas de los primeros 10 alumnos con sus notas en cada trimestre
ggplot(datos_grafica, aes(x = Notas, y = Puntuacion, group = sujeto, color = factor(sujeto))) +
  geom_line() +
  geom_point() +
  labs(title = "Evolución de algunas de las notas trimestrales a lo largo del año 2023
    de los alumnos de Cuarto Grado en Matemática",
       x = "Trimestre",
       y = "Nota") +
  theme_minimal()

In [ ]:
# Armamos una tabla con las materias troncales y el desempeño logrado por los estudiantes
aux <- filter(Notas_Victoria_4Grado, (asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" ))
materias_desempeño <- aux %>%
    select('asignatura', 'Desempeño') %>%
    table()
materias_desempeño

# Reordenamos la tabla
materias_desempeño1 <- cbind(materias_desempeño[,5], materias_desempeño[,1], materias_desempeño[,2], materias_desempeño[,4], materias_desempeño[,3])
colnames(materias_desempeño1) <- c("Regular", "Aprobado", "Bueno","Muy Bueno","Distinguido")
rownames(materias_desempeño1) <- c("ARTES VISUALES", "CS. NATURALES", "CS. SOCIALES","ED. FISICA", "ED. MUSICAL", "ED. TECNOLOGICA", "LENGUA", "MATEMATICA")
materias_desempeño1

print("Tabla con marginales fila y columna")
materias_desempeño2 <- cbind(materias_desempeño1, "Total"= margin.table(materias_desempeño1,1))
materias_desempeño2 <- rbind(materias_desempeño2, "Total"=margin.table(materias_desempeño2,2))
materias_desempeño2

print("Tabla contingencia ~ Porcentaje Total")
tabla_cont_total <- prop.table(materias_desempeño1)*100
tabla_cont_total <- cbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,1))
tabla_cont_total <- rbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,2))
round(tabla_cont_total,1)

print("Tabla contingencia ~ Porcentaje Fila")
tabla_cont_filas <- prop.table(rbind(materias_desempeño1, "Total"=margin.table(materias_desempeño1, 2)),1)*100
tabla_cont_filas <- cbind(tabla_cont_filas, "Total"=margin.table(tabla_cont_filas,1))
round(tabla_cont_filas,1)

print("Tabla contingencia ~ Porcentaje Columna")
tabla_cont_col <- prop.table(cbind(materias_desempeño1, "Total"=margin.table(materias_desempeño1, 1)),2)*100
tabla_cont_col <- rbind(tabla_cont_col, "Total"=margin.table(tabla_cont_col,2))
round(tabla_cont_col,1)


In [ ]:
# Organizamos los datos para hacer un grafico de barras adosadas
tabla_variable_fila <- round(prop.table(table(aux$asignatura, aux$Desempeño), 1)*100,1)
tabla_variable_fila <- t(tabla_variable_fila)
grafico <- barplot(tabla_variable_fila,
                  main= " Grafico de Contigencia por fila",
                  beside= TRUE,
                  space= c(0.1, 1.5),
                  ylim= c(0, 50),
                  cex.axis= 1.2,
                  cex.names= 0.1,
                  cex.main= 1.5,
                  col = c("green","blue", "purple","orange","red"),
                  xlab = "Artes   Cs.Nat   Cs.Soc   Ed.Fis   Ed.Mus   Ed.Tecno  Leng  Mat",
                  cex.lab = 1.2,
                  ylab= "Porcentaje",
                  col.lab= "black")
text(grafico, tabla_variable_fila, labels = tabla_variable_fila, pos = 3, cex = 0.7, col = "black")
legend("topright", rownames(tabla_variable_fila),
       title = "Desempeño",
       title.adj = 0.4,     
       title.col = "black",      
       lty = c(1),
       col = c("green","blue", "purple","orange","red"),
       lwd = 4,
       cex = 1)

# Organizamos los datos para hacer un grafico de barras apiladas
tabla_variable_col <- round(prop.table(table(aux$asignatura, aux$Desempeño), 2) * 100, 1)
grafico <- barplot(tabla_variable_col,
                   main = "Gráfico de Contingencia por columna",
                   beside = FALSE,
                   space = rep(0.25, ncol(tabla_variable_col)),
                   ylim = c(0, 110),
                   cex.axis = 1.2, 
                   cex.names = 1, 
                   cex.main = 1.5, 
                   col = c("green", "red", "blue", "orange", "purple", "cyan", "yellow", "magenta"),
                   xlab = "Desempeños",
                   cex.lab = 1.5,
                   ylab = "Porcentaje",
                   col.lab = "black")
legend(x = 0, y = 115, rownames(tabla_variable_col),cex = 0.5, fill=c("green", "red", "blue", "orange", "purple", "cyan", "yellow", "magenta"),xpd = TRUE,horiz = FALSE,bty = "n")

In [ ]:
# Como muestran los graficos y las tablas, segun el tipo de materia "causa" el desempeño final
#de cada alumno
# Para saber si realmente existe una relación entre las variables aplicaremos el test de chi cuadrado
# H0 ---> No hay relación entre las variables
# H1 ---> Si hay relación entre las variables
chi <-chisq.test(aux$asignatura, aux$Desempeño)
chi

# Vemos la fuerza de asociación
CramerV(aux$asignatura, aux$Desempeño)

In [ ]:
# Frecuencias observadas
chi$observed

# Frecuencias esperadas si las variables fueran independientes
round(chi$expected,0)

# Residuos estandarizados
round(chi$residuals,1)

# Residuos ajustados
round(chi$stdres,1)

In [ ]:
# Análisis Multivariado
# Vector de medias de las variables de primer grado agrupadas por las asignaturas
vector_medias <- Notas_Victoria_4Grado %>% group_by(asignatura) %>% summarise_all(mean)
# Nos quedamos con las materias que nos interesan
vector_medias <- vector_medias %>% filter(asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" | asignatura== "CATEQUESIS" | asignatura == "INFORMATICA" |asignatura =="INGLES" )
vector_medias

In [ ]:
# Buscamos valores outliers
aux <- mahalanobis_distance(data= Notas_Victoria_4Grado[,c("NotasPrimerT", "NotasSegundoT", "NotasTercerT", "Promedio")])$is.outlier    
ftable(aux)

In [ ]:
# Grafico de perfiles paralelos para ver el rendimiento de los estudiantes en 11 materias
# Definimos colores manuales para que se distingan
colores <- c("ARTES VISUALES" = "purple","CATEQUESIS" = "orange",
            "CIENCIAS NATURALES" = "green","CIENCIAS SOCIALES" = "blue",
            "EDUCACION FISICA" = "red","EDUCACION MUSICAL" = "turquoise",
            "EDUCACION TECNOLOGICA" = "black","INFORMATICA" = "cyan",
            "INGLES" = "yellow","LENGUA" = "magenta",
            "MATEMATICA" = "skyblue")

# Grafico
ggparcoord(data = vector_medias,
           columns=c(11:14),
           groupColumn = "asignatura",
           showPoints = TRUE,
           scale = "globalminmax") +
  scale_color_manual(values = colores) +
  theme(axis.text.x = element_text(angle = 30, hjust = 1))

In [ ]:
# Prueba de normalidad
mardia2(Notas_Victoria_4Grado[,11:14])
# Los datos no siguen una distribución normal multivariada

In [ ]:
# Asumimos que las medias de las variables para cada grupo se aproximan a una 
#distribución normal multivariada debido al Teorema del Límite Central Multivariado (TLCM). 
# Esto nos permite hacer la comparación de tres o más poblaciones multivariadas independientes
mod4<-manova(cbind(NotasPrimerT, NotasSegundoT, NotasTercerT, Promedio)~turno,data=Notas_Victoria_4Grado)
summary(mod4)

# El MANOVA nos indica que hay diferencias entre los vectores medios

## Análisis de Quinto grado

In [ ]:
# Calculamos con nuestro modelo de predicción lineal las edades de los alumnos de quinto grado
# Aplicamos la función a los estudiantes 
Notas_Victoria_5Grado <- Notas_Victoria_5Grado %>%
  mutate(EdadEstimada = estimar_edad(Documento))

# Ponemos este filtro para aquellos DNI que pertenecen a personas nacionalizado o estan mal cargados, ya que
#el modelo los etiquetaría con una edad muy erronea
Notas_Victoria_5Grado <- Notas_Victoria_5Grado %>%
  mutate(EdadEstimada = ifelse(EdadEstimada < 0 | EdadEstimada > 20, NA, EdadEstimada))

# Reordenamos las variables
Notas_Victoria_5Grado <- dplyr::select(Notas_Victoria_5Grado,'CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno', 'Año_curso', 'Documento', 'asignatura', 'idalumno', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT', 'NotasSegundoT', 'NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', 'EdadEstimada', "total_asig", "porc_aprob", "porc_desaprob")

# Vemos los resultados
Notas_Victoria_5Grado

In [ ]:
# Calculamos el promedio total de cada alumno
aux <- Notas_Victoria_5Grado %>% group_by(idalumno) %>% dplyr::summarise(Promedio_Total = round(mean(Promedio, na.rm = TRUE),2)) 

# Lo unimos con el dataframe original
Notas_Victoria_5Grado <- inner_join(aux, Notas_Victoria_5Grado, by = "idalumno")

# Asigna la materia con mejor promedio para cada estudiante
Notas_Victoria_5Grado$Mejor_asig <- Notas_Victoria_5Grado %>% group_by(idalumno) %>% mutate(Mejor_asig = asignatura[which.max(Promedio)]) %>% ungroup() %>% .$Mejor_asig

# Asigna la materia con peor promedio para cada estudiante
Notas_Victoria_5Grado$Peor_asig <- Notas_Victoria_5Grado %>% group_by(idalumno) %>% mutate(Peor_asig = asignatura[which.min(Promedio)]) %>% ungroup() %>% .$Peor_asig

In [ ]:
# Reordenamos las variables para que queden bien organizadas
Notas_Victoria_5Grado <- Notas_Victoria_5Grado %>% 
  dplyr::select('CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno', 'Año_curso', 'Documento', 'asignatura', 'idalumno', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT', 'NotasSegundoT', 'NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', 'EdadEstimada', "total_asig", "porc_aprob", "porc_desaprob", 'Mejor_asig', 'Peor_asig', 'Promedio_Total')

In [ ]:
# Calculamos las medidas de tendencia central y de dispersión para las 4 materias troncales de primaria
print("Medidas estadisticas descriptivas de quinto grado")
Notas_Victoria_5Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>% 
  group_by(asignatura) %>%
  dplyr::summarise(Conteo = n(),
            Media = round(mean(Promedio, na.rm = TRUE), 2),
            Mediana = round(median(Promedio, na.rm = TRUE), 2),
            Moda = Mode(Promedio),            
            Varianza = round(var(Promedio, na.rm = TRUE), 2),
            Desvio = round(sd(Promedio, na.rm = TRUE), 2),
            Minimo = round(min(Promedio, na.rm = TRUE), 2),
            Maximo = round(max(Promedio, na.rm = TRUE), 2),
            Rango = Maximo - Minimo,
            IQR = round(IQR(Promedio, na.rm = TRUE), 2),
            Q1 = round(quantile(Promedio, 0.25, na.rm = TRUE), 2),
            Q3 = round(quantile(Promedio, 0.75, na.rm = TRUE), 2),
            Sup. = round((Q3 + 1.5 * IQR), 2),
            Inf. = round((Q1 - 1.5 * IQR), 2),
            CoefVar = round(sd(Promedio, na.rm = TRUE) / mean(Promedio, na.rm = TRUE) * 100, 2))

In [ ]:
# Gráfico de boxplot comparativo
datos_filtrados <- Notas_Victoria_5Grado %>%
  filter(asignatura %in% c("CIENCIAS NATURALES", "CIENCIAS SOCIALES", "MATEMATICA", "LENGUA"))

p <- ggplot(datos_filtrados, aes(x = asignatura, y = Promedio, fill = asignatura)) +
  geom_boxplot() +
  ggtitle("                      Comparación de Promedios en materias de quinto grado") +  
  labs(
    x = "Materias",
    y = "Promedios"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )

# Mostramos el gráfico
p

In [ ]:
# Cuantiles para añadir información en los boxplots
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.75)
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.25)
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.14)

quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.75)
quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.50)
quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.25)
quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.14)

quantile(filter(datos_filtrados, asignatura == "CIENCIAS SOCIALES")$Promedio, probs = 0.15)
quantile(filter(datos_filtrados, asignatura == "CIENCIAS SOCIALES")$Promedio, probs = 0.10)

quantile(filter(datos_filtrados, asignatura == "CIENCIAS NATURALES")$Promedio, probs = 0.15)
quantile(filter(datos_filtrados, asignatura == "CIENCIAS NATURALES")$Promedio, probs = 0.10)

In [ ]:
# Histograma de promedios en matemática
notas_mat_quinto <- filter(Notas_Victoria_5Grado,asignatura == "MATEMATICA")$Promedio
histograma_mat <- fdt(notas_mat_quinto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_mat,
     type= "fh", 
     col= "#FF9932",
     main= "Histograma de los promedios de quinto grado
     en la materia Matemática")

# Medidas de distribución
coef_sim = skewness(notas_mat_quinto, na.rm=TRUE)
kurtosis = kurtosis(notas_mat_quinto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_mat_quinto)

In [ ]:
# Histograma de promedios en lengua
notas_len_quinto <- filter(Notas_Victoria_5Grado,asignatura == "LENGUA")$Promedio
histograma_len <- fdt(notas_len_quinto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_len,
     type= "fh", 
     col= "#FF86FF",
     main= "Histograma de los promedios de quinto grado
     en la materia Lengua")

# Medidas de distribución
coef_sim = skewness(notas_len_quinto, na.rm=TRUE)
kurtosis = kurtosis(notas_len_quinto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_len_quinto)

In [ ]:
# Histograma de promedios en ciencias naturales
notas_nat_quinto <- filter(Notas_Victoria_5Grado,asignatura == "CIENCIAS NATURALES")$Promedio
histograma_nat <- fdt(notas_nat_quinto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_nat,
     type= "fh", 
     col= "#00BB00",
     main= "Histograma de los promedios de quinto grado
     en la materia Ciencias Naturales")

# Medidas de distribución
coef_sim = skewness(notas_nat_quinto, na.rm=TRUE)
kurtosis = kurtosis(notas_nat_quinto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_nat_quinto)

In [ ]:
# Histograma de promedios en ciencias sociales
notas_soc_quinto <- filter(Notas_Victoria_5Grado,asignatura == "CIENCIAS SOCIALES")$Promedio
histograma_soc <- fdt(notas_soc_quinto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_soc,
     type= "fh", 
     col= "#3D87FF",
     main= "Histograma de los promedios de quinto grado
     en la materia Ciencias Sociales")

# Medidas de distribución
coef_sim = skewness(notas_soc_quinto, na.rm=TRUE)
kurtosis = kurtosis(notas_soc_quinto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_soc_quinto)

In [ ]:
# Puntuación Z para saber que tipo de turno esta más alejado de un promedio de 8 en Matemática
# Filtramos los datos necesarios 
quinto_mañana <- filter(Notas_Victoria_5Grado, asignatura == "LENGUA" & turno == "MAÑANA")
quinto_tarde <- filter(Notas_Victoria_5Grado, asignatura == "LENGUA" & turno == "TARDE")
quinto_completo <- filter(Notas_Victoria_5Grado, asignatura == "LENGUA" & turno == "COMPLETO")

# Calculamos la media de los promedios
prom_quinto_mañana <- mean(quinto_mañana$Promedio, na.rm = TRUE)
prom_quinto_tarde <- mean(quinto_tarde$Promedio, na.rm = TRUE)
prom_quinto_completo <- mean(quinto_completo$Promedio, na.rm = TRUE)

# Calculamos los desvios de los promedios
desvio_quinto_mañana <- sd(quinto_mañana$Promedio, na.rm = TRUE)
desvio_quinto_tarde <- sd(quinto_tarde$Promedio, na.rm = TRUE)
desvio_quinto_completo <- sd(quinto_completo$Promedio, na.rm = TRUE)

# Establecemos el promedio a comparar
promedio_comparar <- 8

# Puntuación Z para el turno mañana
punt_z_mañana <- (promedio_comparar - prom_quinto_mañana) / desvio_quinto_mañana
print(paste(" Puntuación Z para los promedios de Lengua en quinto grado del turno mañana: ", round(punt_z_mañana, 2)))

# Puntuación Z para el turno tarde
punt_z_tarde <- (promedio_comparar - prom_quinto_tarde) / desvio_quinto_tarde
print(paste(" Puntuación Z para los promedios de Lengua en quinto grado del turno tarde: ", round(punt_z_tarde, 2)))

# Puntuación Z para el turno completo
punt_z_completo <- (promedio_comparar - prom_quinto_completo) / desvio_quinto_completo
print(paste(" Puntuación Z para los promedios de Lengua en quinto grado del turno completo: ", round(punt_z_completo, 2)))

In [ ]:
# Puntuación Z en estudiantes de escuelas de gestión pública y gestión privada
# Filtramos los datos necesarios 
quinto_publica <- filter(Notas_Victoria_5Grado, asignatura == "MATEMATICA" & Gestión == "Pública")
quinto_privada <- filter(Notas_Victoria_5Grado, asignatura == "MATEMATICA" & Gestión == "Privada")

# Calculamos la media de los promedios
prom_quinto_publica <- mean(quinto_publica$Promedio, na.rm = TRUE)
prom_quinto_privada <- mean(quinto_privada$Promedio, na.rm = TRUE)

# Calculamos los desvios de los promedios
desvio_quinto_publica <- sd(quinto_publica$Promedio, na.rm = TRUE)
desvio_quinto_privada <- sd(quinto_privada$Promedio, na.rm = TRUE)

# Establecemos el promedio a comparar
promedio_comparar <- 8

# Puntuación Z para escuelas publicas
punt_z_publica <- (promedio_comparar - prom_quinto_publica) / desvio_quinto_publica
print(paste(" Puntuación Z para los promedios de Matemática en quinto grado de escuelas públicas: ", round(punt_z_publica, 2)))

# Puntuación Z para escuelas privadas
punt_z_privada <- (promedio_comparar - prom_quinto_privada) / desvio_quinto_privada
print(paste(" Puntuación Z para los promedios de Matemática en quinto grado de escuelas privadas: ", round(punt_z_privada, 2)))

In [ ]:
# Unimos en un data frame las materias seleccionadas para calcular correlaciones
df <- suppressWarnings(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "CIENCIAS NATURALES"),"Documento","Promedio") %>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "CIENCIAS SOCIALES"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "MATEMATICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "LENGUA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "EDUCACION FISICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "EDUCACION MUSICAL"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "EDUCACION TECNOLOGICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_5Grado,asignatura == "ARTES VISUALES"),"Documento","Promedio"),by="Documento"))
df <- rename(df,Prom_Nat=Promedio.x, Prom_Soc=Promedio.y, Prom_Mat=Promedio.x.x, Prom_Len=Promedio.y.y, Prom_EdFisica=Promedio.x.x.x, Prom_Mus=Promedio.y.y.y, Prom_Tecno=Promedio.x.x.x.x, Prom_Art=Promedio.y.y.y.y)

# Gráfico de la matriz de correlación
matriz_correlacion <- cor(df[, -1], method="spearman")
corrplot(matriz_correlacion, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Matriz de correlación", cex.main = 1.5)

# Gráfico de correlación parcial
variables <- c("Prom_Nat", "Prom_Soc", "Prom_Mat", "Prom_Len")
dataM <- df[, variables]
correlacion_parcial <- pcor(dataM, method = "pearson")$estimate
corrplot(correlacion_parcial, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Correlación parcial", cex.main = 1.5)

In [ ]:
# Test de una media
# Filtramos los datos
# En este caso, a los promedios de quinto grado en las materias de Lengua y Matematica
quinto_mat <- filter(Notas_Victoria_5Grado, asignatura == "MATEMATICA")$Promedio
quinto_len <- filter(Notas_Victoria_5Grado, asignatura == "LENGUA")$Promedio

# Ya sabemos que no son datos con distribuciones normales
# Entonces...
# Al querer comparar las media de los promedios conseguidos por los alumnos de quinto grado en Matematica y
# Lengua se deberían usar test parametricos, pero con el supuesto de que los datos tengan una distribucion 
#normal. Al no cumplir con esta condición, utilizaremos un test no parametrico como lo es el test de Wilcox
#pero  en vez de comparar las medias, compararemos las medianas.

# H0 -> la mediana de los alumnos de quinto grado en Matematica es 8
# H1 -> la mediana de los alumnos de quinto grado en Matematica es mayor a 8
print("Test de Wilcox para los promedios de Matemática: ")
wilcox.test(quinto_mat, mu=8, alternative = "greater", conf.level = 0.95)

# H0 -> la mediana de los alumnos de quinto grado en Lengua es 8
# H1 -> la mediana de los alumnos de quinto grado en Lengua es mayor a 8
print("Test de Wilcox para los promedios de Lengua")
wilcox.test(quinto_len, mu=8, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de Matemática: ")
median(quinto_mat)
print("Mediana de los promedios de Lengua: ")
median(quinto_len)

In [ ]:
# Test de dos medias independientes
# Queremos ver si los promedios de los alumnos de quinto grado que asisten a una escuela privada son mejores que los de la escuela publica
datos_priv <- filter(Notas_Victoria_5Grado, Gestión == "Privada" & asignatura =="LENGUA")$Promedio
datos_pub <- filter(Notas_Victoria_5Grado, Gestión == "Pública" & asignatura =="LENGUA")$Promedio

# Distribuciones normales?
print("Test de shapiro para los promedios de escuelas privadas: ")
shapiro.test(datos_priv)
print("Test de shapiro para los promedios de escuelas públicas: ")
shapiro.test(datos_pub)

# Los promedios en los dos grupos de datos no siguen una distribucion normal

#H0: Los promedios de los alumnos de quinto grado en Lengua son iguales en escuelas privadas y públicas.
#H1: Los promedios de los alumnos de quinto grado en Lengua en escuelas privadas son mayores que en escuelas públicas.
wilcox.test(datos_priv,datos_pub, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de escuelas privadas: ")
median(datos_priv)
print("Mediana de los promedios de escuelas públicas: ")
median(datos_pub)

In [ ]:
# Test de más de dos medias independientes
datos_mañana <- filter(Notas_Victoria_5Grado, turno == "MAÑANA" & asignatura == "MATEMATICA")
datos_tarde <- filter(Notas_Victoria_5Grado, turno == "TARDE" & asignatura == "MATEMATICA")
datos_completo <- filter(Notas_Victoria_5Grado, turno == "COMPLETO" & asignatura == "MATEMATICA")

# Combinamos los datos para graficarlos
datos_comb <- bind_rows(
  datos_mañana %>% mutate(turno = "MAÑANA"),
  datos_tarde %>% mutate(turno = "TARDE"),
  datos_completo %>% mutate(turno = "COMPLETO"))

# Boxplots comparativos interactivos
plot <- plot_ly(datos_comb, x = ~turno, y = ~Promedio, color = ~turno, type = "box") %>%
  layout(
    title = "Comparación de Promedios de Matemáticas en quinto por Turno",
    xaxis = list(title = "Turno"),
    yaxis = list(title = "Promedio"),
    showlegend = FALSE)
plot

In [ ]:
# Pasamos los datos a tipo numerico
quinto = as.numeric(c(datos_mañana$Promedio, datos_tarde$Promedio, datos_completo$Promedio))

# Para que se repita la cantidad de alumno en cada turno su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana$Promedio)), rep("TARDE",length(datos_tarde$Promedio)), rep("COMPLETO", length(datos_completo$Promedio))))    

# Armamos el dataframe
datos_quinto <- data.frame(turno, quinto)

# Agrupamos los datos según el turno, calculamos la media de los promedios y contabilizamos las frecuencias
datos_quinto %>% group_by(turno) %>%
    dplyr::summarise(Media = round(mean(quinto),1), datos = n())

# Test de ANOVA
#H0 ---> Los tres turnos de QUINTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
#Nivel de significancia = 0.05
fm_quinto = aov(quinto ~ turno)
summary(fm_quinto)

# Potencia del test (CAMBIAR)
potencia_test <- pwr.anova.test(k = 3, n = c(76, 233, 201), f = 0.25, sig.level = 0.05)$power
potencia_test 

# Test HSD de Tukey
TukeyHSD(fm_quinto, "turno", conf.level = .95)

# Pero para saber que todos estos resultados son contundentes, la distribución de los datos tiene que ser
#normal, por eso se calcularan los test de shapiro
shapiro.test(datos_quinto$quinto[datos_quinto$turno == "MAÑANA"])
shapiro.test(datos_quinto$quinto[datos_quinto$turno == "TARDE"])
shapiro.test(datos_quinto$quinto[datos_quinto$turno == "COMPLETO"])

# Como los datos no siguen una distribucion normal, se descarta todo lo calculado. Se procede a calcularlo
#con un test no parametrico

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

# Test no parametrico para quinto grado
#H0 ---> Los tres turnos de quinto grado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_quinto$quinto, datos_quinto$turno)
pairwise.wilcox.test(datos_quinto$quinto, datos_quinto$turno)

In [ ]:
# Test de más de dos medias independientes
datos_mañana <- filter(Notas_Victoria_5Grado, turno == "MAÑANA" & asignatura == "LENGUA")
datos_tarde <- filter(Notas_Victoria_5Grado, turno == "TARDE" & asignatura == "LENGUA")
datos_completo <- filter(Notas_Victoria_5Grado, turno == "COMPLETO" & asignatura == "LENGUA")

# Combinamos los datos para graficarlos
datos_comb <- bind_rows(
  datos_mañana %>% mutate(turno = "MAÑANA"),
  datos_tarde %>% mutate(turno = "TARDE"),
  datos_completo %>% mutate(turno = "COMPLETO"))

# Boxplots comparativos interactivos
plot <- plot_ly(datos_comb, x = ~turno, y = ~Promedio, color = ~turno, type = "box") %>%
  layout(
    title = "Comparación de Promedios de Lengua en quinto por Turno",
    xaxis = list(title = "Turno"),
    yaxis = list(title = "Promedio"),
    showlegend = FALSE)
plot

In [ ]:
# Pasamos los datos a tipo numerico
quinto = as.numeric(c(datos_mañana$Promedio, datos_tarde$Promedio, datos_completo$Promedio))

# Para que se repita la cantidad de alumno en cada turno su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana$Promedio)), rep("TARDE",length(datos_tarde$Promedio)), rep("COMPLETO", length(datos_completo$Promedio))))    

# Armamos el dataframe
datos_quinto <- data.frame(turno, quinto)

# Agrupamos los datos según el turno, calculamos la media de los promedios y contabilizamos las frecuencias
datos_quinto %>% group_by(turno) %>%
    dplyr::summarise(Media = round(mean(quinto),1), datos = n())

# Test de ANOVA
#H0 ---> Los tres turnos de QUINTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
#Nivel de significancia = 0.05
fm_quinto = aov(quinto ~ turno)
summary(fm_quinto)

# Potencia del test (CAMBIAR)
potencia_test <- pwr.anova.test(k = 3, n = c(76, 250, 200), f = 0.25, sig.level = 0.05)$power
potencia_test 

# Test HSD de Tukey
TukeyHSD(fm_quinto, "turno", conf.level = .95)

# Pero para saber que todos estos resultados son contundentes, la distribución de los datos tiene que ser
#normal, por eso se calcularan los test de shapiro
shapiro.test(datos_quinto$quinto[datos_quinto$turno == "MAÑANA"])
shapiro.test(datos_quinto$quinto[datos_quinto$turno == "TARDE"])
shapiro.test(datos_quinto$quinto[datos_quinto$turno == "COMPLETO"])

# Como los datos no siguen una distribucion normal, se descarta todo lo calculado. Se procede a calcularlo
#con un test no parametrico

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

# Test no parametrico para quinto grado
#H0 ---> Los tres turnos de quinto grado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_quinto$quinto, datos_quinto$turno)
pairwise.wilcox.test(datos_quinto$quinto, datos_quinto$turno)

In [ ]:
# Test para mas de dos medias pareadas
# Filtramos los datos 
datos_primer_tri <- filter(Notas_Victoria_5Grado, asignatura == "MATEMATICA")$NotasPrimerT
datos_segundo_tri <- filter(Notas_Victoria_5Grado, asignatura == "MATEMATICA")$NotasSegundoT
datos_tercer_tri <- filter(Notas_Victoria_5Grado, asignatura == "MATEMATICA")$NotasTercerT

# Unimos los datos filtrados a un data frame
datos_completos <- data.frame(
  Trimestre = rep(c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"), each = length(datos_primer_tri)),
  Notas = c(datos_primer_tri, datos_segundo_tri, datos_tercer_tri))

# Convertimos la variable Trimestre a factor
datos_completos$Trimestre <- factor(datos_completos$Trimestre, levels = c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"))

# Creamos un gráfico interactivo de las notas en cada trimestre
plot <- plot_ly() %>%
  add_boxplot(data = datos_completos, x = ~Trimestre, y = ~Notas, color = ~Trimestre, colors = c("blue", "red", "green"), name = "Matemáticas", offsetgroup = 1) %>%
 layout(
    title = "Comparación de notas por trimestre de Matemáticas en Quinto",
    xaxis = list(title = "Trimestre"),
    yaxis = list(title = "Notas"),
    showlegend = FALSE)
plot

In [ ]:
# Calculamos el tamaño de los alumnos de tercer grado en Matemática para utilizar posteriormente
tamanio = length(datos_primer_tri)

# Armamos el data frame con lo datos
notas <- data.frame(
 sujeto = 1:tamanio,
 datos_primer_tri <- filter(Notas_Victoria_5Grado, asignatura == "MATEMATICA")$NotasPrimerT,
 datos_segundo_tri <- filter(Notas_Victoria_5Grado, asignatura == "MATEMATICA")$NotasSegundoT,
 datos_tercer_tri <- filter(Notas_Victoria_5Grado, asignatura == "MATEMATICA")$NotasTercerT)

# Esto sirve para armar los datos de manera larga
datos_largo <- melt(notas, id.vars= "sujeto", variable.name= "Notas", value.name= "Puntuacion")

# Realizar el ANOVA de medias pareadas
modelo_anova <- aov(Puntuacion ~ Notas + Error(sujeto/Notas), data = datos_largo)

# Cambiar el tipo de modelo a lm
modelo_lm <- lm(modelo_anova)

# Calcular el ANOVA con el modelo lm
anova_resultado <- Anova(modelo_lm, type = "III")
print(anova_resultado)

# Calculamos normalidad de los residuos y constatamos que las varianzas sean constantes
print("Test de Normalidad:")
shapiro.test(modelo_lm$residuals)
print("Test de homocedasticidad: ")
bartlett.test(modelo_lm$residuals ~ datos_largo$Notas)

In [ ]:
# Armamos el data frame con lo datos
notas= as.numeric(c(datos_primer_tri,datos_segundo_tri,datos_tercer_tri))
trimestres <- as.factor(c(rep("Primer Trimestre",length(datos_primer_tri)), rep("Segundo Trimestre",length(datos_segundo_tri)),rep("Tercer Trimestre",length(datos_tercer_tri))))
datos <- data.frame(Alumnos=rep(1:tamanio,3), notas, trimestres)
head(datos)

# Calculamos medias de cada grupo
by(data = datos$notas, INDICES = datos$trimestres, FUN = median)

# Test para encontrar diferencias significativas entre los grupos
friedman.test(notas ~ trimestres | Alumnos, data=datos)

# Potencia del test
friedman_anova <- pwr.anova.test(k = 3, n = tamanio, f = 0.25, sig.level = 0.05)$power
print("Potencia del test:")
friedman_anova

# Para saber entre que conjuntos de datos estan las diferencias
pairwise.wilcox.test(datos$notas, datos$trimestres, paired = TRUE)

# Grafico de coordenadas paralelas de los primeros 10 estudiantes (no se hacen todos porque no se entenderia el grafico con tantas lineas)
# Primero modificamos el nombre de los datos para que a la hora de ponerlos en el grafico, se vea mas conciso cada clase
datos_largo$Notas <- gsub("datos_primer_tri.*", "Primer Trimestre", datos_largo$Notas)
datos_largo$Notas <- gsub("datos_segundo_tri.*", "Segundo Trimestre", datos_largo$Notas)
datos_largo$Notas <- gsub("datos_tercer_tri.*", "Tercer Trimestre", datos_largo$Notas)

# Filtramos los primeros 10 alumnos
datos_grafica <- datos_largo %>%
  filter(sujeto <= 10)

# Grafico de coordenadas paralelas de los primeros 10 alumnos con sus notas en cada trimestre
ggplot(datos_grafica, aes(x = Notas, y = Puntuacion, group = sujeto, color = factor(sujeto))) +
  geom_line() +
  geom_point() +
  labs(title = "Evolución de algunas de las notas trimestrales a lo largo del año 2023
    de los alumnos de Tercer Grado en Matemática",
       x = "Trimestre",
       y = "Nota") +
  theme_minimal()

In [ ]:
# Armamos una tabla con las materias troncales y el desempeño logrado por los estudiantes
aux <- filter(Notas_Victoria_5Grado, (asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" ))
materias_desempeño <- aux %>%
    select('asignatura', 'Desempeño') %>%
    table()
materias_desempeño

# Reordenamos la tabla
materias_desempeño1 <- cbind(materias_desempeño[,5], materias_desempeño[,1], materias_desempeño[,2], materias_desempeño[,4], materias_desempeño[,3])
colnames(materias_desempeño1) <- c("Regular", "Aprobado", "Bueno","Muy Bueno","Distinguido")
rownames(materias_desempeño1) <- c("ARTES VISUALES", "CS. NATURALES", "CS. SOCIALES","ED. FISICA", "ED. MUSICAL", "ED. TECNOLOGICA", "LENGUA", "MATEMATICA")
materias_desempeño1

print("Tabla con marginales fila y columna")
materias_desempeño2 <- cbind(materias_desempeño1, "Total"= margin.table(materias_desempeño1,1))
materias_desempeño2 <- rbind(materias_desempeño2, "Total"=margin.table(materias_desempeño2,2))
materias_desempeño2

print("Tabla contingencia ~ Porcentaje Total")
tabla_cont_total <- prop.table(materias_desempeño1)*100
tabla_cont_total <- cbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,1))
tabla_cont_total <- rbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,2))
round(tabla_cont_total,1)

print("Tabla contingencia ~ Porcentaje Fila")
tabla_cont_filas <- prop.table(rbind(materias_desempeño1, "Total"=margin.table(materias_desempeño1, 2)),1)*100
tabla_cont_filas <- cbind(tabla_cont_filas, "Total"=margin.table(tabla_cont_filas,1))
round(tabla_cont_filas,1)

print("Tabla contingencia ~ Porcentaje Columna")
tabla_cont_col <- prop.table(cbind(materias_desempeño1, "Total"=margin.table(materias_desempeño1, 1)),2)*100
tabla_cont_col <- rbind(tabla_cont_col, "Total"=margin.table(tabla_cont_col,2))
round(tabla_cont_col,1)


In [ ]:
# Organizamos los datos para hacer un grafico de barras adosadas
tabla_variable_fila <- round(prop.table(table(aux$asignatura, aux$Desempeño), 1)*100,1)
tabla_variable_fila <- t(tabla_variable_fila)
grafico <- barplot(tabla_variable_fila,
                  main= " Grafico de Contigencia por fila",
                  beside= TRUE,
                  space= c(0.1, 1.5),
                  ylim= c(0, 50),
                  cex.axis= 1.2,
                  cex.names= 0.1,
                  cex.main= 1.5,
                  col = c("green","blue", "purple","orange","red"),
                  xlab = "Artes   Cs.Nat   Cs.Soc   Ed.Fis   Ed.Mus   Ed.Tecno  Leng  Mat",
                  cex.lab = 1.2,
                  ylab= "Porcentaje",
                  col.lab= "black")
text(grafico, tabla_variable_fila, labels = tabla_variable_fila, pos = 3, cex = 0.7, col = "black")
legend("topright", rownames(tabla_variable_fila),
       title = "Desempeño",
       title.adj = 0.4,     
       title.col = "black",      
       lty = c(1),
       col = c("green","blue", "purple","orange","red"),
       lwd = 4,
       cex = 1)

# Organizamos los datos para hacer un grafico de barras apiladas
tabla_variable_col <- round(prop.table(table(aux$asignatura, aux$Desempeño), 2) * 100, 1)
grafico <- barplot(tabla_variable_col,
                   main = "Gráfico de Contingencia por columna",
                   beside = FALSE,
                   space = rep(0.25, ncol(tabla_variable_col)),
                   ylim = c(0, 110),
                   cex.axis = 1.2, 
                   cex.names = 1, 
                   cex.main = 1.5, 
                   col = c("green", "red", "blue", "orange", "purple", "cyan", "yellow", "magenta"),
                   xlab = "Desempeños",
                   cex.lab = 1.5,
                   ylab = "Porcentaje",
                   col.lab = "black")
legend(x = 0, y = 115, rownames(tabla_variable_col),cex = 0.5, fill=c("green", "red", "blue", "orange", "purple", "cyan", "yellow", "magenta"),xpd = TRUE,horiz = FALSE,bty = "n")

In [ ]:
# Como muestran los graficos y las tablas, segun el tipo de materia "causa" el desempeño final
#de cada alumno
# Para saber si realmente existe una relación entre las variables aplicaremos el test de chi cuadrado
# H0 ---> No hay relación entre las variables
# H1 ---> Si hay relación entre las variables
chi <-chisq.test(aux$asignatura, aux$Desempeño)
chi

# Vemos la fuerza de asociación
CramerV(aux$asignatura, aux$Desempeño)

In [ ]:
# Frecuencias observadas
chi$observed

# Frecuencias esperadas si las variables fueran independientes
round(chi$expected,0)

# Residuos estandarizados
round(chi$residuals,1)

# Residuos ajustados
round(chi$stdres,1)

In [ ]:
# Análisis Multivariado
# Vector de medias de las variables de primer grado agrupadas por las asignaturas
vector_medias <- Notas_Victoria_5Grado %>% group_by(asignatura) %>% summarise_all(mean)
# Nos quedamos con las materias que nos interesan
vector_medias <- vector_medias %>% filter(asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" | asignatura== "CATEQUESIS" | asignatura == "INFORMATICA" |asignatura =="INGLES" )
vector_medias

In [ ]:
# Buscamos valores outliers
aux <- mahalanobis_distance(data= Notas_Victoria_5Grado[,c("NotasPrimerT", "NotasSegundoT", "NotasTercerT", "Promedio")])$is.outlier    
ftable(aux)

In [ ]:
# Grafico de perfiles paralelos para ver el rendimiento de los estudiantes en 11 materias
# Definimos colores manuales para que se distingan
colores <- c("ARTES VISUALES" = "purple","CATEQUESIS" = "orange",
            "CIENCIAS NATURALES" = "green","CIENCIAS SOCIALES" = "blue",
            "EDUCACION FISICA" = "red","EDUCACION MUSICAL" = "turquoise",
            "EDUCACION TECNOLOGICA" = "black","INFORMATICA" = "cyan",
            "INGLES" = "yellow","LENGUA" = "magenta",
            "MATEMATICA" = "skyblue")

# Grafico
ggparcoord(data = vector_medias,
           columns=c(11:14),
           groupColumn = "asignatura",
           showPoints = TRUE,
           scale = "globalminmax") +
  scale_color_manual(values = colores) +
  theme(axis.text.x = element_text(angle = 30, hjust = 1))

In [ ]:
# Prueba de normalidad
mardia2(Notas_Victoria_5Grado[,11:14])
# Los datos no siguen una distribución normal multivariada

In [ ]:
# COMPARACION DE TRES O MÁS POBLACIONES MULTIVARIADAS INDEPENDIENTES
mod5<-manova(cbind(NotasPrimerT, NotasSegundoT, NotasTercerT, Promedio)~turno,data=Notas_Victoria_5Grado)
summary(mod5)

# El MANOVA nos indica que hay diferencias entre los vectores medios

#Supuestos de normalidad
Notas_Victoria_5Grado %>% group_by(turno) %>%  shapiro_test(NotasPrimerT, NotasSegundoT, NotasTercerT, Promedio)
# Ningun grupo de datos sigue una distribucion normal multivariada

In [ ]:
# Decidimos para que los alumnos esten en condiciones de estar en la bandera, tienen que tener las 8 materias troncales de primaria
#como venimos trabajando anteriormente, ademas de que tienen que tener las 8 si o si, si le falta una nota en estas materias, se lo descarta
abanderados <- distinct(filter(Notas_Victoria_5Grado, Condición == "Aprobado", Año_curso == "QUINTO", asignatura %in% c("MATEMATICA", "LENGUA", "CIENCIAS SOCIALES", "CIENCIAS NATURALES", "EDUCACION MUSICAL", "EDUCACION FISICA", "EDUCACION TECNOLOGICA", "ARTES VISUALES")))

# Filtramos los alumnos que tengan todas las materias troncales
abanderados <- abanderados %>%
  group_by(idalumno) %>%
  filter(n() == 8) %>%
  ungroup()

# Calculamos el promedio para cada alumno 
promedios_finales_quinto <- abanderados %>% group_by(idalumno) %>% dplyr::summarise(Promedio_Total = mean(Promedio, na.rm = TRUE)) 

# Unimos el promedio total de cada alumno con los datos originales
datos_bandera_entrantes <- inner_join(promedios_finales_quinto, Notas_Victoria_5Grado, by = "idalumno")

# Nos quedamos con el primer registro de cada alumno
datos_bandera_entrantes <- datos_bandera_entrantes %>%
  group_by(idalumno) %>%
  dplyr::summarise(across(everything(), first))

# Seleccionamos y rendodeamos algunas variables para mejorar el entendimiento del data frame
datos_bandera_entrantes <- dplyr::select(datos_bandera_entrantes, 'CodigoUnicoEscolar', 'idalumno', 'Documento', 'Gestión','turno', 'Promedio_Total.x')
datos_bandera_entrantes <- datos_bandera_entrantes %>%
  mutate(Promedio_Total = round(Promedio_Total.x, 2))

# Agrupamos segun el CodigoUnicoEscolar de cada escuela, ademas de que seleccionamos los 6 mejores promedios y los ordenamos
datos_bandera_entrantes <- datos_bandera_entrantes %>%
  group_by(CodigoUnicoEscolar) %>%
  top_n(6, Promedio_Total) %>%
  arrange(CodigoUnicoEscolar, desc(Promedio_Total))

# Agregamos la posición de la bandera basado en el ranking de Promedio_Total
datos_bandera_entrantes <- datos_bandera_entrantes %>%
  mutate(Posicion = row_number())
mejores_promedios_quinto <- datos_bandera_entrantes %>%
    mutate(Posicion = case_when(
    Posicion == 1 ~ "Abanderado Nacional",
    Posicion == 2 ~ "Abanderado Provincial",
    Posicion == 3 ~ "Escolta derecha nacional",
    Posicion == 4 ~ "Escolta derecha provincial",
    Posicion == 5 ~ "Escolta izquierda nacional",
    Posicion == 6 ~ "Escolta izquierda provincial",
    Posicion == 7 ~ "Primer suplente",
    Posicion == 8 ~ "Segundo suplente"))

# Mostramos los abanderados salientes para cada escuela de la base de datos
print("Los nuevos abanderados asumirán con orgullo y responsabilidad el honor de portar las banderas nacionales y provinciales durante el año 2024.")
mejores_promedios_quinto
#print(mejores_promedios_quinto, n = 59) 

In [ ]:
ggplot(mejores_promedios_quinto, aes(x = factor(CodigoUnicoEscolar), y = Promedio_Total, group = Posicion, color = Posicion)) +
  geom_line() +
  geom_point() +
  facet_wrap(~CodigoUnicoEscolar, scales = "free_x") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(title = "Comparación de Promedios por Posición en las Banderas",
       x = "Código Único Escolar",
       y = "Promedio Total") +
  theme_minimal() +
  theme(legend.position = "bottom", 
        legend.text = element_text(size = 9),  
        legend.title = element_text(size = 8)) + 
  scale_color_brewer(palette = "Set1")

## Análisis de Sexto grado

In [ ]:
# Calculamos con nuestro modelo de predicción lineal las edades de los alumnos de sexto grado
# Aplicamos la función a los estudiantes 
Notas_Victoria_6Grado <- Notas_Victoria_6Grado %>%
  mutate(EdadEstimada = estimar_edad(Documento))

# Ponemos este filtro para aquellos DNI que pertenecen a personas nacionalizado o estan mal cargados, ya que
#el modelo los etiquetaría con una edad muy erronea
Notas_Victoria_6Grado <- Notas_Victoria_6Grado %>%
  mutate(EdadEstimada = ifelse(EdadEstimada < 0 | EdadEstimada > 20, NA, EdadEstimada))

# Reordenamos las variables
Notas_Victoria_6Grado <- dplyr::select(Notas_Victoria_6Grado,'CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno', 'Año_curso', 'Documento', 'asignatura', 'idalumno', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT', 'NotasSegundoT', 'NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', 'EdadEstimada', "total_asig", "porc_aprob", "porc_desaprob")

# Vemos los resultados
Notas_Victoria_6Grado

In [ ]:
# Calculamos el promedio total de cada alumno
aux <- Notas_Victoria_6Grado %>% group_by(idalumno) %>% dplyr::summarise(Promedio_Total = round(mean(Promedio, na.rm = TRUE),2)) 

# Lo unimos con el dataframe original
Notas_Victoria_6Grado <- inner_join(aux, Notas_Victoria_6Grado, by = "idalumno")

# Asigna la materia con mejor promedio para cada estudiante
Notas_Victoria_6Grado$Mejor_asig <- Notas_Victoria_6Grado %>% group_by(idalumno) %>% mutate(Mejor_asig = asignatura[which.max(Promedio)]) %>% ungroup() %>% .$Mejor_asig

# Asigna la materia con peor promedio para cada estudiante
Notas_Victoria_6Grado$Peor_asig <- Notas_Victoria_6Grado %>% group_by(idalumno) %>% mutate(Peor_asig = asignatura[which.min(Promedio)]) %>% ungroup() %>% .$Peor_asig

In [ ]:
# Reordenamos las variables para que queden bien organizadas
Notas_Victoria_6Grado <- Notas_Victoria_6Grado %>% 
  dplyr::select('CodigoUnicoEscolar', 'departamento', 'Gestión', 'turno', 'Año_curso', 'Documento', 'asignatura', 'idalumno', 'Modalidad', 'ModEnseñanza', 'NotasPrimerT', 'NotasSegundoT', 'NotasTercerT', 'Promedio', 'Condición', 'Desempeño', 'Tendencia', 'Eximido', 'Rango_edad', 'EdadEstimada', "total_asig", "porc_aprob", "porc_desaprob", 'Mejor_asig', 'Peor_asig', 'Promedio_Total')

In [ ]:
# Calculamos las medidas de tendencia central y de dispersión para las 4 materias troncales de primaria
print("Medidas estadisticas descriptivas de sexto grado")
Notas_Victoria_6Grado %>%
  filter(asignatura == "CIENCIAS NATURALES" | asignatura == "CIENCIAS SOCIALES" | asignatura == "MATEMATICA" | asignatura == "LENGUA") %>% 
  group_by(asignatura) %>%
  dplyr::summarise(Conteo = n(),
            Media = round(mean(Promedio, na.rm = TRUE), 2),
            Mediana = round(median(Promedio, na.rm = TRUE), 2),
            Moda = Mode(Promedio),            
            Varianza = round(var(Promedio, na.rm = TRUE), 2),
            Desvio = round(sd(Promedio, na.rm = TRUE), 2),
            Minimo = round(min(Promedio, na.rm = TRUE), 2),
            Maximo = round(max(Promedio, na.rm = TRUE), 2),
            Rango = Maximo - Minimo,
            IQR = round(IQR(Promedio, na.rm = TRUE), 2),
            Q1 = round(quantile(Promedio, 0.25, na.rm = TRUE), 2),
            Q3 = round(quantile(Promedio, 0.75, na.rm = TRUE), 2),
            Sup. = round((Q3 + 1.5 * IQR), 2),
            Inf. = round((Q1 - 1.5 * IQR), 2),
            CoefVar = round(sd(Promedio, na.rm = TRUE) / mean(Promedio, na.rm = TRUE) * 100, 2))

In [ ]:
# Gráfico de boxplot comparativo
datos_filtrados <- Notas_Victoria_6Grado %>%
  filter(asignatura %in% c("CIENCIAS NATURALES", "CIENCIAS SOCIALES", "MATEMATICA", "LENGUA"))

p <- ggplot(datos_filtrados, aes(x = asignatura, y = Promedio, fill = asignatura)) +
  geom_boxplot() +
  ggtitle("                      Comparación de Promedios en materias de sexto grado") +  
  labs(
    x = "Materias",
    y = "Promedios"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black"),
    legend.position = "none"
  )

# Mostramos el gráfico
p

In [ ]:
# Cuantiles para añadir información en los boxplots
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.75)
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.25)
quantile(filter(datos_filtrados, asignatura == "MATEMATICA")$Promedio, probs = 0.14)

quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.75)
quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.59)
quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.25)
quantile(filter(datos_filtrados, asignatura == "LENGUA")$Promedio, probs = 0.12)

quantile(filter(datos_filtrados, asignatura == "CIENCIAS SOCIALES")$Promedio, probs = 0.07)
quantile(filter(datos_filtrados, asignatura == "CIENCIAS SOCIALES")$Promedio, probs = 0.08)

quantile(filter(datos_filtrados, asignatura == "CIENCIAS NATURALES")$Promedio, probs = 0.05)
quantile(filter(datos_filtrados, asignatura == "CIENCIAS NATURALES")$Promedio, probs = 0.06)

In [ ]:
# Histograma de promedios en matemática
notas_mat_sexto <- filter(Notas_Victoria_6Grado,asignatura == "MATEMATICA")$Promedio
histograma_mat <- fdt(notas_mat_sexto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_mat,
     type= "fh", 
     col= "#FF9932",
     main= "Histograma de los promedios de sexto grado
     en la materia Matemática")

# Medidas de distribución
coef_sim = skewness(notas_mat_sexto, na.rm=TRUE)
kurtosis = kurtosis(notas_mat_sexto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_mat_sexto)

In [ ]:
# Histograma de promedios en lengua
notas_len_sexto <- filter(Notas_Victoria_6Grado,asignatura == "LENGUA")$Promedio
histograma_len <- fdt(notas_len_sexto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_len,
     type= "fh", 
     col= "#FF86FF",
     main= "Histograma de los promedios de sexto grado
     en la materia Lengua")

# Medidas de distribución
coef_sim = skewness(notas_len_sexto, na.rm=TRUE)
kurtosis = kurtosis(notas_len_sexto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_len_sexto)

In [ ]:
# Histograma de promedios en ciencias naturales
notas_nat_sexto <- filter(Notas_Victoria_6Grado,asignatura == "CIENCIAS NATURALES")$Promedio
histograma_nat <- fdt(notas_nat_sexto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_nat,
     type= "fh", 
     col= "#00BB00",
     main= "Histograma de los promedios de sexto grado
     en la materia Ciencias Naturales")

# Medidas de distribución
coef_sim = skewness(notas_nat_sexto, na.rm=TRUE)
kurtosis = kurtosis(notas_nat_sexto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_nat_sexto)

In [ ]:
# Histograma de promedios en ciencias sociales
notas_soc_sexto <- filter(Notas_Victoria_6Grado,asignatura == "CIENCIAS SOCIALES")$Promedio
histograma_soc <- fdt(notas_soc_sexto, breaks= "Sturges", na.rm= TRUE)
plot(histograma_soc,
     type= "fh", 
     col= "#3D87FF",
     main= "Histograma de los promedios de sexto grado
     en la materia Ciencias Sociales")

# Medidas de distribución
coef_sim = skewness(notas_soc_sexto, na.rm=TRUE)
kurtosis = kurtosis(notas_soc_sexto, na.rm=TRUE)
print(paste0("coeficiente de simetría: ", round(coef_sim, 2)))
print(paste0("kurtosis: ", round(kurtosis, 2)))

# Test de normalidad
hipotesis_nula <- "Se distribuyen de forma normal"
hipotesis_alternativa <- "No se distribuyen de forma normal"
shapiro.test(notas_soc_sexto)

In [ ]:
# Puntuación Z para saber que tipo de turno esta más alejado de un promedio de 8 en Matemática
# Filtramos los datos necesarios 
sexto_mañana <- filter(Notas_Victoria_6Grado, asignatura == "MATEMATICA" & turno == "MAÑANA")
sexto_tarde <- filter(Notas_Victoria_6Grado, asignatura == "MATEMATICA" & turno == "TARDE")
sexto_completo <- filter(Notas_Victoria_6Grado, asignatura == "MATEMATICA" & turno == "COMPLETO")

# Calculamos la media de los promedios
prom_sexto_mañana <- mean(sexto_mañana$Promedio, na.rm = TRUE)
prom_sexto_tarde <- mean(sexto_tarde$Promedio, na.rm = TRUE)
prom_sexto_completo <- mean(sexto_completo$Promedio, na.rm = TRUE)

# Calculamos los desvios de los promedios
desvio_sexto_mañana <- sd(sexto_mañana$Promedio, na.rm = TRUE)
desvio_sexto_tarde <- sd(sexto_tarde$Promedio, na.rm = TRUE)
desvio_sexto_completo <- sd(sexto_completo$Promedio, na.rm = TRUE)

# Establecemos el promedio a comparar
promedio_comparar <- 8

# Puntuación Z para el turno mañana
punt_z_mañana <- (promedio_comparar - prom_sexto_mañana) / desvio_sexto_mañana
print(paste(" Puntuación Z para los promedios de Matemática en sexto grado del turno mañana: ", round(punt_z_mañana, 2)))

# Puntuación Z para el turno tarde
punt_z_tarde <- (promedio_comparar - prom_sexto_tarde) / desvio_sexto_tarde
print(paste(" Puntuación Z para los promedios de Matemática en sexto grado del turno tarde: ", round(punt_z_tarde, 2)))

# Puntuación Z para el turno completo
punt_z_completo <- (promedio_comparar - prom_sexto_completo) / desvio_sexto_completo
print(paste(" Puntuación Z para los promedios de Matemática en sexto grado del turno completo: ", round(punt_z_completo, 2)))

In [ ]:
# Puntuación Z en estudiantes de escuelas de gestión pública y gestión privada
# Filtramos los datos necesarios 
sexto_publica <- filter(Notas_Victoria_6Grado, asignatura == "MATEMATICA" & Gestión == "Pública")
sexto_privada <- filter(Notas_Victoria_6Grado, asignatura == "MATEMATICA" & Gestión == "Privada")

# Calculamos la media de los promedios
prom_sexto_publica <- mean(sexto_publica$Promedio, na.rm = TRUE)
prom_sexto_privada <- mean(sexto_privada$Promedio, na.rm = TRUE)

# Calculamos los desvios de los promedios
desvio_sexto_publica <- sd(sexto_publica$Promedio, na.rm = TRUE)
desvio_sexto_privada <- sd(sexto_privada$Promedio, na.rm = TRUE)

# Establecemos el promedio a comparar
promedio_comparar <- 8

# Puntuación Z para escuelas publicas
punt_z_publica <- (promedio_comparar - prom_sexto_publica) / desvio_sexto_publica
print(paste(" Puntuación Z para los promedios de Matemática en sexto grado de escuelas públicas: ", round(punt_z_publica, 2)))

# Puntuación Z para escuelas privadas
punt_z_privada <- (promedio_comparar - prom_sexto_privada) / desvio_sexto_privada
print(paste(" Puntuación Z para los promedios de Matemática en sexto grado de escuelas privadas: ", round(punt_z_privada, 2)))

In [ ]:
# Unimos en un data frame las materias seleccionadas para calcular correlaciones
df <- suppressWarnings(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "CIENCIAS NATURALES"),"Documento","Promedio") %>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "CIENCIAS SOCIALES"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "MATEMATICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "LENGUA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "EDUCACION FISICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "EDUCACION MUSICAL"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "EDUCACION TECNOLOGICA"),"Documento","Promedio"),by="Documento")%>% inner_join(dplyr::select(filter(Notas_Victoria_6Grado,asignatura == "ARTES VISUALES"),"Documento","Promedio"),by="Documento"))
df <- rename(df,Prom_Nat=Promedio.x, Prom_Soc=Promedio.y, Prom_Mat=Promedio.x.x, Prom_Len=Promedio.y.y, Prom_EdFisica=Promedio.x.x.x, Prom_Mus=Promedio.y.y.y, Prom_Tecno=Promedio.x.x.x.x, Prom_Art=Promedio.y.y.y.y)

# Gráfico de la matriz de correlación
matriz_correlacion <- cor(df[, -1], method="spearman")
corrplot(matriz_correlacion, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Matriz de correlación", cex.main = 1.5)

# Gráfico de correlación parcial
variables <- c("Prom_Nat", "Prom_Soc", "Prom_Mat", "Prom_Len")
dataM <- df[, variables]
correlacion_parcial <- pcor(dataM, method = "pearson")$estimate
corrplot(correlacion_parcial, method = "color", type = "upper", 
         addCoef.col = "black", tl.col = "black", tl.srt = 45)
title("Correlación parcial", cex.main = 1.5)

In [ ]:
# Test de una media
# Filtramos los datos
# En este caso, a los promedios de sexto grado en las materias de Lengua y Matematica
sexto_mat <- filter(Notas_Victoria_6Grado, asignatura == "MATEMATICA")$Promedio
sexto_len <- filter(Notas_Victoria_6Grado, asignatura == "LENGUA")$Promedio

# Ya sabemos que no son datos con distribuciones normales
# Entonces...
# Al querer comparar las media de los promedios conseguidos por los alumnos de sexto grado en Matematica y
# Lengua se deberían usar test parametricos, pero con el supuesto de que los datos tengan una distribucion 
#normal. Al no cumplir con esta condición, utilizaremos un test no parametrico como lo es el test de Wilcox
#pero  en vez de comparar las medias, compararemos las medianas.

# H0 -> la mediana de los alumnos de sexto grado en Matematica es 8
# H1 -> la mediana de los alumnos de sexto grado en Matematica es mayor a 8
print("Test de Wilcox para los promedios de Matemática: ")
wilcox.test(sexto_mat, mu=8, alternative = "greater", conf.level = 0.95)

# H0 -> la mediana de los alumnos de sexto grado en Lengua es 8
# H1 -> la mediana de los alumnos de sexto grado en Lengua es mayor a 8
print("Test de Wilcox para los promedios de Lengua")
wilcox.test(sexto_len, mu=8, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de Matemática: ")
median(sexto_mat)
print("Mediana de los promedios de Lengua: ")
median(sexto_len)

In [ ]:
# Test de dos medias independientes
# Queremos ver si los promedios de los alumnos de sexto grado que asisten a una escuela privada son mejores que los de la escuela publica
datos_priv <- filter(Notas_Victoria_6Grado, Gestión == "Privada" & asignatura =="LENGUA")$Promedio
datos_pub <- filter(Notas_Victoria_6Grado, Gestión == "Pública" & asignatura =="LENGUA")$Promedio

# Distribuciones normales?
print("Test de shapiro para los promedios de escuelas privadas: ")
shapiro.test(datos_priv)
print("Test de shapiro para los promedios de escuelas públicas: ")
shapiro.test(datos_pub)

# Los promedios en los dos grupos de datos no siguen una distribucion normal

#H0: Los promedios de los alumnos de sexto grado en Lengua son iguales en escuelas privadas y públicas.
#H1: Los promedios de los alumnos de sexto grado en Lengua en escuelas privadas son mayores que en escuelas públicas.
wilcox.test(datos_priv,datos_pub, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de escuelas privadas: ")
median(datos_priv)
print("Mediana de los promedios de escuelas públicas: ")
median(datos_pub)

In [ ]:
# Test de más de dos medias independientes
datos_mañana <- filter(Notas_Victoria_6Grado, turno == "MAÑANA" & asignatura == "MATEMATICA")
datos_tarde <- filter(Notas_Victoria_6Grado, turno == "TARDE" & asignatura == "MATEMATICA")
datos_completo <- filter(Notas_Victoria_6Grado, turno == "COMPLETO" & asignatura == "MATEMATICA")

# Combinamos los datos para graficarlos
datos_comb <- bind_rows(
  datos_mañana %>% mutate(turno = "MAÑANA"),
  datos_tarde %>% mutate(turno = "TARDE"),
  datos_completo %>% mutate(turno = "COMPLETO"))

# Boxplots comparativos interactivos
plot <- plot_ly(datos_comb, x = ~turno, y = ~Promedio, color = ~turno, type = "box") %>%
  layout(
    title = "Comparación de Promedios de Matemáticas en sexto por Turno",
    xaxis = list(title = "Turno"),
    yaxis = list(title = "Promedio"),
    showlegend = FALSE)
plot

In [ ]:
# Pasamos los datos a tipo numerico
sexto = as.numeric(c(datos_mañana$Promedio, datos_tarde$Promedio, datos_completo$Promedio))

# Para que se repita la cantidad de alumno en cada turno su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana$Promedio)), rep("TARDE",length(datos_tarde$Promedio)), rep("COMPLETO", length(datos_completo$Promedio))))    

# Armamos el dataframe
datos_sexto <- data.frame(turno, sexto)

# Agrupamos los datos según el turno, calculamos la media de los promedios y contabilizamos las frecuencias
datos_sexto %>% group_by(turno) %>%
    dplyr::summarise(Media = round(mean(sexto),1), datos = n())

# Test de ANOVA
#H0 ---> Los tres turnos de SEXTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
#Nivel de significancia = 0.05
fm_sexto = aov(sexto ~ turno)
summary(fm_sexto)

# Potencia del test
potencia_test <- pwr.anova.test(k = 3, n = c(71, 241, 200), f = 0.25, sig.level = 0.05)$power
potencia_test 

# Test HSD de Tukey
TukeyHSD(fm_sexto, "turno", conf.level = .95)

# Pero para saber que todos estos resultados son contundentes, la distribución de los datos tiene que ser
#normal, por eso se calcularan los test de shapiro
shapiro.test(datos_sexto$sexto[datos_sexto$turno == "MAÑANA"])
shapiro.test(datos_sexto$sexto[datos_sexto$turno == "TARDE"])
shapiro.test(datos_sexto$sexto[datos_sexto$turno == "COMPLETO"])

# Como los datos no siguen una distribucion normal, se descarta todo lo calculado. Se procede a calcularlo
#con un test no parametrico

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

# Test no parametrico para sexto grado
#H0 ---> Los tres turnos de sexto grado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_sexto$sexto, datos_sexto$turno)
pairwise.wilcox.test(datos_sexto$sexto, datos_sexto$turno)

In [ ]:
# Test de más de dos medias independientes
datos_mañana <- filter(Notas_Victoria_6Grado, turno == "MAÑANA" & asignatura == "LENGUA")
datos_tarde <- filter(Notas_Victoria_6Grado, turno == "TARDE" & asignatura == "LENGUA")
datos_completo <- filter(Notas_Victoria_6Grado, turno == "COMPLETO" & asignatura == "LENGUA")

# Combinamos los datos para graficarlos
datos_comb <- bind_rows(
  datos_mañana %>% mutate(turno = "MAÑANA"),
  datos_tarde %>% mutate(turno = "TARDE"),
  datos_completo %>% mutate(turno = "COMPLETO"))

# Boxplots comparativos interactivos
plot <- plot_ly(datos_comb, x = ~turno, y = ~Promedio, color = ~turno, type = "box") %>%
  layout(
    title = "Comparación de Promedios de Lengua en sexto por Turno",
    xaxis = list(title = "Turno"),
    yaxis = list(title = "Promedio"),
    showlegend = FALSE)
plot

In [ ]:
# Pasamos los datos a tipo numerico
sexto = as.numeric(c(datos_mañana$Promedio, datos_tarde$Promedio, datos_completo$Promedio))

# Para que se repita la cantidad de alumno en cada turno su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana$Promedio)), rep("TARDE",length(datos_tarde$Promedio)), rep("COMPLETO", length(datos_completo$Promedio))))    

# Armamos el dataframe
datos_sexto <- data.frame(turno, sexto)

# Agrupamos los datos según el turno, calculamos la media de los promedios y contabilizamos las frecuencias
datos_sexto %>% group_by(turno) %>%
    dplyr::summarise(Media = round(mean(sexto),1), datos = n())

# Test de ANOVA
#H0 ---> Los tres turnos de SEXTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
#Nivel de significancia = 0.05
fm_sexto = aov(sexto ~ turno)
summary(fm_sexto)

# Potencia del test
potencia_test <- pwr.anova.test(k = 3, n = c(71, 250, 202), f = 0.25, sig.level = 0.05)$power
potencia_test 

# Test HSD de Tukey
TukeyHSD(fm_sexto, "turno", conf.level = .95)

# Pero para saber que todos estos resultados son contundentes, la distribución de los datos tiene que ser
#normal, por eso se calcularan los test de shapiro
shapiro.test(datos_sexto$sexto[datos_sexto$turno == "MAÑANA"])
shapiro.test(datos_sexto$sexto[datos_sexto$turno == "TARDE"])
shapiro.test(datos_sexto$sexto[datos_sexto$turno == "COMPLETO"])

# Como los datos no siguen una distribucion normal, se descarta todo lo calculado. Se procede a calcularlo
#con un test no parametrico

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

# Test no parametrico para sexto grado
#H0 ---> Los tres turnos de sexto grado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_sexto$sexto, datos_sexto$turno)
pairwise.wilcox.test(datos_sexto$sexto, datos_sexto$turno)

In [ ]:
# Test para mas de dos medias pareadas
# Filtramos los datos 
datos_primer_tri <- filter(Notas_Victoria_6Grado, asignatura == "MATEMATICA")$NotasPrimerT
datos_segundo_tri <- filter(Notas_Victoria_6Grado, asignatura == "MATEMATICA")$NotasSegundoT
datos_tercer_tri <- filter(Notas_Victoria_6Grado, asignatura == "MATEMATICA")$NotasTercerT

# Unimos los datos filtrados a un data frame
datos_completos <- data.frame(
  Trimestre = rep(c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"), each = length(datos_primer_tri)),
  Notas = c(datos_primer_tri, datos_segundo_tri, datos_tercer_tri))

# Convertimos la variable Trimestre a factor
datos_completos$Trimestre <- factor(datos_completos$Trimestre, levels = c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"))

# Creamos un gráfico interactivo de las notas en cada trimestre
plot <- plot_ly() %>%
  add_boxplot(data = datos_completos, x = ~Trimestre, y = ~Notas, color = ~Trimestre, colors = c("blue", "red", "green"), name = "Matemáticas", offsetgroup = 1) %>%
 layout(
    title = "Comparación de notas por trimestre de Matemáticas en Sexto",
    xaxis = list(title = "Trimestre"),
    yaxis = list(title = "Notas"),
    showlegend = FALSE)
plot

In [ ]:
# Calculamos el tamaño de los alumnos de tercer grado en Matemática para utilizar posteriormente
tamanio = length(datos_primer_tri)

# Armamos el data frame con lo datos
notas <- data.frame(
 sujeto = 1:tamanio,
 datos_primer_tri <- filter(Notas_Victoria_6Grado, asignatura == "MATEMATICA")$NotasPrimerT,
 datos_segundo_tri <- filter(Notas_Victoria_6Grado, asignatura == "MATEMATICA")$NotasSegundoT,
 datos_tercer_tri <- filter(Notas_Victoria_6Grado, asignatura == "MATEMATICA")$NotasTercerT)

# Esto sirve para armar los datos de manera larga
datos_largo <- melt(notas, id.vars= "sujeto", variable.name= "Notas", value.name= "Puntuacion")

# Realizar el ANOVA de medias pareadas
modelo_anova <- aov(Puntuacion ~ Notas + Error(sujeto/Notas), data = datos_largo)

# Cambiar el tipo de modelo a lm
modelo_lm <- lm(modelo_anova)

# Calcular el ANOVA con el modelo lm
anova_resultado <- Anova(modelo_lm, type = "III")
print(anova_resultado)

# Calculamos normalidad de los residuos y constatamos que las varianzas sean constantes
print("Test de Normalidad:")
shapiro.test(modelo_lm$residuals)
print("Test de homocedasticidad: ")
bartlett.test(modelo_lm$residuals ~ datos_largo$Notas)

In [ ]:
# Armamos el data frame con lo datos
notas= as.numeric(c(datos_primer_tri,datos_segundo_tri,datos_tercer_tri))
trimestres <- as.factor(c(rep("Primer Trimestre",length(datos_primer_tri)), rep("Segundo Trimestre",length(datos_segundo_tri)),rep("Tercer Trimestre",length(datos_tercer_tri))))
datos <- data.frame(Alumnos=rep(1:tamanio,3), notas, trimestres)
head(datos)

# Calculamos medias de cada grupo
by(data = datos$notas, INDICES = datos$trimestres, FUN = median)

# Test para encontrar diferencias significativas entre los grupos
friedman.test(notas ~ trimestres | Alumnos, data=datos)

# Potencia del test
friedman_anova <- pwr.anova.test(k = 3, n = tamanio, f = 0.25, sig.level = 0.05)$power
print("Potencia del test:")
friedman_anova

# Para saber entre que conjuntos de datos estan las diferencias
pairwise.wilcox.test(datos$notas, datos$trimestres, paired = TRUE)

# Grafico de coordenadas paralelas de los primeros 10 estudiantes (no se hacen todos porque no se entenderia el grafico con tantas lineas)
# Primero modificamos el nombre de los datos para que a la hora de ponerlos en el grafico, se vea mas conciso cada clase
datos_largo$Notas <- gsub("datos_primer_tri.*", "Primer Trimestre", datos_largo$Notas)
datos_largo$Notas <- gsub("datos_segundo_tri.*", "Segundo Trimestre", datos_largo$Notas)
datos_largo$Notas <- gsub("datos_tercer_tri.*", "Tercer Trimestre", datos_largo$Notas)

# Filtramos los primeros 10 alumnos
datos_grafica <- datos_largo %>%
  filter(sujeto <= 10)

# Grafico de coordenadas paralelas de los primeros 10 alumnos con sus notas en cada trimestre
ggplot(datos_grafica, aes(x = Notas, y = Puntuacion, group = sujeto, color = factor(sujeto))) +
  geom_line() +
  geom_point() +
  labs(title = "Evolución de algunas de las notas trimestrales a lo largo del año 2023
    de los alumnos de Sexto Grado en Matemática",
       x = "Trimestre",
       y = "Nota") +
  theme_minimal()

In [ ]:
# Armamos una tabla con las materias troncales y el desempeño logrado por los estudiantes
aux <- filter(Notas_Victoria_6Grado, (asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" ))
materias_desempeño <- aux %>%
    select('asignatura', 'Desempeño') %>%
    table()
materias_desempeño

# Reordenamos la tabla
materias_desempeño1 <- cbind(materias_desempeño[,5], materias_desempeño[,1], materias_desempeño[,2], materias_desempeño[,4], materias_desempeño[,3])
colnames(materias_desempeño1) <- c("Regular", "Aprobado", "Bueno","Muy Bueno","Distinguido")
rownames(materias_desempeño1) <- c("ARTES VISUALES", "CS. NATURALES", "CS. SOCIALES","ED. FISICA", "ED. MUSICAL", "ED. TECNOLOGICA", "LENGUA", "MATEMATICA")
materias_desempeño1

print("Tabla con marginales fila y columna")
materias_desempeño2 <- cbind(materias_desempeño1, "Total"= margin.table(materias_desempeño1,1))
materias_desempeño2 <- rbind(materias_desempeño2, "Total"=margin.table(materias_desempeño2,2))
materias_desempeño2

print("Tabla contingencia ~ Porcentaje Total")
tabla_cont_total <- prop.table(materias_desempeño1)*100
tabla_cont_total <- cbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,1))
tabla_cont_total <- rbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,2))
round(tabla_cont_total,1)

print("Tabla contingencia ~ Porcentaje Fila")
tabla_cont_filas <- prop.table(rbind(materias_desempeño1, "Total"=margin.table(materias_desempeño1, 2)),1)*100
tabla_cont_filas <- cbind(tabla_cont_filas, "Total"=margin.table(tabla_cont_filas,1))
round(tabla_cont_filas,1)

print("Tabla contingencia ~ Porcentaje Columna")
tabla_cont_col <- prop.table(cbind(materias_desempeño1, "Total"=margin.table(materias_desempeño1, 1)),2)*100
tabla_cont_col <- rbind(tabla_cont_col, "Total"=margin.table(tabla_cont_col,2))
round(tabla_cont_col,1)


In [ ]:
# Organizamos los datos para hacer un grafico de barras adosadas
tabla_variable_fila <- round(prop.table(table(aux$asignatura, aux$Desempeño), 1)*100,1)
tabla_variable_fila <- t(tabla_variable_fila)
grafico <- barplot(tabla_variable_fila,
                  main= " Grafico de Contigencia por fila",
                  beside= TRUE,
                  space= c(0.1, 1.5),
                  ylim= c(0, 60),
                  cex.axis= 1.2,
                  cex.names= 0.1,
                  cex.main= 1.5,
                  col = c("green","blue", "purple","orange","red"),
                  xlab = "Artes   Cs.Nat   Cs.Soc   Ed.Fis   Ed.Mus   Ed.Tecno  Leng  Mat",
                  cex.lab = 1.2,
                  ylab= "Porcentaje",
                  col.lab= "black")
text(grafico, tabla_variable_fila, labels = tabla_variable_fila, pos = 3, cex = 0.7, col = "black")
legend("topright", rownames(tabla_variable_fila),
       title = "Desempeño",
       title.adj = 0.4,     
       title.col = "black",      
       lty = c(1),
       col = c("green","blue", "purple","orange","red"),
       lwd = 4,
       cex = 1)

# Organizamos los datos para hacer un grafico de barras apiladas
tabla_variable_col <- round(prop.table(table(aux$asignatura, aux$Desempeño), 2) * 100, 1)
grafico <- barplot(tabla_variable_col,
                   main = "Gráfico de Contingencia por columna",
                   beside = FALSE,
                   space = rep(0.25, ncol(tabla_variable_col)),
                   ylim = c(0, 110),
                   cex.axis = 1.2, 
                   cex.names = 1, 
                   cex.main = 1.5, 
                   col = c("green", "red", "blue", "orange", "purple", "cyan", "yellow", "magenta"),
                   xlab = "Desempeños",
                   cex.lab = 1.5,
                   ylab = "Porcentaje",
                   col.lab = "black")
legend(x = 0, y = 115, rownames(tabla_variable_col),cex = 0.5, fill=c("green", "red", "blue", "orange", "purple", "cyan", "yellow", "magenta"),xpd = TRUE,horiz = FALSE,bty = "n")

In [ ]:
# Como muestran los graficos y las tablas, segun el tipo de materia "causa" el desempeño final
#de cada alumno
# Para saber si realmente existe una relación entre las variables aplicaremos el test de chi cuadrado
# H0 ---> No hay relación entre las variables
# H1 ---> Si hay relación entre las variables
chi <-chisq.test(aux$asignatura, aux$Desempeño)
chi

# Vemos la fuerza de asociación
CramerV(aux$asignatura, aux$Desempeño)

In [ ]:
# Frecuencias observadas
chi$observed

# Frecuencias esperadas si las variables fueran independientes
round(chi$expected,0)

# Residuos estandarizados
round(chi$residuals,1)

# Residuos ajustados
round(chi$stdres,1)

In [ ]:
# Análisis Multivariado
# Vector de medias de las variables de primer grado agrupadas por las asignaturas
vector_medias <- Notas_Victoria_6Grado %>% group_by(asignatura) %>% summarise_all(mean)
# Nos quedamos con las materias que nos interesan
vector_medias <- vector_medias %>% filter(asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES" |asignatura == "MATEMATICA" |asignatura == "LENGUA" |asignatura == "EDUCACION MUSICAL" |asignatura == "EDUCACION TECNOLOGICA" |asignatura == "EDUCACION FISICA" |asignatura == "ARTES VISUALES" | asignatura== "CATEQUESIS" | asignatura == "INFORMATICA" |asignatura =="INGLES" )
vector_medias

In [ ]:
# Buscamos valores outliers
aux <- mahalanobis_distance(data= Notas_Victoria_6Grado[,c("NotasPrimerT", "NotasSegundoT", "NotasTercerT", "Promedio")])$is.outlier    
ftable(aux)

In [ ]:
# Grafico de perfiles paralelos para ver el rendimiento de los estudiantes en 11 materias
# Definimos colores manuales para que se distingan
colores <- c("ARTES VISUALES" = "purple","CATEQUESIS" = "orange",
            "CIENCIAS NATURALES" = "green","CIENCIAS SOCIALES" = "blue",
            "EDUCACION FISICA" = "red","EDUCACION MUSICAL" = "turquoise",
            "EDUCACION TECNOLOGICA" = "black","INFORMATICA" = "cyan",
            "INGLES" = "yellow","LENGUA" = "magenta",
            "MATEMATICA" = "skyblue")

# Grafico
ggparcoord(data = vector_medias,
           columns=c(11:14),
           groupColumn = "asignatura",
           showPoints = TRUE,
           scale = "globalminmax") +
  scale_color_manual(values = colores) +
  theme(axis.text.x = element_text(angle = 30, hjust = 1))

In [ ]:
# Prueba de normalidad
mardia2(Notas_Victoria_6Grado[,11:14])
# Los datos no siguen una distribución normal multivariada

In [ ]:
# COMPARACION DE DOS POBLACIONES MULTIVARIADAS INDEPENDIENTES

# Asumimos que las medias de las variables para cada grupo se aproximan a una 
#distribución normal multivariada debido al Teorema del Límite Central Multivariado (TLCM). 
#Esto nos permite aplicar el Test de Hotelling.

# Queremos saber si los vectores entre los tipo de gestión son iguales o no
perfil_medio_tipo <- Notas_Victoria_6Grado %>% group_by(Gestión) %>% summarise_all(mean)

# Grafico de coordenadas paralelas
ggparcoord(data = perfil_medio_tipo,
           columns = 11:14,
           groupColumn = "Gestión",
           showPoints = TRUE,
           scale = "globalminmax")

# Test de hotelling 
# H0 ---> los vectores medios son iguales
# H1 ---> los vectores medios difieren en un grupo
prueba_N<-dplyr::select(Notas_Victoria_6Grado,Gestión,NotasPrimerT, NotasSegundoT, NotasTercerT, Promedio)
fit = hotelling.test(.~Gestión, data = prueba_N)
fit 
# Rechazamos H0, uno de los vectores medios difiere entre los grupos de Gestión

In [ ]:
# Decidimos para que los alumnos esten en condiciones de estar en la bandera, tienen que tener las 8 materias troncales de primaria
#como venimos trabajando anteriormente, ademas de que tienen que tener las 8 si o si, si le falta una nota en estas materias, se lo descarta
abanderados <- distinct(filter(Notas_Victoria_primaria, Condición == "Aprobado", Año_curso == "SEXTO", asignatura %in% c("MATEMATICA", "LENGUA", "CIENCIAS SOCIALES", "CIENCIAS NATURALES", "EDUCACION MUSICAL", "EDUCACION FISICA", "EDUCACION TECNOLOGICA", "ARTES VISUALES")))

# Filtramos los alumnos que tengan todas las materias troncales
abanderados <- abanderados %>%
  group_by(idalumno) %>%
  filter(n() == 8) %>%
  ungroup()

# Calculamos el promedio para cada alumno 
promedios_finales_sexto <- abanderados %>% group_by(idalumno) %>% dplyr::summarise(Promedio_Total = mean(Promedio, na.rm = TRUE)) 

# Unimos el promedio total de cada alumno con los datos originales
datos_bandera_salientes <- inner_join(promedios_finales_sexto, Notas_Victoria_6Grado, by = "idalumno")

# Nos quedamos con el primer registro de cada alumno
datos_bandera_salientes <- datos_bandera_salientes %>%
  group_by(idalumno) %>%
  dplyr::summarise(across(everything(), first))

# Seleccionamos y rendodeamos algunas variables para mejorar el entendimiento del data frame
datos_bandera_salientes <- dplyr::select(datos_bandera_salientes, 'CodigoUnicoEscolar', 'idalumno', 'Documento', 'Gestión','turno', 'Promedio_Total.x')
datos_bandera_salientes <- datos_bandera_salientes %>%
  mutate(Promedio_Total = round(Promedio_Total.x, 2))

# Agrupamos segun el CodigoUnicoEscolar de cada escuela, ademas de que seleccionamos los 6 mejores promedios y los ordenamos
datos_bandera_salientes <- datos_bandera_salientes %>%
  group_by(CodigoUnicoEscolar) %>%
  top_n(6, Promedio_Total) %>%
  arrange(CodigoUnicoEscolar, desc(Promedio_Total))

# Agregamos la posición de la bandera basado en el ranking de Promedio_Total
datos_bandera_salientes <- datos_bandera_salientes %>%
  mutate(Posicion = row_number())
mejores_promedios_sexto <- datos_bandera_salientes %>%
    mutate(Posicion = case_when(
    Posicion == 1 ~ "Abanderado Nacional",
    Posicion == 2 ~ "Abanderado Provincial",
    Posicion == 3 ~ "Escolta derecha nacional",
    Posicion == 4 ~ "Escolta derecha provincial",
    Posicion == 5 ~ "Escolta izquierda nacional",
    Posicion == 6 ~ "Escolta izquierda provincial",
    Posicion == 7 ~ "Primer suplente"))

# Mostramos los abanderados salientes para cada escuela de la base de datos
print("Agradecemos a los abanderados salientes por su compromiso y dedicación al portar con honor las banderas nacionales y provinciales durante el 2023.")
mejores_promedios_sexto
#print(mejores_promedios_sexto, n = 66) 

In [ ]:
ggplot(mejores_promedios_sexto, aes(x = factor(CodigoUnicoEscolar), y = Promedio_Total, group = Posicion, color = Posicion)) +
  geom_line() +
  geom_point() +
  facet_wrap(~CodigoUnicoEscolar, scales = "free_x") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(title = "Comparación de Promedios por Posición en las Banderas",
       x = "Código Único Escolar",
       y = "Promedio Total") +
  theme_minimal() +
  theme(legend.position = "bottom", 
        legend.text = element_text(size = 9),  
        legend.title = element_text(size = 8)) + 
  scale_color_brewer(palette = "Set1")

## Anexo

In [ ]:
# Tabla de frecuencia de los alumnos por su nivel de eximición
print("Tabla de frecuencias de los alumnos y su nivel de eximición")
Fa <- table(Notas_Victoria_primaria_unico$Eximido)
Fr <- prop.table(Fa)
fip <- prop.table(Fa)*100
FA <- cumsum(Fa)
FR <- cumsum(Fr)
tabla_frecuencias_eximido <- round(cbind(Fa, Fr, fip,FA, FR),2)
tabla_frecuencias_eximido

# Gráfico de sectores de alumnos por su nivel de eximición
pie_commute <- round(100* table(Notas_Victoria_primaria_unico$Eximido) / length(Notas_Victoria_primaria_unico$Eximido))
colors = c('#2ca02c', '#ffbf00','#d62728')
pie(pie_commute, labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
    col = colors,
    radius = .90,
    main = "Distribucion Porcentual de la Matricula de 2023
    en el departamento Victoria en el
    nivel de eximición",
    col.main = "black",
    border = 'black',
    cex = 0.75)

In [ ]:
# Test de dos medias independientes
# Queremos ver si los promedios de los alumnos de Primer grado que asisten a una escuela común son mejores que los de la escuela NINAS
datos_comun <- filter(Notas_Victoria_1Grado, ModEnseñanza == "Comun")$Promedio
datos_nina <- filter(Notas_Victoria_1Grado, ModEnseñanza == "Escuelas NINA")$Promedio

# Distribuciones normales?
print("Test de shapiro para los promedios de escuelas primarias comunes: ")
shapiro.test(datos_comun)
print("Test de shapiro para los promedios de escuelas NINAS: ")
shapiro.test(datos_nina)

# Los promedios en los dos grupos de datos no siguen una distribucion normal

#H0: Los promedios de los alumnos de primer grado son iguales en escuelas comunes y escuelas NINAS.
#H1: Los promedios de los alumnos de primer grado en escuelas comunes son mayores que en escuelas NINAS.
wilcox.test(datos_comun,datos_nina, alternative = "greater", conf.level = 0.95)

# Medianas de los datos
print("Mediana de los promedios de escuelas primarias comunes: ")
median(datos_comun)
print("Mediana de los promedios de escuelas NINAS: ")
median(datos_nina)

In [ ]:
# Test de dos medias pareadas
# Queremos ver si las notas de los alumnos de Primer grado son mejores en el tercer trimestre comparado al primer trimestre
# Filtramos los datos
primero_1t <- Notas_Victoria_1Grado$NotasPrimerT
primero_3t <- Notas_Victoria_1Grado$NotasTercerT

# Calculamos las diferencias
diferencias_primero <- primero_3t - primero_1t

# La distribucion de las diferencias es normal?
ks.test(diferencias_primero, "pnorm") #p-value =  2.2e-16

# Las diferencias entre los dos grupos de datos no siguen una distribucion normal

# H0 -> Las notas de los alumnos de primer grado son iguales en el tercer trimestre que en el primer trimestre.
# H1 -> Las notas de los alumnos de primer grado son mayores en el tercer trimestre que en el primer trimestre.
wilcox.test(x=primero_3t, y=primero_1t, alternative = "greater", paired = TRUE, correct = FALSE, conf.level=0.95)

print("Medidas del Primer trimestre")
print(mean(primero_1t))
print(median(primero_1t))
print("Medidas del Tercer trimestre")
print(mean(primero_3t))
print(median(primero_3t))

In [ ]:
# Test de más de dos medias independientes
# ANOVA BIDIRECCIONAL
# Filtramos los datos que vamos a usar
aux <- filter(Notas_Victoria_1Grado, asignatura == "MATEMATICA" | asignatura == "LENGUA" | asignatura == "CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES")

# Filtramos las columnas que tomaremos como factores
datos_materia_gestion <- aux %>%
  mutate(materia = as.factor(asignatura),
         Gestión = as.factor(Gestión))

# Grafico comparativo de los promedios de las materias y tipo de gestión de las escuelas
interaction.plot(x.factor = datos_materia_gestion$asignatura, trace.factor = datos_materia_gestion$Gestión, response = datos_materia_gestion$Promedio,
                 col = c("blue", "red"), lty = 1, lwd = 2,
                 xlab = "Materias", ylab = "Promedio", trace.label = "Gestión")

# Vemos normalidad de los datos
shapiro.test(datos_materia_gestion$Promedio)

# Como los datos no son normales, no se cumple un supuesto del ANOVA BIDIRECCIONAL, entonces el análisis
#no puede seguir adelante

In [ ]:
# Test de más de dos medias independientes
# ANCOVA
# Convertimos la variable explicativa a factor
aux <- Notas_Victoria_1Grado %>%
  mutate(turno = as.factor(turno))

# Grafico comparativo de los promedios en los diferentes turnos y las distintas edades estimadas
interaction.plot(aux$EdadEstimada, aux$turno, aux$Promedio,
                 col = c("red", "blue", "green"), lty = 1, lwd = 2,
                 xlab = "Edad Estimada", ylab = "Promedio", trace.label = "Turno")

# Vemos normalidad de los datos
ks.test(aux$Promedio, "pnorm")

# Como los datos no son normales, no se cumple un supuesto del ANCOVA, entonces el análisis no puede seguir adelante

In [ ]:
# Análisis Bivariado
# Armamos una tabla de contigencia con los tipos de gestiones de las escuelas y la condicion final 
#logrados por los estudiantes en las meterias
condicion_gestion <- Notas_Victoria_1Grado %>%
    select('Gestión', 'Condición') %>%
    table()

# Reordenamos la tabla
condicion_gestion1 <- rbind(condicion_gestion[2,], condicion_gestion[1,])
condicion_gestion1 <- cbind(condicion_gestion1[,1], condicion_gestion1[,2])
colnames(condicion_gestion1) <- c("Aprobado", "Rinde")
rownames(condicion_gestion1) <- c("Pública", "Privada")

print("Tabla con marginales fila y columna")
condicion_gestion2 <- cbind(condicion_gestion1, "Total"= margin.table(condicion_gestion1,1))
condicion_gestion2 <- rbind(condicion_gestion2, "Total"=margin.table(condicion_gestion2,2))
condicion_gestion2

print("Tabla contingencia ~ Porcentaje Total")
tabla_cont_total <- prop.table(condicion_gestion1)*100
tabla_cont_total <- cbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,1))
tabla_cont_total <- rbind(tabla_cont_total, "Total"=margin.table(tabla_cont_total,2))
round(tabla_cont_total,1)

print("Tabla contingencia ~ Porcentaje Fila")
tabla_cont_filas <- prop.table(rbind(condicion_gestion1, "Total"=margin.table(condicion_gestion1, 2)),1)*100
tabla_cont_filas <- cbind(tabla_cont_filas, "Total"=margin.table(tabla_cont_filas,1))
round(tabla_cont_filas,1)

print("Tabla contingencia ~ Porcentaje Columna")
tabla_cont_col <- prop.table(cbind(condicion_gestion1, "Total"=margin.table(condicion_gestion1, 1)),2)*100
tabla_cont_col <- rbind(tabla_cont_col, "Total"=margin.table(tabla_cont_col,2))
round(tabla_cont_col,1)

In [ ]:
# Organizamos los datos para hacer un grafico de barras adosadas
tabla_variable_fila <- round(prop.table(table(Notas_Victoria_1Grado$Gestión, Notas_Victoria_1Grado$Condición), 1)*100,1)
tabla_variable_fila <- t(tabla_variable_fila)
grafico <- barplot(tabla_variable_fila,
                  main= " Grafico de Contigencia por fila",
                  beside= TRUE,
                  space= c(0.1, 2.5),
                  ylim= c(0, 100),
                  cex.axis= 1.2,
                  cex.names= 1.2,
                  cex.main= 1.5,
                  col= c("green", "red"),
                  xlab = "Gestión",
                  cex.lab = 1.5,
                  ylab= "Porcentaje",
                  col.lab= "black")

legend(x = 4.5, y = 100, rownames(tabla_variable_fila),
       title = "Condición",
       title.adj = 0.5,     
       title.col = "black",      
       lty = c(1),
       col= c("green", "red"),
       lwd = 2,
       cex = 1.2) 

# Organizamos los datos para hacer un grafico de barras apiladas
tabla_variable_col <- round(prop.table(table(Notas_Victoria_1Grado$Gestión, Notas_Victoria_1Grado$Condición), 1)*100,1)
tabla_variable_col <- t(tabla_variable_col)
grafico <- barplot(tabla_variable_col,
                  main= "Gráfico de contigencia por columna",
                  beside= FALSE,
                  space = rep(0.25, ncol(tabla_variable_col)),
                  ylim = c(0,110),
                  cex.axis = 1.2, 
                  cex.names=1.5, 
                  cex.main=1.5, 
                  col=c("green", "red"),
                  xlab = "Gestión",
                  cex.lab=1.5,
                  ylab = "Porcentaje",
                  col.lab = "black")

#leyenda
legend(x = 0.8, y = 115, rownames(tabla_variable_col),cex = 1, fill=c("green", "red"),xpd = TRUE,horiz = TRUE,bty = "n")

In [ ]:
# Al parecer según el tipo de gestión de la escuela "causa" la cantidad de materias que los estudiantes
#aprueban o les quedan pendientes
# Para saber si realmente existe una relación entre las variables aplicaremos el test de chi cuadrado
# H0 ---> No hay relación entre las variables
# H1 ---> Si hay relación entre las variables
chi <-chisq.test(Notas_Victoria_1Grado$Gestión, Notas_Victoria_1Grado$Condición)
chi

# Vemos la fuerza de asociación
CramerV(Notas_Victoria_1Grado$Gestión, Notas_Victoria_1Grado$Condición)

In [ ]:
# Frecuencias observadas
chi$observed

# Frecuencias esperadas si las variables fueran independientes
round(chi$expected,0)

# Residuos estandarizados
round(chi$residuals,1)

# Residuos ajustados
round(chi$stdres,1)

In [ ]:
aux <- distinct(Notas_Victoria_1Grado, idalumno, .keep_all = TRUE)
table(aux$Peor_asig)
table(aux$Mejor_asig)

In [ ]:
aux <- distinct(Notas_Victoria_2Grado, idalumno, .keep_all = TRUE)
table(aux$Peor_asig)
table(aux$Mejor_asig)

In [ ]:
aux <- distinct(Notas_Victoria_3Grado, idalumno, .keep_all = TRUE)
table(aux$Peor_asig)
table(aux$Mejor_asig)

In [ ]:
aux <- distinct(Notas_Victoria_4Grado, idalumno, .keep_all = TRUE)
table(aux$Peor_asig)
table(aux$Mejor_asig)

In [ ]:
aux <- distinct(Notas_Victoria_5Grado, idalumno, .keep_all = TRUE)
table(aux$Peor_asig)
table(aux$Mejor_asig)

In [ ]:
aux <- distinct(Notas_Victoria_6Grado, idalumno, .keep_all = TRUE)
table(aux$Peor_asig)
table(aux$Mejor_asig)